# All output have been cleared

# Inverstigation of EDP

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }<\style>"))

In [ ]:
import datetime
print(datetime.datetime.now())

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from __future__ import division
import pandas as pd ; pd.set_option("display.max_columns",200)
import numpy as np
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

In [ ]:
result_path = u"C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Program/Program_only_married_couples/hdf_married_couples/"

In [ ]:
df = pd.read_hdf(result_path+"married_couples_step_last.h5", "fiscr")

In [ ]:
threshold_byr_rbg = pd.DataFrame()
threshold_byr_rbg["2011"] = pd.Series([63233,73516,94451,115185,135941,150684])
threshold_byr_rbg["2014"] = pd.Series([58291,63530,73806,84103,94368,104633])

We define the taxable income (Revenu Brut Global) based on the 7 income categories present in the database.

In [ ]:
household_income_vars = ["ZTSAM", "ZRAGM", "ZRACM","ZRNCM", "ZFONM", "ZPERM","ZRICM"]
for var in household_income_vars:
    df["Delta_{}".format(var)] = df["{}_2014".format(var)] - df["{}_2011".format(var)]

In [ ]:
threshold_byr_rbg = pd.DataFrame()
threshold_byr_rbg["2011"] = pd.Series([63233,73516,94451,115185,135941,150684])
threshold_byr_rbg["2014"] = pd.Series([58291,63530,73806,84103,94368,104633])

In [ ]:
import importlib
year = 2014

ir_2014 = importlib.import_module("Irpp_formulas.IRPP_from_scratch_civil_union")

ir_2011 = importlib.import_module("Irpp_formulas.IRPP_from_scratch_civil_union_{}".format(2011))

dict_gain_ir_2011_by_child = dict()
dict_gain_ir_2014_by_child = dict()
for child in range(1,6):
    ir_x_children = ir_2011.ir_avec_plafond_qf_enfant(pd.Series(
        range(0,2*10**5, 1000)), ir_2014.parts_fiscales_enfants(pd.Series([child]*200).astype("float")))
    ir_0_children = ir_2011.ir_avec_plafond_qf_enfant(pd.Series(
        range(0,2*10**5, 1000)), ir_2011.parts_fiscales_enfants(pd.Series([0.0]*200).astype("float")))
    dict_gain_ir_2011_by_child[child] = ir_0_children - ir_x_children 
    
    ir_x_children = ir_2014.ir_avec_plafond_qf_enfant(pd.Series(
        range(0,2*10**5, 1000)), ir_2014.parts_fiscales_enfants(pd.Series([child]*200).astype("float")))
    ir_0_children = ir_2014.ir_avec_plafond_qf_enfant(pd.Series(
        range(0,2*10**5, 1000)), ir_2014.parts_fiscales_enfants(pd.Series([0.0]*200).astype("float")))
    dict_gain_ir_2014_by_child[child] = ir_0_children - ir_x_children 
    
    
    

plt.figure(figsize=(20,10))
for child in range(1,5):
    plt.plot( pd.Series(range(0,2*10**5, 1000)),dict_gain_ir_2011_by_child[child])
    plt.plot( pd.Series(range(0,2*10**5, 1000)),dict_gain_ir_2014_by_child[child])
for thrsh in threshold_byr_rbg["2011"][0:4]:
    plt.vlines(thrsh, 0 , 16000, linewidth = 0.6, color = "r")
for thrsh in threshold_byr_rbg["2014"][0:4]:
    plt.vlines(thrsh, 0 , 16000, linewidth = 0.6, color = "b")
plt.xlim(50000,160000)

#ir_2011.ir_avec_plafond_qf_enfant(pd.Series(range(0,2*10**5, 1000)), ir.parts_fiscales_enfants(pd.Series([3.0]*200).astype("float"))).plot()

In [ ]:
parts_fiscales_enfants = ir_2014.parts_fiscales_enfants(df.NBPAC_2011)

In [ ]:
df["Irpp_2011"] = ir_2011.ir_avec_plafond_qf_enfant(df.Rbg_2011, parts_fiscales_enfants)
df["Irpp_2014"] = ir_2014.ir_avec_plafond_qf_enfant(df.Rbg_2014, parts_fiscales_enfants)

In [ ]:
df_temp = df.query("(NBPAC_2011 >=1) & (NBPAC_2011==NBPAC_2014) ")
print(df_temp.shape)
fig = plt.figure(figsize=(10,5))
plt.scatter(df_temp.ZIMPOM_2011,df_temp.Irpp_2011, s=0.02 )
plt.xlim(0,15000)
plt.ylim(0,15000)

In [ ]:
df_temp = df.query("(NBPAC_2011 ==0) & (NBPAC_2011==NBPAC_2014) ")
print(df_temp.shape)
fig = plt.figure(figsize=(10,5))
plt.scatter(df_temp.ZIMPOM_DECL_2014,df_temp.Irpp_2014, s=0.02 )
plt.xlim(0,15000)
plt.ylim(0,15000)

In [ ]:
df_temp = df.query("(NBPAC_2011 >=1) & (NBPAC_2011==NBPAC_2014) ")
print(df_temp.shape)
fig = plt.figure(figsize=(10,5))
plt.scatter(df_temp.ZIMPOM_DECL_2014,df_temp.Irpp_2014, s=0.02 )
plt.xlim(0,15000)
plt.ylim(0,15000)

In [ ]:
df_temp = df.query("(NBPAC_2011 >=1) & (NBPAC_2011==NBPAC_2014<4)&(50_000<Rbg_2011<200_000)")
print(df_temp.shape)
fig = plt.figure(figsize=(10,5))
plt.scatter(df_temp.ZIMPOM_DECL_2014,df_temp.Irpp_2014, s=0.002 )
plt.xlim(0,15000)
plt.ylim(0,15000)

## Needed variables

In [ ]:
nb_enf_max_for_annalysis = 5

In [ ]:
child_bool_list = ["child_{}".format(child) for child in  np.arange(1,nb_enf_max_for_annalysis+1)]
cumulative_child_bool_list = ["cumulative_child_{}".format(child) for child in  np.arange(1,nb_enf_max_for_annalysis+1)] 

In [ ]:
all_thresholds = pd.concat([threshold_byr_rbg["2011"], threshold_byr_rbg["2014"]]).sort_values()
thrsh_bool_list = ["Threshold_{}".format(thrsh) for thrsh in  all_thresholds ]

In [ ]:
between_threshold_for_child_rank = ["Between_threshold_{}_child".format(nb_child) for nb_child in range(1,nb_enf_max_for_annalysis+1)]
over_threshold_for_child_rank = ["Over_threshold_{}_child".format(nb_child) for nb_child in range(1,nb_enf_max_for_annalysis+1)]
children_number =  ["child_{}".format(nb_child) for nb_child in range(1,nb_enf_max_for_annalysis+1)]

interaction_child_between_threshold  = ["{}:{}".format(child,interaction) for child, interaction in zip(children_number, between_threshold_for_child_rank)]
interaction_child_over_threshold  = ["{}:{}".format(child,interaction) for child, interaction in zip(children_number, over_threshold_for_child_rank)]

In [ ]:
hardcoded_over = ["Over_{}_X_{}_child".format(child,child) for child in range(1,nb_enf_max_for_annalysis+1)]
hardcoded_between = ["Between_{}_X_{}_child".format(child,child) for child in range(1,nb_enf_max_for_annalysis+1)]

### Income sample selection

In [ ]:
df.shape

In [ ]:
#Nouvelle version ou il faut appartenir à l'intervalle de revenu que sur un seule des deux années
df = df[((
            (df.Rbg_2011>50000)
            &(df.Rbg_2011<200000)
        )|(
            (df.Rbg_2014>50000)
            & (df.Rbg_2014<200000)))]

In [ ]:
nb_enf_max_for_annalysis

In [ ]:
df = df[df.NBPAC_2011<=nb_enf_max_for_annalysis]

In [ ]:
age_max_parents = 62

In [ ]:
print(((df.Age_mother>age_max_parents)|(df.Age_father>age_max_parents)).value_counts()) 
df = df[~((df.Age_mother>age_max_parents)|(df.Age_father>age_max_parents))]

In [ ]:
## On vire les variations extrèmes pour ne pas avoir de revenus négatifs.

In [ ]:
low = 0.025
high = 0.975
quant_df = df.Delta_Rbg.quantile([low,high])
print(quant_df)
df = df[df.Delta_Rbg>quant_df.loc[low]]
df = df[df.Delta_Rbg<quant_df.loc[high]]

In [ ]:
print(df.Rbg_2011.min())
print(df.Rbg_2014.min())
print(df.Rbg_2011.max())
print(df.Rbg_2014.max())


In [ ]:
print((df.Rbg_2011<=1000).value_counts())
print((df.Rbg_2014<=1000).value_counts())

In [ ]:
df.query("-5000<Delta_Rbg<5000").Delta_Rbg.hist(bins = 100)

In [ ]:
for var in household_income_vars:
    plt.figure(figsize=(10,5))
    df.query("(-15000<Delta_{}<15000) &(Delta_{}!=0)".format(var, var))["Delta_{}".format(var)].hist(bins = 100)
    plt.title("Delta_{}".format(var))

In [ ]:
var = "ZPERM"
plt.figure(figsize=(10,5))
for nb_child in range(0,6):
    df.query("(-15000<Delta_{}<65000) &(Delta_{}!=0)&(ZPERM_2011>10000)&(NBPAC_2011==@nb_child)".format(var, var))["Delta_{}".format(var)].hist(bins = 100, label = "{}".format(nb_child))
plt.title("Delta_{}".format(var))
plt.legend()

In [ ]:
var = "ZPERM"
plt.figure(figsize=(10,5))
df.query("(-15000<Delta_{}<65000) &(Delta_{}!=0)&(ZPERM_2011<1000)".format(var, var))["Delta_{}".format(var)].hist(bins = 100)
plt.title("Delta_{}".format(var))

In [ ]:
df.query("(-7000<Delta_Rbg<7000)").Delta_Rbg.hist(bins = 100)

In [ ]:
(df.ZPERM_2014>0).value_counts(normalize = True)

We have 28% of the sample that has retirement income. It creates a mass in the change in income at 2000 euros.

In [ ]:
df.query("(-5000<Delta_Rbg<5000)&(ZPERM_2011<50)").Delta_Rbg.hist(bins = 100)

In [ ]:
df.query("(-5000<Delta_Rbg<5000)&(ZPERM_2014<50)").Delta_Rbg.hist(bins = 100)

In [ ]:
df.query("(-15000<Delta_Rbg<25000)&(ZPERM_2011<50)").Delta_Rbg.hist(bins = 100)

In [ ]:
df.query("(-15000<Delta_Rbg<25000)").Delta_Rbg.hist(bins = 100)

In [ ]:
(df.Rbg_2014==0).value_counts()

In [ ]:
df.query("NBPAC_2011<5").shape

In [ ]:
df["Delta_NBPAC"] = df.NBPAC_2014 - df.NBPAC_2011

In [ ]:
(df.Delta_NBPAC!=0).value_counts(normalize = True)

15% has experienced a birth or a child departure over the 2011,2014 period.  
We will focus our analysis only on households with a fixed family structure.

In [ ]:
df.shape

In [ ]:
df[df.NBPAC_2011 == df.NBPAC_2014].shape

### We only keep family structure children leaving such that households are not in the treatment anymore.
Not in the treatmen anymore:
- One or more children leave household
- Poor enough to be in treatment 0 with that new family situation.

In [ ]:
#df = df[df.NBPAC_2011 >= df.NBPAC_2014]

In [ ]:
df[df.NBPAC_2011 >= df.NBPAC_2014].Age_eldest_child.hist(bins = 100)

In [ ]:
df.Age_eldest_child

In [ ]:
threshold_byr_rbg = pd.DataFrame()
threshold_byr_rbg["2011"] = pd.Series([63233,73516,94451,115185,135941,150684])
threshold_byr_rbg["2014"] = pd.Series([58291,63530,73806,84103,94368,104633])

# Canonical (in article)

In [ ]:
df["Rbg_splines"]  = pd.qcut(df.Rbg_2011, 30)

In [ ]:
no_attached = (df.TYPE_FISC_2014.str.count("[3-9]")<1)&(df.TYPE_FISC_2011.str.count("[3-9]")<1)
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)&(@no_attached) ")) #SAME NBPAC
fit = m.fit()
summary = fit.summary(); print(summary)


In [ ]:
no_attached = (df.TYPE_FISC_2014.str.count("[3-9]")<1)&(df.TYPE_FISC_2011.str.count("[3-9]")<1)
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_2011_2014_between_threshold +interaction_child_2011_2014_over_threshold_2014))
                    , df.query("((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)&(@no_attached) ")) #SAME NBPAC
fit = m.fit()
summary = fit.summary(); print(summary)


In [ ]:
no_attached = (df.TYPE_FISC_2014.str.count("[3-9]")<1)&(df.TYPE_FISC_2011.str.count("[3-9]")<1)
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)&(@no_attached) ").sample(n = 11971)) #SAME NBPAC
fit = m.fit()
summary = fit.summary(); print(summary)


In [ ]:
interactions_rbg_between = ["Interactions_rbg_child{}_between".format(child) for child in range(1,6)]
interactions_rbg_over = ["Interactions_rbg_child{}_over".format(child) for child in range(1,6)]
interactions_child_over = ["Interactions_child{}_over".format(child) for child in range(1,6)]
np.random.seed(42)

df_for_zimpom =  df.sample(n = 50_000).query("((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))&(Age_father>40)"+
                              "&(NBPAC_2011 == NBPAC_2014)&(@no_attached) ")
regression_delta_zimpom_formula = "(Delta_ZIMPOM)~   {} - 1 + Not_touched_by_reform + Delta_Rbg_times_not_touched_reform".format(
        ' + '.join( interactions_child_over+interactions_rbg_over+interactions_rbg_between))
m = sm.OLS.from_formula(regression_delta_zimpom_formula
                    ,df_for_zimpom)#SAME NBPAC
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
(df_for_zimpom["Interactions_rbg_child1_between"] * 0.3).shape

In [ ]:
np.array([1,2,4,6,8])*836

In [ ]:
fit.params[:5] - np.array([1,2,4,5,8])*836

In [ ]:
no_attached = (df.TYPE_FISC_2014.str.count("[3-9]")<1)&(df.TYPE_FISC_2011.str.count("[3-9]")<1)
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)&(@no_attached) ")) #SAME NBPAC
fit = m.fit()
summary = fit.summary(); print(summary)


# With dependent children

In [ ]:
no_attached = (df.TYPE_FISC_2014.str.count("[3-9]")<1)&(df.TYPE_FISC_2011.str.count("[3-9]")<1)
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014) ")) #SAME NBPAC
fit = m.fit()
summary = fit.summary(); print(summary)


# Child(ren) has(ve) left the household

In [ ]:

m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 > NBPAC_2014) ")) #SAME NBPAC
fit = m.fit()
summary = fit.summary(); print(summary)


In [ ]:

m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 < NBPAC_2014) ")) #SAME NBPAC
fit = m.fit()
summary = fit.summary(); print(summary)


In [ ]:
for child in range(0,7):
    df["child_{}_2014".format(child)] = df.NBPAC_2014 == child
between_threshold_for_child_rank = ["Between_threshold_{}_child".format(nb_child) for nb_child in range(1,nb_enf_max_for_annalysis+1)]
over_threshold_for_child_rank = ["Over_threshold_{}_child".format(nb_child) for nb_child in range(1,nb_enf_max_for_annalysis+1)]
children_number_2014 =  ["child_{}_2014".format(nb_child) for nb_child in range(1,nb_enf_max_for_annalysis+1)]

interaction_child_between_threshold_2014  = ["{}_2014:{}".format(child,interaction) for child, interaction in zip(children_number, between_threshold_for_child_rank)]
interaction_child_over_threshold_2014  = ["{}_2014:{}".format(child,interaction) for child, interaction in zip(children_number, over_threshold_for_child_rank)]

In [ ]:
interaction_child_2011_2014_between_threshold  = ["{}:{}".format(child,interaction) for child, interaction in zip(children_number, hardcoded_between_2014)]
interaction_child_2011_2014_over_threshold_2014  = ["{}:{}".format(child,interaction) for child, interaction in zip(children_number, hardcoded_over_2014)]

In [ ]:
hardcoded_over_2014 = ["Over_{}_X_{}_child_2014".format(child,child) for child in range(1,nb_enf_max_for_annalysis+1)]
hardcoded_between_2014 = ["Between_{}_X_{}_child_2014".format(child,child) for child in range(1,nb_enf_max_for_annalysis+1)]

In [ ]:
hardcoded_over_2011_child = ["Over_{}_X_{}_child_2014".format(child,child) for child in range(1,nb_enf_max_for_annalysis+1)]
hardcoded_between_2014 = ["Between_{}_X_{}_child_2014".format(child,child) for child in range(1,nb_enf_max_for_annalysis+1)]

In [ ]:
## Interactions hardcoded
for child in range(1,7):
    df["Over_{}_X_{}_child_2014".format(child,child)] = df["Over_threshold_{}_child".format(child)] * (df["NBPAC_2014"]==child)
    df["Between_{}_X_{}_child_2014".format(child,child)] = df["Between_threshold_{}_child".format(child)] * (df["NBPAC_2014"]==child)

In [ ]:
no_attached = (df.TYPE_FISC_2014.str.count("[3-9]")<1)&(df.TYPE_FISC_2011.str.count("[3-9]")<1)
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number #below_threshold_for_child_rank+interaction_child_below_thresholds
                  +hardcoded_over_2014+hardcoded_between_2014+over_threshold_for_child_rank+between_threshold_for_child_rank))
                    , df.query("((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 >= NBPAC_2014) ").sample(n = 12000)) #SAME NBPAC
fit = m.fit()
summary = fit.summary(); print(summary)


In [ ]:
df.Rbg_2014.hist(bins = 900)

In [ ]:

df.Rbg_2011.hist(bins = 900)

In [ ]:
print((df.Rbg_2011 == 0).value_counts(normalize = True))
print((df.Rbg_2014 == 0).value_counts(normalize = True))

In [ ]:
range(nb_enf_max_for_annalysis+1)

In [ ]:
%%time
df = df[df.NBPAC_2011.isin(range(nb_enf_max_for_annalysis+1))]#,6,7])]
#df = df[df.AGERF_2011<50]

In [ ]:
df.shape

In [ ]:
import statsmodels.api as sm
df["Rbg_splines"]  = pd.cut(df.Rbg_2011, 10)

In [ ]:
df = df[(df.NBPAC_2011 == df.NBPAC_2014)]

### Stat Des

In [ ]:
stat_des_variables = ["Rbg_2011","Rbg_2014","Delta_rbg","Age_father", "Age_mother","NBPAC_2011" , "Age_youngest_child", "Age_eldest_child"]

In [ ]:
df[stat_des_variables].describe().loc["mean"]

In [ ]:
df[stat_des_variables].query("NBPAC_2011==1").describe().loc["mean"]

In [ ]:
df[stat_des_variables].query("NBPAC_2011==2").describe().loc["mean"]

In [ ]:
df[stat_des_variables].query("NBPAC_2011==0").describe().loc["mean"]

In [ ]:
### TODO: faire un multi-index avec all , 0 child, 1 ,2,3,4 and more child en colones et les variables en ignes.
stat_des_df = pd.DataFrame()

In [ ]:
df.Rbg_2014.hist(bins = 100, alpha = 0.5, label = "2014")
df.Rbg_2011.hist(bins = 100,alpha = 0.5, label = "2011")
plt.legend()

In [ ]:
import seaborn as sns
sns.distplot(df.Rbg_2014, color = "r")
sns.distplot(df.Rbg_2011)

In [ ]:
threshold_byr_rbg = pd.DataFrame()
threshold_byr_rbg["2011"] = pd.Series([63233,73516,94451,115185,135941,150684])
threshold_byr_rbg["2014"] = pd.Series([58291,63530,73806,84103,94368,104633])
threshold_byr_rbg.index = range(1,7); threshold_byr_rbg.index.name = 'nb_enf'
threshold_byr_rbg




df["Not_touched_by_reform"] = (df.NBPAC_2011 == 0).astype("int")
for nb_enf in range(1,7): ### in modification
    df["Not_touched_by_reform".format(nb_enf)
          ] +=((df.Rbg_2011<  threshold_byr_rbg["2014"].iloc[nb_enf-1])
                *( df.NBPAC_2011 == nb_enf)).astype("float")

In [ ]:
df["Distance_to_treatment_2"]  = 0
for child in range(1,nb_enf_max_for_annalysis+1):
    df["Distance_to_treatment_2"] += df["Over_threshold_{}_child".format(child)] * df[child_bool_list[child-1]] * ((threshold_byr_rbg["2011"].iloc[child-1]) - df.Rbg_2014)

In [ ]:
df.Not_touched_by_reform.value_counts()

In [ ]:
df_desc_stat = pd.DataFrame()

for child in  range(0, nb_enf_max_for_annalysis+1):
        df_desc_stat.loc[child, "Treatment_0"] = df.query(
                                            "NBPAC_2011=={}".format(child)
                                            )["Not_touched_by_reform".format(child)].value_counts()[True]


for child in  range(1, nb_enf_max_for_annalysis+1):
    df_desc_stat.loc[child, "Treatment_1"] = df.query(
                                            "NBPAC_2011=={}".format(child)
                                            )["Between_threshold_{}_child".format(child)].value_counts()[True]
    df_desc_stat.loc[child, "Treatment_2"] = df.query(
                                            "NBPAC_2011=={}".format(child)
                                            )["Over_threshold_{}_child".format(child)].value_counts()[True]
df_desc_stat["Total"] = df_desc_stat.sum(axis = 1)
df_desc_stat.loc["Total"] = df_desc_stat.sum(axis = 0)

In [ ]:
df_desc_stat

#### Still in the same treatment span

In [ ]:

df["Rbg2014_below_threshold_{}_child".format(0)
          ] =  True
for nb_enf in range(1,7): ### in modification
    df["Rbg2014_Over_threshold_{}_child".format(nb_enf)
          ] =  (df.Rbg_2014 > threshold_byr_rbg["2011"].iloc[nb_enf-1])
    df["Rbg2014_Between_threshold_{}_child".format(nb_enf)
          ] =  (df.Rbg_2014 > threshold_byr_rbg["2014"].iloc[nb_enf-1]) & (df.Rbg_2014 < threshold_byr_rbg["2011"].iloc[nb_enf-1])
    df["Rbg2014_below_threshold_{}_child".format(nb_enf)
          ] =  (df.Rbg_2014 < threshold_byr_rbg["2014"].iloc[nb_enf-1])



In [ ]:
df_desc_stat_2014 = pd.DataFrame()

for child in  [0]:
        df_desc_stat_2014.loc[child, "Treatment_0"] = df.query(
                                            "NBPAC_2011=={}".format(child)
                                            )["Rbg2014_below_threshold_{}_child".format(child)].value_counts()[True]



for child in  range(1, nb_enf_max_for_annalysis+1):
    df_desc_stat_2014.loc[child, "Treatment_1"] = df.query(
                                            "NBPAC_2011=={}".format(child)
                                            )["Rbg2014_Between_threshold_{}_child".format(child)].value_counts()[True]
    df_desc_stat_2014.loc[child, "Treatment_2"] = df.query(
                                            "NBPAC_2011=={}".format(child)
                                            )["Rbg2014_Over_threshold_{}_child".format(child)].value_counts()[True]
    
    df_desc_stat_2014.loc[child, "Treatment_0"] = df.query(
                                            "NBPAC_2011=={}".format(child)
                                            )["Rbg2014_below_threshold_{}_child".format(child)].value_counts()[True]
    
    
    
df_desc_stat_2014["Total"] = df_desc_stat_2014.sum(axis = 1)

In [ ]:
df_desc_stat_2014

#### Transitions 

In [ ]:
df_desc_stat = pd.DataFrame()

for child in  [0]:
    df_desc_stat.loc[child, "Treatment_0"] = df.query(
                                            "NBPAC_2011=={}".format(child)
                                            )["Rbg2014_below_threshold_{}_child".format(child)].value_counts()[True]



for child in  range(1, nb_enf_max_for_annalysis+1):
    df_desc_stat.loc[child, "Treatment_1"] = df.query(
                                            "NBPAC_2011=={}".format(child)
                                            )["Rbg2014_Between_threshold_{}_child".format(child)].value_counts()[True]
    df_desc_stat.loc[child, "Treatment_2"] = df.query(
                                            "NBPAC_2011=={}".format(child)
                                            )["Rbg2014_Over_threshold_{}_child".format(child)].value_counts()[True]
    
    df_desc_stat.loc[child, "Treatment_0"] = df.query(
                                            "NBPAC_2011=={}".format(child)
                                            )["Rbg2014_below_threshold_{}_child".format(child)].value_counts()[True]
    
    
df_desc_stat["Total"] = df_desc_stat.sum(axis = 1)

In [ ]:
df_desc_stat

In [ ]:
df_temp_transition =df

In [ ]:
df_temp_transition["Treatment_0_2014"] = False * df.shape[0]
df_temp_transition["Treatment_0_2011"] = False * df.shape[0]


df_temp_transition["Treatment_1_2014"] = False * df.shape[0]
df_temp_transition["Treatment_1_2011"] = False * df.shape[0]

df_temp_transition["Treatment_2_2014"] = False * df.shape[0]
df_temp_transition["Treatment_2_2011"] = False * df.shape[0]



for child in range(1,7):
    df_temp_transition["Treatment_1_2014"] += (df.NBPAC_2011 == child) * df["Rbg2014_Between_threshold_{}_child".format(child)]
    df_temp_transition["Treatment_1_2011"] += (df.NBPAC_2011 == child) * df["Between_threshold_{}_child".format(child)]
    
    df_temp_transition["Treatment_2_2014"] += (df.NBPAC_2011 == child) * df["Rbg2014_Over_threshold_{}_child".format(child)]
    df_temp_transition["Treatment_2_2011"] += (df.NBPAC_2011 == child) * df["Over_threshold_{}_child".format(child)]
    



df_temp_transition["Treatment_1_2011"] = df_temp_transition["Treatment_1_2011"].astype("bool")
df_temp_transition["Treatment_2_2011"] = df_temp_transition["Treatment_2_2011"].astype("bool")
df_temp_transition["Treatment_1_2014"] = df_temp_transition["Treatment_1_2014"].astype("bool")
df_temp_transition["Treatment_2_2014"] = df_temp_transition["Treatment_2_2014"].astype("bool")
df_temp_transition["Treatment_0_2011"] = ~(df_temp_transition.Treatment_1_2011 |df_temp_transition.Treatment_2_2011)
df_temp_transition["Treatment_0_2014"] = ~(df_temp_transition.Treatment_1_2014 |df_temp_transition.Treatment_2_2014)

In [ ]:
df_temp_transition.Treatment_0_2011.value_counts()

In [ ]:
df_temp_transition.shape

In [ ]:

print((df_temp_transition.Treatment_1_2011 * df_temp_transition.Treatment_1_2014).value_counts()[1])
print((df_temp_transition.Treatment_2_2011 * df_temp_transition.Treatment_2_2014).value_counts()[1])
print((df_temp_transition.Treatment_0_2011 * df_temp_transition.Treatment_0_2014).value_counts()[1])
print("*"*12)
print ((df_temp_transition.Treatment_1_2011 * df_temp_transition.Treatment_2_2014).value_counts()[1])
print ((df_temp_transition.Treatment_2_2011 * df_temp_transition.Treatment_1_2014).value_counts()[1])
print("*"*12)
print((df_temp_transition.Treatment_0_2011 * df_temp_transition.Treatment_1_2014).value_counts()[1])
print((df_temp_transition.Treatment_0_2011 * df_temp_transition.Treatment_2_2014).value_counts()[1])
print((df_temp_transition.Treatment_1_2011 * df_temp_transition.Treatment_0_2014).value_counts()[1])
print((df_temp_transition.Treatment_2_2011 * df_temp_transition.Treatment_0_2014).value_counts()[1])

In [ ]:

print ((df_temp_transition.Treatment_1_2011 * df_temp_transition.Treatment_1_2014).value_counts()[1]/df_temp_transition.shape[0])
print ((df_temp_transition.Treatment_2_2011 * df_temp_transition.Treatment_2_2014).value_counts()[1]/df_temp_transition.shape[0])
print ((df_temp_transition.Treatment_0_2011 * df_temp_transition.Treatment_0_2014).value_counts()[1]/df_temp_transition.shape[0])
print("*"*12)
print ((df_temp_transition.Treatment_1_2011 * df_temp_transition.Treatment_2_2014).value_counts()[1]/df_temp_transition.shape[0])
print ((df_temp_transition.Treatment_2_2011 * df_temp_transition.Treatment_1_2014).value_counts()[1]/df_temp_transition.shape[0])
print("*"*12)
print ((df_temp_transition.Treatment_0_2011 * df_temp_transition.Treatment_1_2014).value_counts()[1]/df_temp_transition.shape[0])
print ((df_temp_transition.Treatment_0_2011 * df_temp_transition.Treatment_2_2014).value_counts()[1]/df_temp_transition.shape[0])
print ((df_temp_transition.Treatment_1_2011 * df_temp_transition.Treatment_0_2014).value_counts()[1]/df_temp_transition.shape[0])
print ((df_temp_transition.Treatment_2_2011 * df_temp_transition.Treatment_0_2014).value_counts()[1]/df_temp_transition.shape[0])

In [ ]:
df_desc_transition = pd.DataFrame()
df_desc_transition.loc["Treatment_0","Treatment_0" ] = (df_temp_transition.Treatment_0_2011 * df_temp_transition.Treatment_0_2014).value_counts()[True]
df_desc_transition.loc["Treatment_0","Treatment_1" ] = (df_temp_transition.Treatment_0_2011 * df_temp_transition.Treatment_1_2014).value_counts()[True]
df_desc_transition.loc["Treatment_0","Treatment_2" ] = (df_temp_transition.Treatment_0_2011 * df_temp_transition.Treatment_2_2014).value_counts()[True]



df_desc_transition.loc["Treatment_1","Treatment_0" ] = (df_temp_transition.Treatment_1_2011 * df_temp_transition.Treatment_0_2014).value_counts()[True]
df_desc_transition.loc["Treatment_1","Treatment_1" ] = (df_temp_transition.Treatment_1_2011 * df_temp_transition.Treatment_1_2014).value_counts()[True]
df_desc_transition.loc["Treatment_1","Treatment_2" ] = (df_temp_transition.Treatment_1_2011 * df_temp_transition.Treatment_2_2014).value_counts()[True]



df_desc_transition.loc["Treatment_2","Treatment_0" ] = (df_temp_transition.Treatment_2_2011 * df_temp_transition.Treatment_0_2014).value_counts()[True]
df_desc_transition.loc["Treatment_2","Treatment_1" ] = (df_temp_transition.Treatment_2_2011 * df_temp_transition.Treatment_1_2014).value_counts()[True]
df_desc_transition.loc["Treatment_2","Treatment_2" ] = (df_temp_transition.Treatment_2_2011 * df_temp_transition.Treatment_2_2014).value_counts()[True]


display(df_desc_transition)
display(df_desc_transition/df_temp_transition.shape[0])




In [ ]:
df_desc_transition/df_desc_transition.sum(0)

In [ ]:
for nb_child in range(0,7):
    df_desc_transition = pd.DataFrame()
    df_temp = df_temp_transition.query("(NBPAC_2011==NBPAC_2014)& (NBPAC_2011 ==@nb_child)")
    df_desc_transition.loc["Treatment_0","Treatment_0" ] = (df_temp.Treatment_0_2011 * df_temp.Treatment_0_2014).sum(axis=0)
    df_desc_transition.loc["Treatment_0","Treatment_1" ] = (df_temp.Treatment_0_2011 * df_temp.Treatment_1_2014).sum(axis=0)
    df_desc_transition.loc["Treatment_0","Treatment_2" ] = (df_temp.Treatment_0_2011 * df_temp.Treatment_2_2014).sum(axis=0)
    df_desc_transition.loc["Treatment_1","Treatment_0" ] = (df_temp.Treatment_1_2011 * df_temp.Treatment_0_2014).sum(axis=0)
    df_desc_transition.loc["Treatment_1","Treatment_1" ] = (df_temp.Treatment_1_2011 * df_temp.Treatment_1_2014).sum(axis=0)
    df_desc_transition.loc["Treatment_1","Treatment_2" ] = (df_temp.Treatment_1_2011 * df_temp.Treatment_2_2014).sum(axis=0)
    df_desc_transition.loc["Treatment_2","Treatment_0" ] = (df_temp.Treatment_2_2011 * df_temp.Treatment_0_2014).sum(axis=0)
    df_desc_transition.loc["Treatment_2","Treatment_1" ] = (df_temp.Treatment_2_2011 * df_temp.Treatment_1_2014).sum(axis=0)
    df_desc_transition.loc["Treatment_2","Treatment_2" ] = (df_temp.Treatment_2_2011 * df_temp.Treatment_2_2014).sum(axis=0)

    print("*"*9+"{} child".format(nb_child)+"*"*9)
    print("Headcount")
    display(df_desc_transition)
    print("Share")
    display((df_desc_transition.T/df_desc_transition.sum(1)).T)




### Common trend assumption

In [ ]:
df.query("-50000<Delta_Rbg<50000").Delta_Rbg.hist(bins = 100)

In [ ]:
import seaborn as sns

We see that there is something not very smoooth happenning for households without children:

In [ ]:
for child in range(0,6):
    sns.distplot(df.query("(-50000<Delta_rbg<50000)&(NBPAC_2011=={})".format(child)).Delta_rbg, hist= False, label = child)
plt.xlim(-10000,20000)

In [ ]:
df.shape

When getting rid of households with an income tax equal to 0 in 2011, the curve is a lot smoother.
The average change in taxable income is way nearer to the average change for one children than the one we observe when we take out the households which does not have retirement income.

In [ ]:
import seaborn as sns

In [ ]:
for child in range(0,6):
    sns.distplot(df.query("(-50000<Delta_rbg<50000)&(NBPAC_2011=={})& (ZPERM_2011==0)".format(child)).Delta_rbg, hist= False, label = child)
plt.xlim(-10000,20000)

In [ ]:
df["Below_thresholds"] = ~(df["Between_thresholds"] | df["Over_threshold"])

In [ ]:
print(df.query("(NBPAC_2011==2)&(Between_thresholds)").shape)
print(df.query("(NBPAC_2011==2)&(Over_threshold)").shape)

In [ ]:
plt.figure(figsize=(20,10))
sns.distplot(df.query("(-50000<Delta_rbg<50000)&(NBPAC_2011=={})& (ZPERM_2011==0)& Between_threshold_2_child".format(3)).Delta_rbg.sample(3000), hist= False, label = "between 2 child with 3  child")
for child in [2]:
    sns.distplot(df.query("(-50000<Delta_rbg<50000)&(NBPAC_2011=={})& (ZPERM_2011==0)& Below_thresholds".format(child)).Delta_rbg.sample(3000), hist= False, label = "below 2 child")
    sns.distplot(df.query("(-50000<Delta_rbg<50000)&(NBPAC_2011=={})& (ZPERM_2011==0)& Between_thresholds".format(child)).Delta_rbg.sample(3000), hist= False, label = "between 2 child")
    sns.distplot(df.query("(-50000<Delta_rbg<50000)&(NBPAC_2011=={})& (ZPERM_2011==0)& Over_threshold & ~(Over_threshold_3_child)".format(child)).Delta_rbg.sample(3000), hist= False, label = "over 2 child")
#plt.xlim(-10000,20000)

In [ ]:
for child in range(0,6):
    sns.distplot(df.query("(-50000<Delta_rbg<50000)&(NBPAC_2011=={})& (ZPERM_2014==0)".format(child)).Delta_rbg, hist= False, label = child)
plt.xlim(-10000,20000)

In [ ]:
for child in range(0,6):
    sns.distplot(df.query("(-50000<Delta_rbg<50000)&(NBPAC_2011=={})& (ZPERM_2014==0)& (Treatment_0_2011)".format(child)).Delta_rbg, hist= False, label = child)
plt.xlim(-10000,20000)

In [ ]:
sns.distplot(df.query("(-50000<Delta_rbg<50000)& (Rbg_2014>58291) & (ZPERM_2014==0)& (Treatment_0_2011)".format(child)).Delta_rbg, hist= False, label = child)
sns.distplot(df.query("(-50000<Delta_rbg<50000)& (Rbg_2014>58291) & (ZPERM_2014==0)& (Treatment_1_2011)".format(child)).Delta_rbg, hist= False, label = child)
plt.xlim(-10000,20000)

In [ ]:
sns.distplot(df.query("(-50000<Delta_rbg<50000)& (Rbg_2014>58291) & (ZPERM_2014==0)& (Treatment_0_2011)&(NBPAC_2011>0)".format(child)).Delta_rbg, hist= False, label = 'Not treated')
sns.distplot(df.query("(-50000<Delta_rbg<50000)& (Rbg_2014>58291) & (ZPERM_2014==0)& (Treatment_1_2011)".format(child)).Delta_rbg, hist= False, label = "Treatment 1")
sns.distplot(df.query("(-50000<Delta_rbg<50000)& (Rbg_2014>58291) & (ZPERM_2014==0)& (Treatment_2_2011)".format(child)).Delta_rbg, hist= False, label = "Treatment 2")
plt.xlim(-10000,20000)

In [ ]:
sns.distplot(df.query("(-50000<Delta_rbg<50000)& (Rbg_2014>58291) & (ZPERM_2014==0)& (Treatment_0_2011)&(NBPAC_2011>0)".format(child)).Delta_rbg, hist= False, label = 'Not treated')
sns.distplot(df.query("(-50000<Delta_rbg<50000)& (Rbg_2014>58291) & (ZPERM_2014==0)& (Treatment_1_2011)".format(child)).Delta_rbg, hist= False, label = "Treatment 1")
sns.distplot(df.query("(-50000<Delta_rbg<50000)& (Rbg_2014>58291) & (ZPERM_2014==0)& (Treatment_2_2011)".format(child)).Delta_rbg, hist= False, label = "Treatment 2")
plt.xlim(-10000,20000)

In [ ]:
sns.distplot(df.query("(-50000<Delta_rbg<50000)& (Rbg_2014>58291) & (ZPERM_2014==0)& (Treatment_0_2011)&(NBPAC_2011>0)".format(child)).Delta_rbg, hist= False, label = 'Not treated')
sns.distplot(df.query("(-50000<Delta_rbg<50000)& (Rbg_2014>58291) & (ZPERM_2014==0)& (Treatment_0_2011)".format(child)).Delta_rbg, hist= False, label = "Treatment 1")
plt.xlim(-10000,20000)

In [ ]:
threshold_byr_rbg

In [ ]:
for child in range(0,7):
    df["child_{}".format(child)] = df.NBPAC_2011 == child

In [ ]:
print(df.query("NBPAC_2011==0").Delta_Rbg.mean())
print(df.query("child_1==True").Delta_Rbg.mean())
print(df.query("child_2==True").Delta_Rbg.mean())
print(df.query("child_3==True").Delta_Rbg.mean())
print(df.query("child_4==True").Delta_Rbg.mean())
print(df.query("child_5==True").Delta_Rbg.mean())
print(df.query("child_6==True").Delta_Rbg.mean())

In [ ]:
print(df.query("(Between_threshold_1_child==True)").Delta_Rbg.mean())
print(df.query("Between_threshold_2_child==True").Delta_Rbg.mean())
print(df.query("Between_threshold_3_child==True").Delta_Rbg.mean())
print(df.query("Between_threshold_4_child==True").Delta_Rbg.mean())
print(df.query("Between_threshold_5_child==True").Delta_Rbg.mean())

In [ ]:
df["child_0"] = df.NBPAC_2011==0

In [ ]:
df_delta_trimmed = df.query("(-50000<Delta_rbg<50000)")

In [ ]:
df_delta_trimmed.query("NBPAC_2011==3").Rbg_2011.hist(bins = 1000)


## Double diff 

### Pooled 

In [ ]:
m = sm.RLM.from_formula("(Delta_rbg)~   Over_threshold+ Between_thresholds" 
                     
                                                                                            , df)
fit = m.fit()


In [ ]:
fit.summary2()

In [ ]:
m = sm.RLM.from_formula("(Delta_rbg)~   Over_threshold+ Between_thresholds" 
                     
                                                                                            , df.query("NBPAC_2011 > 0".format(child)))
fit = m.fit()
fit.summary2()

### By child rank 

In [ ]:
df.shape

In [ ]:
 df_temp.query("NBPAC_2011 == {}".format(1))

In [ ]:
df = df
result_fit_list = list()
for child in range(1,nb_enf_max_for_annalysis+1):
    m = sm.RLM.from_formula("(Delta_rbg)~   Over_threshold+ Between_thresholds" 
                     
                                                                                            , df.query("NBPAC_2011 == {}".format(child)))
    fit = m.fit()
    result_fit_list.append(fit)
summary_col_result = summary_col(result_fit_list, stars = True)

In [ ]:
print(summary_col(result_fit_list,model_names =["{} children".format(nb_child) for nb_child in range(1,7)], stars = True))

#### Double diff with no child family as control

In [ ]:
df_temp = df.query("NBPAC_2011==NBPAC_2014").dropna(subset = ["Delta_rbg", "Delta_Rbg"]+hardcoded_over+hardcoded_between+over_threshold_for_child_rank+between_threshold_for_child_rank)
result_fit_list = list()
for child in range(1,nb_enf_max_for_annalysis+1):
    df_temp2 = df_temp.query("((NBPAC_2011 == {})|(NBPAC_2011 == 0))& (Rbg_2011>{})".format(child,threshold_byr_rbg.loc[child,"2014"]) )
    df_temp2["Between_X_child"] = df_temp2["Between_{}_X_{}_child".format(child,child)]
    df_temp2["Over_X_child"] = df_temp2["Over_{}_X_{}_child".format(child,child)]
    df_temp2["Between_threshold_n_child"] = df_temp2["Between_threshold_{}_child".format(child)]
    df_temp2["Over_threshold_n_child"] = df_temp2["Over_threshold_{}_child".format(child)]
    m = sm.OLS(df_temp2["Delta_Rbg"].astype("float"), df_temp2[["Between_X_child", "Over_X_child", "Intercept"]].astype("float"))
    fit = m.fit()
    result_fit_list.append(fit)
from statsmodels.iolib.summary2 import summary_col
print(summary_col(result_fit_list, stars = True))

In [ ]:
condition_rbg = "True" #TODO: enelever le condition_rbg des cellules, True pour que ce soit neutre.

In [ ]:
df_temp = df.query("NBPAC_2011==NBPAC_2014&"+condition_rbg).dropna(subset = ["Delta_rbg", "Delta_Rbg"]+hardcoded_over+hardcoded_between+over_threshold_for_child_rank+between_threshold_for_child_rank)
result_fit_list = list()
for child in range(1,7):
    df_temp2 = df_temp.query("((NBPAC_2011 == {})|(NBPAC_2011 == 0))& (Rbg_2011>{})".format(child,threshold_byr_rbg.loc[child,"2014"]) )
    df_temp2["Between_X_child"] = df_temp2["Between_{}_X_{}_child".format(child,child)]
    df_temp2["Over_X_child"] = df_temp2["Over_{}_X_{}_child".format(child,child)]
    df_temp2["Between_threshold_n_child"] = df_temp2["Between_threshold_{}_child".format(child)]
    df_temp2["Over_threshold_n_child"] = df_temp2["Over_threshold_{}_child".format(child)]
    m = sm.OLS(df_temp2["Delta_Rbg"].astype("float"), df_temp2[["Between_X_child", "Over_X_child", "Intercept"]].astype("float"))
    fit = m.fit()
    result_fit_list.append(fit)
from statsmodels.iolib.summary2 import summary_col
print(summary_col(result_fit_list, stars = True))

In [ ]:
no_attached = (df.TYPE_FISC_2014.str.count("[3-9]")<1)&(df.TYPE_FISC_2011.str.count("[3-9]")<1)

In [ ]:
#Colinear if we put an intercept
df_temp = df.query("NBPAC_2011==NBPAC_2014&"+condition_rbg+"&(@no_attached)").dropna(subset = ["Delta_rbg", "Delta_Rbg"]+hardcoded_over+hardcoded_between+over_threshold_for_child_rank+between_threshold_for_child_rank)
result_fit_list = list()
for child in range(1,7):
    df_temp2 = df_temp.query("((NBPAC_2011 == {})|(NBPAC_2011 == 0))& (Rbg_2011>{})".format(child,threshold_byr_rbg.loc[child,"2014"]) )
    df_temp2["Between_X_child"] = df_temp2["Between_{}_X_{}_child".format(child,child)]
    df_temp2["Over_X_child"] = df_temp2["Over_{}_X_{}_child".format(child,child)]
    df_temp2["Between_threshold_n_child"] = df_temp2["Between_threshold_{}_child".format(child)]
    df_temp2["Over_threshold_n_child"] = df_temp2["Over_threshold_{}_child".format(child)]
    m = sm.OLS(df_temp2["Delta_Rbg"].astype("float"), df_temp2[["Between_X_child", "Over_X_child","Between_threshold_n_child","Over_threshold_n_child"]].astype("float"))
    fit = m.fit()
    result_fit_list.append(fit)
from statsmodels.iolib.summary2 import summary_col
print(summary_col(result_fit_list, stars = True))

In [ ]:
df_temp2.query("NBPAC_2011==0")[["Between_threshold_1_child"]].Between_threshold_1_child.value_counts()

In [ ]:
from statsmodels.iolib.summary2 import summary_col
print(summary_col(result_fit_list, stars = True))

In [ ]:
for child in [1]:
    to_keep_regressors = []
    child_rank_to_include = 0
    for liste in [between_threshold_for_child_rank, over_threshold_for_child_rank, children_number,interaction_child_between_threshold,interaction_child_over_threshold]:
        print(liste[child])
        to_keep_regressors.append(liste[child])

#### Double diff with no child family as control

In [ ]:
to_keep_regressors = []
child_rank_to_include = 0
df_temp = df.query("NBPAC_2011==NBPAC_2014")
for child in range(1,2):
    to_keep_regressors = []
    child_rank_to_include = 0
    for liste in [between_threshold_for_child_rank, over_threshold_for_child_rank, children_number,interaction_child_between_threshold,interaction_child_over_threshold]:
        to_keep_regressors.append(liste[child-1])
    m = sm.RLM.from_formula("(Delta_rbg)~   {}  ".format(
            ' + '.join( to_keep_regressors)), df_temp.query("(NBPAC_2011 == {})|(NBPAC_2011 == 0)".format(child)))
    fit = m.fit()
    summary = fit.summary(); print(summary)

In [ ]:
to_keep_regressors = []
child_rank_to_include = 0
df_temp = df.query("NBPAC_2011==NBPAC_2014")
for child in range(1,2):
    to_keep_regressors = []
    child_rank_to_include = 0
    for liste in [between_threshold_for_child_rank, over_threshold_for_child_rank, children_number,interaction_child_between_threshold,interaction_child_over_threshold]:
        to_keep_regressors.append(liste[child-1])
    m = sm.RLM.from_formula("(Delta_rbg)~   {}  ".format(
            ' + '.join( to_keep_regressors)), df_temp.query("(NBPAC_2011 == {})|(NBPAC_2011 == 0)".format(child)))
    fit = m.fit()
    summary = fit.summary(); print(summary)

In [ ]:
df.query(
                                                                    "(NBPAC_2011 == {})".format(1)).child_1.value_counts()

In [ ]:
map(between_threshold_for_child_rank.__getitem__,(0,1,2))

In [ ]:
to_keep_regressors

In [ ]:
to_keep_regressors = []
child_rank_to_include = 0
for child in range(1,nb_enf_max_for_annalysis+1):
    to_keep_regressors = []
    child_rank_to_include = 0
    for liste in [between_threshold_for_child_rank, over_threshold_for_child_rank]:
        to_keep_regressors.append(liste[child-1])
    m = sm.OLS.from_formula("(Delta_rbg)~   {} - child_{}".format(
            ' + '.join( to_keep_regressors), child), df.query(
                                                                    "(NBPAC_2011 == {})".format(child)))
    fit = m.fit()
    summary = fit.summary(); print(summary)

In [ ]:
to_keep_regressors = []
child_rank_to_include = 0
for child in range(1,nb_enf_max_for_annalysis+1):
    to_keep_regressors = []
    child_rank_to_include = 0
    for liste in [between_threshold_for_child_rank, over_threshold_for_child_rank]:
        to_keep_regressors.append(liste[child-1])
    m = sm.OLS.from_formula("(Delta_rbg)~   {} - child_{}".format(
            ' + '.join( to_keep_regressors), child), df.query(
                                                                    "(NBPAC_2011 == 0)".format(child)))
    fit = m.fit()
    summary = fit.summary(); print(summary)

In [ ]:
to_keep_regressors = []
child_rank_to_include = 0
for child in range(1,nb_enf_max_for_annalysis+1):
    to_keep_regressors = []
    child_rank_to_include = 0
    for liste in [between_threshold_for_child_rank, over_threshold_for_child_rank]:
        to_keep_regressors.append(liste[child-1])
    m = sm.OLS.from_formula("(Delta_rbg)~   {} - child_{}".format(
            ' + '.join( to_keep_regressors), child), df.query(
                                                                    "(NBPAC_2011 == {})".format(child)))
    fit = m.fit()
    summary = fit.summary(); print(summary)

In [ ]:
from operator import itemgetter
import itertools

In [ ]:
itemgetter(0,2,3)(between_threshold_for_child_rank)

## Pooled estimation with Treatment one, or two.

In [ ]:
df.Treatment_1_2011.value_counts()

In [ ]:
no_attached = (df.TYPE_FISC_2014.str.count("[3-9]")<1)&(df.TYPE_FISC_2011.str.count("[3-9]")<1)

In [ ]:
m = sm.OLS.from_formula("(Delta_rbg)~ Between_thresholds + Over_threshold".format( 
        ' + '.join(cumulative_child_bool_list+thrsh_bool_list))
                                                                                        , df.query("NBPAC_2011 == NBPAC_2014&(@no_attached)"))
fit = m.fit()
summary = fit.summary(); print(summary)

###Add controls for trends

In [ ]:
m = sm.OLS.from_formula("(Delta_rbg)~ Treatment_1_2011 + Treatment_2_2011 +{}".format( #AGERF = Age du référent fiscal
        ' + '.join(thrsh_bool_list))
                                                                                        , df)
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
m = sm.OLS.from_formula("(Delta_rbg)~ Treatment_1_2011 + Treatment_2_2011 +{}".format( #AGERF = Age du référent fiscal
        ' + '.join(child_bool_list))
                                                                                        , df)
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
m = sm.OLS.from_formula("(Delta_rbg)~ Treatment_1_2011 + Treatment_2_2011 +  {} ".format( #AGERF = Age du référent fiscal
        ' + '.join(child_bool_list+between_threshold_for_child_rank+over_threshold_for_child_rank))
                                                                                        , df)
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
threshold_byr_rbg
flatten_thresholds = np.sort(threshold_byr_rbg.values.flatten())

In [ ]:
flatten_thresholds_list = list()
for counter in range(len(flatten_thresholds)-1):
    print(flatten_thresholds[counter], flatten_thresholds[counter+1])
    
    thrsh1, thrsh2 = flatten_thresholds[counter], flatten_thresholds[counter+1]
    flatten_thresholds_list.append("Flatten_thresholds_{}_{}".format(thrsh1, thrsh2))
    df["Flatten_thresholds_{}_{}".format(thrsh1, thrsh2)] = ((df.Rbg_2011>  thrsh1)
                                                                                               & (df.Rbg_2011< thrsh2))
df["Flatten_thresholds_more_then_{}".format( thrsh2)] = df.Rbg_2011> thrsh2
flatten_thresholds_list.append("Flatten_thresholds_more_then_{}".format(thrsh2))

In [ ]:
flatten_thresholds_list = list()
for counter in range(len(flatten_thresholds)-1):
    print(flatten_thresholds[counter], flatten_thresholds[counter+1])
    flatten_thresholds_list.append("Flatten_thresholds_{}_{}".format(thrsh1, thrsh2))
    thrsh1, thrsh2 = flatten_thresholds[counter], flatten_thresholds[counter+1]
    df["Flatten_thresholds_{}_{}".format(thrsh1, thrsh2)] = ((df.Rbg_2011>  thrsh1)
                                                                                               & (df.Rbg_2011< thrsh2))
df["Flatten_thresholds_more_then_{}".format( thrsh2)] = df.Rbg_2011> thrsh2
flatten_thresholds_list.append("Flatten_thresholds_more_then_{}".format(thrsh2))

In [ ]:
m = sm.OLS.from_formula("(Delta_rbg)~ Treatment_1_2011 + Treatment_2_2011 +  {} ".format( #AGERF = Age du référent fiscal
        ' + '.join(child_bool_list+flatten_thresholds_list))
                                                                                        , df)
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
flatten_thresholds_list

In [ ]:
# Boum
# for nb_enf in range(1,7):
#     df["Between_threshold_child_rbg_2014"] = np.zeros(df.shape[0])
#     df["Between_threshold_child_rbg_2011"] = np.zeros(df.shape[0])
# for nb_enf in range(1,7): ### in modification
#     df["Between_threshold_child_rbg_2014".format(nb_enf)
#           ] +=(((df.Rbg_2014>  threshold_byr_rbg["2014"].iloc[nb_enf-1])
#                & (df.Rbg_2014< threshold_byr_rbg["2011"].iloc[nb_enf-1])) *( df.NBPAC_2011 == nb_enf)).astype("float")
#     df["Between_threshold_child_rbg_2011".format(nb_enf)
#           ] +=(((df.Rbg_2011>  threshold_byr_rbg["2014"].iloc[nb_enf-1])
#                & (df.Rbg_2011< threshold_byr_rbg["2011"].iloc[nb_enf-1])) *( df.NBPAC_2011 == nb_enf)).astype("float")



# df["Over_threshold_child_rbg_2014"] = np.zeros(df.shape[0])
# df["Over_threshold_child_rbg_2011"] = np.zeros(df.shape[0])
# for nb_enf in range(1,7): ### in modification
#     df["Over_threshold_child_rbg_2014".format(nb_enf)
#           ] +=(((df.Rbg_2014>  threshold_byr_rbg["2014"].iloc[nb_enf-1])
#                & (df.Rbg_2014< threshold_byr_rbg["2011"].iloc[nb_enf-1])) *( df.NBPAC_2011 == nb_enf)).astype("float")
#     df["Over_threshold_child_rbg_2011".format(nb_enf)
#           ] +=(((df.Rbg_2011>  threshold_byr_rbg["2014"].iloc[nb_enf-1])
#                & (df.Rbg_2011< threshold_byr_rbg["2011"].iloc[nb_enf-1])) *( df.NBPAC_2011 == nb_enf)).astype("float")

In [ ]:
# df["Between_threshold_child_rbg_2014"] = np.zeros(df.shape[0])
# df["Between_threshold_child_rbg_2011"] = np.zeros(df.shape[0])
# for nb_enf in range(1,7): ### in modification
#     df["Between_threshold_child_rbg_2014".format(nb_enf)
#           ] +=(((df.Rbg_2014>  threshold_byr_rbg["2014"].iloc[nb_enf-1])
#                & (df.Rbg_2014< threshold_byr_rbg["2011"].iloc[nb_enf-1])) *( df.NBPAC_2011 == nb_enf)).astype("float")
#     df["Between_threshold_child_rbg_2011".format(nb_enf)
#           ] +=(((df.Rbg_2011>  threshold_byr_rbg["2014"].iloc[nb_enf-1])
#                & (df.Rbg_2011< threshold_byr_rbg["2011"].iloc[nb_enf-1])) *( df.NBPAC_2011 == nb_enf)).astype("float")



# df["Over_threshold_child_rbg_2014"] = np.zeros(df.shape[0])
# df["Over_threshold_child_rbg_2011"] = np.zeros(df.shape[0])
# for nb_enf in range(1,7): ### in modification
#     df["Over_threshold_child_rbg_2014".format(nb_enf)
#           ] +=(((df.Rbg_2014>  threshold_byr_rbg["2014"].iloc[nb_enf-1])
#                & (df.Rbg_2014< threshold_byr_rbg["2011"].iloc[nb_enf-1])) *( df.NBPAC_2011 == nb_enf)).astype("float")
#     df["Over_threshold_child_rbg_2011".format(nb_enf)
#           ] +=(((df.Rbg_2011>  threshold_byr_rbg["2014"].iloc[nb_enf-1])
#                & (df.Rbg_2011< threshold_byr_rbg["2011"].iloc[nb_enf-1])) *( df.NBPAC_2011 == nb_enf)).astype("float")

In [ ]:

for nb_enf in range(1,7):
    df["Below_thresholds_{}_child_rbg_2014".format(nb_enf)] = np.zeros(df.shape[0])
for nb_enf in range(1,7):
    df["Below_thresholds_{}_child_rbg_2014".format(nb_enf)
          ] +=((df.Rbg_2014<  threshold_byr_rbg["2014"].iloc[nb_enf-1])).astype("float")

    


In [ ]:
below_threshold_for_child_rank = ["Below_thresholds_{}_child_rbg_2014".format(nb_child) for nb_child in range(1,nb_enf_max_for_annalysis+1)]
interaction_child_below_thresholds  = ["{}:{}".format(child,interaction) for child, interaction in zip(children_number, below_threshold_for_child_rank)]

## Triple diff estimation

In [ ]:
#df[over_threshold_for_child_rank+["NBPAC_2011"]]

In [ ]:
threshold_byr_rbg

In [ ]:
df.shape

In [ ]:
df.Age_youngest_child.hist(bins = 100)

### Classical 

#### On all the population

In [ ]:
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)&(@no_attached) ")) #SAME NBPAC
fit = m.fit()
summary = fit.summary(); print(summary)


In [ ]:
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))&(@no_attached)"+
                              "&(NBPAC_2011 == NBPAC_2014)")) #SAME NBPAC
fit = m.fit()
summary = fit.summary(); print(summary)


In [ ]:
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)&(@no_attached)")) #SAME NBPAC
fit = m.fit()
summary = fit.summary(); print(summary)


In [ ]:
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)")) #SAME NBPAC
fit = m.fit()
summary = fit.summary(); print(summary)


In [ ]:
    control_variables1 = "+CONTRAT_TRAVAIL_2011_father+CONTRAT_TRAVAIL_2011_mother+ I(DOMEMPL_2011_father)+I(DOMEMPL_2011_mother)+I(Age_father)+ I(Age_mother)"

In [ ]:
control_variables = "+Age_eldest_child+Age_youngest_child+OCCTYP_2011+I(TU10_2011)+I(NBPP_2011)+ I(DEP_2011)+I(CNE_2011)+SURFTOT_2011+Age_father+ Age_mother"

In [ ]:
#Flatten pas de départ ç la retraite
m = sm.OLS.from_formula("(Delta_rbg)~   {}".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  flatten_thresholds_list+hardcoded_between+hardcoded_over))
                    , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)&(ZPERM_2011<100)& (ZPERM_2014<100) & (Age_father<100)&(@no_attached)")) #SAME NBPAC
fit = m.fit()
summary = fit.summary(); print(summary)


In [ ]:
#Flatten
m = sm.OLS.from_formula("(Delta_rbg)~   {}".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  flatten_thresholds_list+hardcoded_between+hardcoded_over))#+control_variables
                    , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 >= 0)&(ZPERM_2011<100) & (Age_father<100)&(@no_attached)")) #SAME NBPAC
fit = m.fit()
summary = fit.summary(); print(summary)


In [ ]:
#Flatten
m = sm.OLS.from_formula("(Delta_rbg)~   {}+I(Age_youngest_child+10)".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  flatten_thresholds_list+hardcoded_between+hardcoded_over))#+control_variables
                    , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 >= 0)&(ZPERM_2011<100) & (Age_father<100)&(@no_attached)")) #SAME NBPAC
fit = m.fit()
summary = fit.summary(); print(summary)


In [ ]:
df.Age_youngest_child
m = sm.OLS.from_formula("(Delta_rbg)~   {} +Age_father+Age_mother+I(Age_father**2)+I(Age_mother**2)+Age_eldest_child+Age_youngest_child".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  flatten_thresholds_list+hardcoded_between+hardcoded_over))
                    , df.query("((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)")) #SAME NBPAC
fit = m.fit()
summary = fit.summary(); print(summary)


In [ ]:
m = sm.OLS.from_formula("Delta_rbg~   {}+Age_father+Age_mother+I(Age_father**2)+I(Age_mother**2)+Age_eldest_child+Age_youngest_child + I(Age_eldest_child**2)+I(Age_youngest_child**2) ".format(
        ' + '.join(children_number[:-2]+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  flatten_thresholds_list[:-2]+hardcoded_between[:-2]+hardcoded_over[:-2]))
                    , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)&(Age_eldest_child<20)&(Age_youngest_child>1)&(NBPAC_2011 <= 4)")) #SAME NBPAC
fit = m.fit()
summary = fit.summary(); print(summary)



In [ ]:
m = sm.OLS.from_formula("Delta_rbg~   {}+Age_father+Age_mother+I(Age_father**2)+I(Age_mother**2)+Age_eldest_child+Age_youngest_child + I(Age_eldest_child**2)+I(Age_youngest_child**2) ".format(
        ' + '.join(children_number[:-1]+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  flatten_thresholds_list[:-1]+hardcoded_between[:-1]+hardcoded_over[:-1]))
                    , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)&(Age_eldest_child<10)&(NBPAC_2011 <= 5)")) #SAME NBPAC
fit = m.fit()
summary = fit.summary(); print(summary)



In [ ]:
df['Likely_to_get_retired'] = False
df.loc[((df.Age_father>58)&(df.Age_father<65)), 'Likely_to_get_retired'] = True

df.loc[((df.Age_mother>58)&(df.Age_mother<65)), 'Likely_to_get_retired'] = True

In [ ]:
df.Likely_to_get_retired.value_counts(dropna = False)

In [ ]:
m = sm.OLS.from_formula("Delta_rbg~   {}+Age_father+Age_mother+I(Age_father**2)+I(Age_mother**2)+Age_eldest_child+Age_youngest_child + I(Age_eldest_child**2)+I(Age_youngest_child**2) ".format(
        ' + '.join(children_number[:-2]+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  flatten_thresholds_list+hardcoded_between+hardcoded_over))
                    , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)&(Likely_to_get_retired==False)")) #SAME NBPAC
fit = m.fit()
summary = fit.summary(); print(summary)



In [ ]:
m = sm.OLS.from_formula("np.log(Rbg_2014/Rbg_2011)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)")) #SAME NBPAC
fit = m.fit()
summary = fit.summary(); print(summary)


#### Different samples

In [ ]:
###Basic
regression_fit_list = list()
regression_delta_zimpom_fit_list = list()
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)



###Both years
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)




## Without retirement income in 2011
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))&(ZPERM_2011<1000)"+
                              "&(NBPAC_2011 == NBPAC_2014)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)


#### Without retirement income both years
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))&(ZPERM_2011<10)&(ZPERM_2014<10)"+
                              "&(NBPAC_2011 == NBPAC_2014)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)


#Super select
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query(
                                "((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))&(ZPERM_2011<10)&(ZPERM_2014<10)"+
                                   "&(NBPAC_2011 == NBPAC_2014)&(ZIMPOM_DECL_2014>100)&(ZIMPOM_2011>100) & (ZVALM_2011<=200) &(ZVAMM_2011<200)& (ZRACM_2011<200) & (ZETRM_2011<100)\
                                    &(ZIMPOM_2014>100) & (ZVALM_2014<=200) &(ZVAMM_2014<200)& (ZRACM_2014<200)& (ZETRM_2014<100)"))

fit = m.fit()
regression_fit_list.append(fit)



#Super select not retired in 2011
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query(
                                "((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))&(ZPERM_2011<10)&(ZPERM_2014<10)"+
                                   "&(NBPAC_2011 == NBPAC_2014)&(ZIMPOM_DECL_2014>100)&(ZIMPOM_2011>100) & (ZVALM_2011<=200) &(ZVAMM_2011<200)& (ZRACM_2011<200) & (ZETRM_2011<100)& (ZPERM_2011<1000)\
                                    &(ZIMPOM_2014>100) & (ZVALM_2014<=200) &(ZVAMM_2014<200)& (ZRACM_2014<200)& (ZETRM_2014<100)"))

fit = m.fit()
regression_fit_list.append(fit)



#Super select not retired in 2014 or 2011
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query(
                                "((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))&(ZPERM_2011<10)&(ZPERM_2014<10)"+
                                   "&(NBPAC_2011 == NBPAC_2014)&(ZIMPOM_DECL_2014>100)&(ZIMPOM_2011>100) & (ZVALM_2011<=200) &(ZVAMM_2011<200)& (ZRACM_2011<200) & (ZETRM_2011<100)& (ZPERM_2011<1000)& (ZPERM_2014<1000)\
                                    &(ZIMPOM_2014>100) & (ZVALM_2014<=200) &(ZVAMM_2014<200)& (ZRACM_2014<200)& (ZETRM_2014<100)"))

fit = m.fit()
regression_fit_list.append(fit)




#Super select not retired in 2014 or 2011
m = sm.OLS.from_formula("(Delta_rbg)~   {} +Age_father+Age_mother+I(Age_father**2)+I(Age_mother**2)+Age_eldest_child+Age_youngest_child + I(Age_eldest_child**2)+I(Age_youngest_child**2)".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query(
                                "((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))&(ZPERM_2011<10)&(ZPERM_2014<10)"+
                                   "&(NBPAC_2011 == NBPAC_2014)&(ZIMPOM_DECL_2014>100)&(ZIMPOM_2011>100) & (ZVALM_2011<=200) &(ZVAMM_2011<200)& (ZRACM_2011<200) & (ZETRM_2011<100)& (ZPERM_2011<1000)& (ZPERM_2014<1000)\
                                    &(ZIMPOM_2014>100) & (ZVALM_2014<=200) &(ZVAMM_2014<200)& (ZRACM_2014<200)& (ZETRM_2014<100)"))

fit = m.fit()
regression_fit_list.append(fit)
#Super select not retired in 2014 or 2011
m = sm.OLS.from_formula("(Delta_rbg)~   {} +Age_eldest_child+Age_youngest_child + I(Age_eldest_child**2)+I(Age_youngest_child**2)".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query(
                                "((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))&(ZPERM_2011<10)&(ZPERM_2014<10)"+
                                   "&(NBPAC_2011 == NBPAC_2014)&(ZIMPOM_DECL_2014>100)&(ZIMPOM_2011>100) & (ZVALM_2011<=200) &(ZVAMM_2011<200)& (ZRACM_2011<200) & (ZETRM_2011<100)& (ZPERM_2011<1000)& (ZPERM_2014<1000)\
                                    &(ZIMPOM_2014>100) & (ZVALM_2014<=200) &(ZVAMM_2014<200)& (ZRACM_2014<200)& (ZETRM_2014<100)&(Age_father<40)"))

fit = m.fit()
regression_fit_list.append(fit)


###log ratio

m = sm.OLS.from_formula("np.log(Rbg_2014/Rbg_2011)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)





#log ratio Super select not retired in 2014 or 2011
m = sm.OLS.from_formula("np.log(Rbg_2014/Rbg_2011)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query(
                                "((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))&(ZPERM_2011<10)&(ZPERM_2014<10)"+
                                   "&(NBPAC_2011 == NBPAC_2014)&(ZIMPOM_DECL_2014>100)&(ZIMPOM_2011>100) & (ZVALM_2011<=200) &(ZVAMM_2011<200)& (ZRACM_2011<200) & (ZETRM_2011<100)& (ZPERM_2011<1000)& (ZPERM_2014<1000)\
                                    &(ZIMPOM_2014>100) & (ZVALM_2014<=200) &(ZVAMM_2014<200)& (ZRACM_2014<200)& (ZETRM_2014<100)"))

fit = m.fit()
regression_fit_list.append(fit)

In [ ]:

print(summary_col(regression_fit_list, stars = True, info_dict={'adjusted-R2':lambda x:"{:4.2f}%".format(x.rsquared_adj*100), "N":lambda x:"{:d}".format(int(x.nobs))}))

In [ ]:
###Basic
regression_fit_list = list()
regression_delta_zimpom_fit_list = list()
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)&(@no_attached)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)



###Both years
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)&(@no_attached)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)




## Without retirement income in 2011
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))&(ZPERM_2011<1000)"+
                              "&(NBPAC_2011 == NBPAC_2014)&(@no_attached)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)


#### Without retirement income both years
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))&(ZPERM_2011<10)&(ZPERM_2014<10)"+
                              "&(NBPAC_2011 == NBPAC_2014)&(@no_attached)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)


#Super select
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query(
                                "((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))&(ZPERM_2011<10)&(ZPERM_2014<10)"+
                                   "&(NBPAC_2011 == NBPAC_2014)&(ZIMPOM_DECL_2014>100)&(ZIMPOM_2011>100) & (ZVALM_2011<=200) &(ZVAMM_2011<200)& (ZRACM_2011<200) & (ZETRM_2011<100)\
                                    &(ZIMPOM_2014>100) & (ZVALM_2014<=200) &(ZVAMM_2014<200)& (ZRACM_2014<200)& (ZETRM_2014<100)&(@no_attached)"))

fit = m.fit()
regression_fit_list.append(fit)



#Super select not retired in 2011
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query(
                                "((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))&(ZPERM_2011<10)&(ZPERM_2014<10)"+
                                   "&(NBPAC_2011 == NBPAC_2014)&(ZIMPOM_DECL_2014>100)&(ZIMPOM_2011>100) & (ZVALM_2011<=200) &(ZVAMM_2011<200)& (ZRACM_2011<200) & (ZETRM_2011<100)& (ZPERM_2011<1000)\
                                    &(ZIMPOM_2014>100) & (ZVALM_2014<=200) &(ZVAMM_2014<200)& (ZRACM_2014<200)& (ZETRM_2014<100)&(@no_attached)"))

fit = m.fit()
regression_fit_list.append(fit)



#Super select not retired in 2014 or 2011
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query(
                                "((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))&(ZPERM_2011<10)&(ZPERM_2014<10)"+
                                   "&(NBPAC_2011 == NBPAC_2014)&(ZIMPOM_DECL_2014>100)&(ZIMPOM_2011>100) & (ZVALM_2011<=200) &(ZVAMM_2011<200)& (ZRACM_2011<200) & (ZETRM_2011<100)& (ZPERM_2011<1000)& (ZPERM_2014<1000)\
                                    &(ZIMPOM_2014>100) & (ZVALM_2014<=200) &(ZVAMM_2014<200)& (ZRACM_2014<200)& (ZETRM_2014<100)&(@no_attached)"))

fit = m.fit()
regression_fit_list.append(fit)




#Super select not retired in 2014 or 2011
m = sm.OLS.from_formula("(Delta_rbg)~   {} +Age_father+Age_mother+I(Age_father**2)+I(Age_mother**2)+Age_eldest_child+Age_youngest_child + I(Age_eldest_child**2)+I(Age_youngest_child**2)".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query(
                                "((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))&(ZPERM_2011<10)&(ZPERM_2014<10)"+
                                   "&(NBPAC_2011 == NBPAC_2014)&(ZIMPOM_DECL_2014>100)&(ZIMPOM_2011>100) & (ZVALM_2011<=200) &(ZVAMM_2011<200)& (ZRACM_2011<200) & (ZETRM_2011<100)& (ZPERM_2011<1000)& (ZPERM_2014<1000)\
                                    &(ZIMPOM_2014>100) & (ZVALM_2014<=200) &(ZVAMM_2014<200)& (ZRACM_2014<200)& (ZETRM_2014<100)&(@no_attached)"))

fit = m.fit()
regression_fit_list.append(fit)
#Super select not retired in 2014 or 2011
m = sm.OLS.from_formula("(Delta_rbg)~   {} +Age_eldest_child+Age_youngest_child + I(Age_eldest_child**2)+I(Age_youngest_child**2)".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query(
                                "((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))&(ZPERM_2011<10)&(ZPERM_2014<10)"+
                                   "&(NBPAC_2011 == NBPAC_2014)&(ZIMPOM_DECL_2014>100)&(ZIMPOM_2011>100) & (ZVALM_2011<=200) &(ZVAMM_2011<200)& (ZRACM_2011<200) & (ZETRM_2011<100)& (ZPERM_2011<1000)& (ZPERM_2014<1000)\
                                    &(ZIMPOM_2014>100) & (ZVALM_2014<=200) &(ZVAMM_2014<200)& (ZRACM_2014<200)& (ZETRM_2014<100)&(Age_father<40)&(@no_attached)"))

fit = m.fit()
regression_fit_list.append(fit)


###log ratio

m = sm.OLS.from_formula("np.log(Rbg_2014/Rbg_2011)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)&(@no_attached)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)





#log ratio Super select not retired in 2014 or 2011
m = sm.OLS.from_formula("np.log(Rbg_2014/Rbg_2011)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query(
                                "((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))&(ZPERM_2011<10)&(ZPERM_2014<10)"+
                                   "&(NBPAC_2011 == NBPAC_2014)&(ZIMPOM_DECL_2014>100)&(ZIMPOM_2011>100) & (ZVALM_2011<=200) &(ZVAMM_2011<200)& (ZRACM_2011<200) & (ZETRM_2011<100)& (ZPERM_2011<1000)& (ZPERM_2014<1000)\
                                    &(ZIMPOM_2014>100) & (ZVALM_2014<=200) &(ZVAMM_2014<200)& (ZRACM_2014<200)& (ZETRM_2014<100)&(@no_attached)"))

fit = m.fit()
regression_fit_list.append(fit)

In [ ]:

print(summary_col(regression_fit_list, stars = True, info_dict={'adjusted-R2':lambda x:"{:4.2f}%".format(x.rsquared_adj*100), "N":lambda x:"{:d}".format(int(x.nobs))}))

### How well do we match the reform ?

In [ ]:
interactions_rbg_between = ["Interactions_rbg_child{}_between".format(child) for child in range(1,7)]
interactions_rbg_over = ["Interactions_rbg_child{}_over".format(child) for child in range(1,7)]
interactions_child_over = ["Interactions_child{}_over".format(child) for child in range(1,7)]

In [ ]:
regression_delta_zimpom_formula = "(Delta_ZIMPOM)~   {} - 1 + Not_touched_by_reform + Delta_Rbg_times_not_touched_reform".format(
        ' + '.join( interactions_child_over+interactions_rbg_over+interactions_rbg_between))

In [ ]:
m = sm.RLM.from_formula(regression_delta_zimpom_formula  , df)#; regression_delta_zimpom_fit_list.append()

In [ ]:
df["Stays_in_same_treatment"]= ((df.Treatment_0_2011 & df.Treatment_0_2014)
                                               +(df.Treatment_1_2011 & df.Treatment_1_2014)
                                               +(df.Treatment_2_2011 & df.Treatment_2_2014))

In [ ]:
## With no attached in conditions
###Basic
regression_fit_list = list()
regression_delta_zimpom_fit_list = list()
m = sm.OLS.from_formula(regression_delta_zimpom_formula,
                     df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)&(@no_attached)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)



###Both years
m = sm.OLS.from_formula(regression_delta_zimpom_formula,
                     df.query("((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)&(@no_attached)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)




## Without retirement income in 2011
m = sm.OLS.from_formula(regression_delta_zimpom_formula,
                 df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))&(ZPERM_2011<1000)"+
                              "&(NBPAC_2011 == NBPAC_2014)&(@no_attached)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)


#### Without retirement income both years
m = sm.OLS.from_formula(regression_delta_zimpom_formula,
                 df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))&(ZPERM_2011<10)&(ZPERM_2014<10)"+
                              "&(NBPAC_2011 == NBPAC_2014)&(@no_attached)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)

#### Without retirement income both years,both year on th ecorrect income span
m = sm.OLS.from_formula(regression_delta_zimpom_formula,
                 df.query("((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))&(ZPERM_2011<10)&(ZPERM_2014<10)"+
                              "&(NBPAC_2011 == NBPAC_2014)&(ZIMPOM_DECL_2014>100)&(ZIMPOM_2011>100) & (ZVALM_2011<=200) &(ZVAMM_2011<200)& (ZRACM_2011<200) & (ZETRM_2011<100)\
                                                                                &(ZIMPOM_2014>100) & (ZVALM_2014<=200) &(ZVAMM_2014<200)& (ZRACM_2014<200)& (ZETRM_2014<100)&(@no_attached)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)





#### Without retirement income both years,both year on th ecorrect income span
m = sm.OLS.from_formula(regression_delta_zimpom_formula,
                 df.query("((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))&(ZPERM_2011<10)&(ZPERM_2014<10)"+
                              "&(NBPAC_2011 == NBPAC_2014)&(ZIMPOM_DECL_2014>100)&(ZIMPOM_2011>100) & (ZVALM_2011<=200) &(ZVAMM_2011<200)& (ZRACM_2011<200) & (ZETRM_2011<100)\
                                                                                &(ZIMPOM_2014>100) & (ZVALM_2014<=200) &(ZVAMM_2014<200)& (ZRACM_2014<200)& (ZETRM_2014<100)&(@no_attached)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)



### Stay in same treatment
m = sm.OLS.from_formula(regression_delta_zimpom_formula,
                     df.query("((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))&(ZPERM_2011<10)&(ZPERM_2014<10)"+
                              "&(NBPAC_2011 == NBPAC_2014)&(@no_attached)&(Stays_in_same_treatment)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)


print(summary_col(regression_fit_list, stars = True, 
                  model_names=["Delta_ZIMPOM", "Both_year", "No_ZPERM_2011", "No_ZPERM", "Both_year_and_no_ZPERM", "Both_pays_tax_no_exotic_income", "Stays_in_same_treatment"],
                  info_dict={'adjusted-R2':lambda x:"{:4.2f}%".format(x.rsquared_adj*100), "N":lambda x:"{:d}".format(int(x.nobs))}))

In [ ]:
###Basic
regression_fit_list = list()
regression_delta_zimpom_fit_list = list()
m = sm.OLS.from_formula(regression_delta_zimpom_formula,
                     df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)



###Both years
m = sm.OLS.from_formula(regression_delta_zimpom_formula,
                     df.query("((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)




## Without retirement income in 2011
m = sm.OLS.from_formula(regression_delta_zimpom_formula,
                 df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))&(ZPERM_2011<1000)"+
                              "&(NBPAC_2011 == NBPAC_2014)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)


#### Without retirement income both years
m = sm.OLS.from_formula(regression_delta_zimpom_formula,
                 df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))&(ZPERM_2011<10)&(ZPERM_2014<10)"+
                              "&(NBPAC_2011 == NBPAC_2014)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)

#### Without retirement income both years,both year on th ecorrect income span
m = sm.OLS.from_formula(regression_delta_zimpom_formula,
                 df.query("((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))&(ZPERM_2011<10)&(ZPERM_2014<10)"+
                              "&(NBPAC_2011 == NBPAC_2014)&(ZIMPOM_DECL_2014>100)&(ZIMPOM_2011>100) & (ZVALM_2011<=200) &(ZVAMM_2011<200)& (ZRACM_2011<200) & (ZETRM_2011<100)\
                                                                                &(ZIMPOM_2014>100) & (ZVALM_2014<=200) &(ZVAMM_2014<200)& (ZRACM_2014<200)& (ZETRM_2014<100)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)





#### Without retirement income both years,both year on th ecorrect income span
m = sm.OLS.from_formula(regression_delta_zimpom_formula,
                 df.query("((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))&(ZPERM_2011<10)&(ZPERM_2014<10)"+
                              "&(NBPAC_2011 == NBPAC_2014)&(ZIMPOM_DECL_2014>100)&(ZIMPOM_2011>100) & (ZVALM_2011<=200) &(ZVAMM_2011<200)& (ZRACM_2011<200) & (ZETRM_2011<100)\
                                                                                &(ZIMPOM_2014>100) & (ZVALM_2014<=200) &(ZVAMM_2014<200)& (ZRACM_2014<200)& (ZETRM_2014<100)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)



###log ratio
m = sm.OLS.from_formula(regression_delta_zimpom_formula,
                     df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"
                              "&(NBPAC_2011 == NBPAC_2014)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)


print(summary_col(regression_fit_list, stars = True, info_dict={'adjusted-R2':lambda x:"{:4.2f}%".format(x.rsquared_adj*100), "N":lambda x:"{:d}".format(int(x.nobs))}))

In [ ]:
df[df.Between_thresholds].query("NBPAC_2011==5").Stays_in_same_treatment.value_counts(normalize =True)

In [ ]:
df_contraint=df.copy()
df_contraint.loc[df_contraint.Over_1_X_1_child == True, "Delta_ZIMPOM"] = df_contraint.Delta_ZIMPOM-836
df_contraint.loc[df_contraint.Over_2_X_2_child == True, "Delta_ZIMPOM"] = df_contraint.Delta_ZIMPOM-836*2
df_contraint.loc[df_contraint.Over_3_X_3_child == True, "Delta_ZIMPOM"] = df_contraint.Delta_ZIMPOM-836*4
df_contraint.loc[df_contraint.Over_4_X_4_child == True, "Delta_ZIMPOM"] = df_contraint.Delta_ZIMPOM-836*6
df_contraint.loc[df_contraint.Over_5_X_5_child == True, "Delta_ZIMPOM"] = df_contraint.Delta_ZIMPOM-836*8
df_contraint.loc[df_contraint.Over_6_X_6_child == True, "Delta_ZIMPOM"] = df_contraint.Delta_ZIMPOM-836*10

In [ ]:
df_contraint=df.copy()
df_contraint.loc[df_contraint.Between_1_X_1_child == True, "Delta_ZIMPOM"] = df_contraint.Delta_ZIMPOM-df_contraint.Delta_rbg * 0.3
df_contraint.loc[df_contraint.Between_2_X_2_child == True, "Delta_ZIMPOM"] = df_contraint.Delta_ZIMPOM-df_contraint.Delta_rbg * 0.3
df_contraint.loc[df_contraint.Between_3_X_3_child == True, "Delta_ZIMPOM"] = df_contraint.Delta_ZIMPOM-df_contraint.Delta_rbg * 0.3
df_contraint.loc[df_contraint.Between_4_X_4_child == True, "Delta_ZIMPOM"] = df_contraint.Delta_ZIMPOM-df_contraint.Delta_rbg * 0.3
df_contraint.loc[df_contraint.Between_5_X_5_child == True, "Delta_ZIMPOM"] = df_contraint.Delta_ZIMPOM-df_contraint.Delta_rbg * 0.3
df_contraint.loc[df_contraint.Between_6_X_6_child == True, "Delta_ZIMPOM"] = df_contraint.Delta_ZIMPOM-df_contraint.Delta_rbg * 0.3

In [ ]:
# for child in range(1,7):
#     print(df[df["Over_{}_X_{}_child".format(child,child)]].MTR_2014.mean())
# for child in range(1,7):
#     print(df[df["Between_{}_X_{}_child".format(child,child)]].MTR_2014.mean())
# print(df[df.Not_touched_by_reform==1].MTR_2014.mean())



# df_contraint=df.copy()
# df_contraint.loc[df_contraint.Over_1_X_1_child == True, "Delta_ZIMPOM"] = df_contraint.Delta_ZIMPOM-df_contraint.Delta_rbg * df[df.Over_1_X_1_child].MTR_2011.mean()
# df_contraint.loc[df_contraint.Over_2_X_2_child == True, "Delta_ZIMPOM"] = df_contraint.Delta_ZIMPOM-df_contraint.Delta_rbg * df[df.Over_2_X_2_child].MTR_2011.mean()
# df_contraint.loc[df_contraint.Over_3_X_3_child == True, "Delta_ZIMPOM"] = df_contraint.Delta_ZIMPOM-df_contraint.Delta_rbg * df[df.Over_3_X_3_child].MTR_2011.mean()
# df_contraint.loc[df_contraint.Over_4_X_4_child == True, "Delta_ZIMPOM"] = df_contraint.Delta_ZIMPOM-df_contraint.Delta_rbg * df[df.Over_4_X_4_child].MTR_2011.mean()
# df_contraint.loc[df_contraint.Over_5_X_5_child == True, "Delta_ZIMPOM"] = df_contraint.Delta_ZIMPOM-df_contraint.Delta_rbg * df[df.Over_5_X_5_child].MTR_2011.mean()
# df_contraint.loc[df_contraint.Over_6_X_6_child == True, "Delta_ZIMPOM"] = df_contraint.Delta_ZIMPOM-df_contraint.Delta_rbg * df[df.Over_6_X_6_child].MTR_2011.mean()



# df_contraint.loc[df_contraint.Between_1_X_1_child == True, "Delta_ZIMPOM"] = df_contraint.Delta_ZIMPOM-df_contraint.Delta_rbg * df[df.Between_1_X_1_child].MTR_2011.mean()
# df_contraint.loc[df_contraint.Between_2_X_2_child == True, "Delta_ZIMPOM"] = df_contraint.Delta_ZIMPOM-df_contraint.Delta_rbg * df[df.Between_2_X_2_child].MTR_2011.mean()
# df_contraint.loc[df_contraint.Between_3_X_3_child == True, "Delta_ZIMPOM"] = df_contraint.Delta_ZIMPOM-df_contraint.Delta_rbg * df[df.Between_3_X_3_child].MTR_2011.mean()
# df_contraint.loc[df_contraint.Between_4_X_4_child == True, "Delta_ZIMPOM"] = df_contraint.Delta_ZIMPOM-df_contraint.Delta_rbg * df[df.Between_4_X_4_child].MTR_2011.mean()
# df_contraint.loc[df_contraint.Between_5_X_5_child == True, "Delta_ZIMPOM"] = df_contraint.Delta_ZIMPOM-df_contraint.Delta_rbg * df[df.Between_5_X_5_child].MTR_2011.mean()
# df_contraint.loc[df_contraint.Between_6_X_6_child == True, "Delta_ZIMPOM"] = df_contraint.Delta_ZIMPOM-df_contraint.Delta_rbg * df[df.Between_6_X_6_child].MTR_2011.mean()

In [ ]:
regression_delta_zimpom_formula = "(Delta_ZIMPOM)~   {} - 1 + Not_touched_by_reform + Delta_Rbg_times_not_touched_reform".format(
        ' + '.join(interactions_child_over))


###Basic
regression_fit_list = list()
regression_delta_zimpom_fit_list = list()
m = sm.OLS.from_formula(regression_delta_zimpom_formula,
                     df_contraint.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)



###Both years
m = sm.OLS.from_formula(regression_delta_zimpom_formula,
                     df_contraint.query("((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)




## Without retirement income in 2011
m = sm.OLS.from_formula(regression_delta_zimpom_formula,
                 df_contraint.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))&(ZPERM_2011<1000)"+
                              "&(NBPAC_2011 == NBPAC_2014)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)


#### Without retirement income both years
m = sm.OLS.from_formula(regression_delta_zimpom_formula,
                 df_contraint.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))&(ZPERM_2011<10)&(ZPERM_2014<10)"+
                              "&(NBPAC_2011 == NBPAC_2014)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)

#### Without retirement income both years,both year on th ecorrect income span
m = sm.OLS.from_formula(regression_delta_zimpom_formula,
                 df_contraint.query("((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))&(ZPERM_2011<10)&(ZPERM_2014<10)"+
                              "&(NBPAC_2011 == NBPAC_2014)&(ZIMPOM_DECL_2014>100)&(ZIMPOM_2011>100) & (ZVALM_2011<=200) &(ZVAMM_2011<200)& (ZRACM_2011<200) & (ZETRM_2011<100)\
                                                                                &(ZIMPOM_2014>100) & (ZVALM_2014<=200) &(ZVAMM_2014<200)& (ZRACM_2014<200)& (ZETRM_2014<100)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)


###log ratio
m = sm.OLS.from_formula(regression_delta_zimpom_formula,
                     df_contraint.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)")) #SAME NBPAC
fit = m.fit()
regression_fit_list.append(fit)


print(summary_col(regression_fit_list, stars = True, info_dict={'adjusted-R2':lambda x:"{:4.2f}%".format(x.rsquared_adj*100), "N":lambda x:"{:d}".format(int(x.nobs))}))

In [ ]:
836*6

### By type of income 

In [ ]:
household_income_vars = ['ZTSAM', 'ZRAGM', 'ZRACM', 'ZRNCM', 'ZFONM', 'ZPERM', 'ZRICM']

In [ ]:
###Basic
regression_fit_list = list()

for var in ["Delta_{}".format(var )for var in household_income_vars]:
    m = sm.OLS.from_formula("({})~   {} ".format(var,
            ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                      between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                      interaction_child_between_threshold+interaction_child_over_threshold))
                        , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                                  "&(NBPAC_2011 == NBPAC_2014)")) #SAME NBPAC
    fit = m.fit()
    regression_fit_list.append(fit)



In [ ]:
print(summary_col(regression_fit_list, stars = True, info_dict={'adjusted-R2':lambda x:"{:4.2f}%".format(x.rsquared_adj*100), "N":lambda x:"{:d}".format(int(x.nobs))}))

In [ ]:
household_income_vars

### By type of income if  a specific income is earned 

In [ ]:
###Basic
regression_fit_list = list()

for var in  household_income_vars:
    endogenous_var = "Delta_{}".format(var )
    m = sm.OLS.from_formula("({})~   {} ".format(endogenous_var,
            ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                      between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                      interaction_child_between_threshold+interaction_child_over_threshold))
                        , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                                  "&(NBPAC_2011 == NBPAC_2014)&(({}_2011>100)|({}_2014>100)) ".format(var,var))) #SAME NBPAC
    fit = m.fit()
    regression_fit_list.append(fit)



In [ ]:
print(summary_col(regression_fit_list, stars = True, info_dict={'adjusted-R2':lambda x:"{:4.2f}%".format(x.rsquared_adj*100), "N":lambda x:"{:d}".format(int(x.nobs))}))

#### By type of income mother father 

In [ ]:
individualized_income_var = [ 'YSALI', 'YCHOI', 'YRSTI',
                               'YALRI', 'YRAGI', 'YBICI', 'YBNCI']

In [ ]:
df.YSALI_2014_father.head()

We have some nan, why ?

In [ ]:
df[["YSALI_2014_father", "YSALI_2014_mother","YSALI_2011_father", "YSALI_2011_mother", "Rbg_2011"]].isna().sum()/df[["YSALI_2014_father", "YSALI_2014_mother","YSALI_2011_father", "YSALI_2011_mother", "Rbg_2011"]].shape[0]

In [ ]:
both_wage = ~(df["YSALI_2014_father"].isna()| df["YSALI_2014_mother"].isna()| df["YSALI_2011_father"].isna() | df["YSALI_2011_mother"].isna())

In [ ]:

for parent in ["father", "mother"]:
    for var in individualized_income_var:
        df["Delta_{}_{}".format(var, parent)] = df["{}_2014_{}".format(var,parent)] - df["{}_2011_{}".format(var, parent)]

In [ ]:
individualized_income_var

In [ ]:
###Basic
regression_fit_list = list()

for var in individualized_income_var[:5]:
    for parent in ["father", "mother"]:
        endogenous_var = "Delta_{}_{}".format(var, parent)
        m = sm.OLS.from_formula("({})~   {} ".format(endogenous_var,
                ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                          between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                          interaction_child_between_threshold+interaction_child_over_threshold))
                            , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                                      "&(NBPAC_2011 == NBPAC_2014)& @both_wage& @no_attached")) #SAME NBPAC
        fit = m.fit()
        regression_fit_list.append(fit)



In [ ]:
print(summary_col(regression_fit_list, stars = True, info_dict={'adjusted-R2':lambda x:"{:4.2f}%".format(x.rsquared_adj*100), "N":lambda x:"{:d}".format(int(x.nobs))}))

In [ ]:
###Basic
regression_fit_list = list()

for var in individualized_income_var[5:]:
    for parent in ["father", "mother"]:
        endogenous_var = "Delta_{}_{}".format(var, parent)
        m = sm.OLS.from_formula("({})~   {} ".format(endogenous_var,
                ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                          between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                          interaction_child_between_threshold+interaction_child_over_threshold))
                            , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                                      "&(NBPAC_2011 == NBPAC_2014)")) #SAME NBPAC
        fit = m.fit()
        regression_fit_list.append(fit)



In [ ]:
print(summary_col(regression_fit_list, stars = True, info_dict={'adjusted-R2':lambda x:"{:4.2f}%".format(x.rsquared_adj*100), "N":lambda x:"{:d}".format(int(x.nobs))}))

#### By  type of income if a specific income is earned.

In [ ]:
###Basic
regression_fit_list = list()

for var in individualized_income_var[:5]:
    for parent in ["father", "mother"]:
        endogenous_var = "Delta_{}_{}".format(var, parent)
        m = sm.OLS.from_formula("({})~   {} ".format(endogenous_var,
                ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                          between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                          interaction_child_between_threshold+interaction_child_over_threshold))
                            , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                                      "&(NBPAC_2011 == NBPAC_2014)&(({}_2011_{}>100)|({}_2014_{}>100))".format(var,parent,var, parent))) 
        fit = m.fit()
        regression_fit_list.append(fit)



In [ ]:
print(summary_col(regression_fit_list, stars = True, info_dict={'adjusted-R2':lambda x:"{:4.2f}%".format(x.rsquared_adj*100), "N":lambda x:"{:d}".format(int(x.nobs))}))

#### Extensive margin

In [ ]:
for parent in ["father", "mother"]:
    for var in individualized_income_var:
        df["Start_to_work_{}".format(parent)] = ((df["YSALI_2014_{}".format(parent)]>5000) & (df["YSALI_2011_{}".format(parent)]<5000)).astype("int")
        df["Stop_to_work_{}".format(parent)] = ((df["YSALI_2014_{}".format(parent)]<5000) & (df["YSALI_2011_{}".format(parent)]>5000)).astype("int")

In [ ]:
###Basic
regression_fit_list = list()
for var in ["Start", "Stop"]:
    for parent in ["father", "mother"]:
        endogenous_var = "{}_to_work_{}".format( var, parent)
        m = sm.OLS.from_formula("({})~   {} ".format(endogenous_var,
                ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                          between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                          interaction_child_between_threshold+interaction_child_over_threshold))
                            , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                                      "&(NBPAC_2011 >= 0)&(@both_wage)")) #SAME NBPAC
        fit = m.fit(maxiter=100)
        regression_fit_list.append(fit)



In [ ]:
print(summary_col(regression_fit_list, stars = True, info_dict={'adjusted-R2':lambda x:"{:4.2f}%".format(x.rsquared_adj*100), "N":lambda x:"{:d}".format(int(x.nobs))}))

In [ ]:
df["Passage_temps_partiel_father"] = ((df.CE_2011_father == "C") & (df.CE_2014_father == "P"  )).astype("int")
df["Passage_temps_partiel_mother"] = ((df.CE_2011_mother == "C") & (df.CE_2014_mother == "P")).astype("int")

df["Passage_temps_plein_father"] = ((df.CE_2011_father == "P") & (df.CE_2014_father == "C"  )).astype("int")
df["Passage_temps_plein_mother"] = ((df.CE_2011_mother == "P") & (df.CE_2014_mother == "C")).astype("int")

In [ ]:
df.CE_2011_father.value_counts(dropna = False)

In [ ]:
print(df.Passage_temps_plein_father.value_counts())
print(df.Passage_temps_plein_mother.value_counts())
print(df.Passage_temps_partiel_father.value_counts())
print(df.Passage_temps_partiel_mother.value_counts())

In [ ]:
df[~(df.CE_2011_father.isna()| df.CE_2014_father.isna())].CE_2011_father

In [ ]:
df

Retire bien du sample les observations ou la variable CE n'est pas renseignée.

In [ ]:
###Basic
regression_fit_list = list()
for var in ["Passage_temps_partiel", "Passage_temps_plein"]:
    for parent in ["father", "mother"]:
        endogenous_var = "{}_{}".format( var, parent)
        print(endogenous_var)
        df_temp = df[~(df["CE_2011_{}".format(parent)].isna()| df["CE_2014_{}".format(parent)].isna())]
        m = sm.OLS.from_formula("({})~   {} ".format(endogenous_var,
                ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                          between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                          interaction_child_between_threshold+interaction_child_over_threshold))
                            , df_temp.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                                      "&(NBPAC_2011 >= 0)")) #SAME NBPAC
        fit = m.fit()
        regression_fit_list.append(fit)



In [ ]:
print(summary_col(regression_fit_list, stars = True, info_dict={'adjusted-R2':lambda x:"{:4.2f}%".format(x.rsquared_adj*100), "N":lambda x:"{:d}".format(int(x.nobs))}))

In [ ]:
((df.CE_2011_father == "C") & (df.CE_2014_father == "P"  )).value_counts()

In [ ]:
((df.CE_2011_father == "P") & (df.CE_2014_father == "C"  )).value_counts()

In [ ]:
((df.CE_2011_mother == "P") & (df.CE_2014_mother == "C"  )).value_counts()

In [ ]:
df.CE_2011_mother.value_counts(dropna = False)

In [ ]:
df.Age_father

In [ ]:
df["Baby_in_household"] = (df.NBPAC_2011>0)&(df.Age_eldest_child<=2)

In [ ]:
###Basic
max_child = 4
regression_fit_list = list()
for var in ["Start", "Stop"]:
    for parent in ["father", "mother"]:
        endogenous_var = "{}_to_work_{}".format( var, parent)
        m = sm.Probit.from_formula("({})~   {} +Age_father+Age_mother".format(endogenous_var,
                ' + '.join(children_number[:max_child]+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                          between_threshold_for_child_rank[:max_child]+children_number[:max_child]+ over_threshold_for_child_rank[:max_child]+
                          interaction_child_between_threshold[:max_child]+interaction_child_over_threshold[:max_child]))
                            , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                                      "&(NBPAC_2011 == NBPAC_2014)&(NBPAC_2011 <= @max_child)&(@both_wage)")) #SAME NBPAC
        fit = m.fit(maxiter=100)
        regression_fit_list.append(fit)



In [ ]:
###Basic
max_child = 4
regression_fit_list = list()
for var in ["Start", "Stop"]:
    for parent in ["father", "mother"]:
        endogenous_var = "{}_to_work_{}".format( var, parent)
        m = sm.Probit.from_formula("({})~   {} + Baby_in_household+Age_father+Age_mother".format(endogenous_var,
                ' + '.join(children_number[:max_child]+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                          between_threshold_for_child_rank[:max_child]+children_number[:max_child]+ over_threshold_for_child_rank[:max_child]+
                          interaction_child_between_threshold[:max_child]+interaction_child_over_threshold[:max_child]))
                            , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                                      "&(NBPAC_2011 == NBPAC_2014)&(NBPAC_2011 <= @max_child)&(@both_wage)")) #SAME NBPAC
        fit = m.fit(maxiter=100)
        regression_fit_list.append(fit)



In [ ]:
###Basic
max_child = 4
regression_fit_list = list()
for var in ["Start", "Stop"]:
    for parent in ["father", "mother"]:
        endogenous_var = "{}_to_work_{}".format( var, parent)
        m = sm.Probit.from_formula("({})~   {} + Baby_in_household+Age_father+Age_mother".format(endogenous_var,
                ' + '.join(children_number[:max_child]+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                         flatten_thresholds_list+hardcoded_between[:max_child]+hardcoded_over[:max_child]))
                            , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                                      "&(NBPAC_2011 == NBPAC_2014)&(NBPAC_2011 <= @max_child)&(@both_wage)")) #SAME NBPAC
        fit = m.fit(maxiter=100)
        regression_fit_list.append(fit)



In [ ]:
df.Age_eldest_child.hist(bins =100)

In [ ]:
df.Age_eldest_child.hist(bins =100)

In [ ]:
print(summary_col(regression_fit_list, stars = True, info_dict={'adjusted-R2':lambda x:"{:4.2f}%".format(x.rsquared_adj*100), "N":lambda x:"{:d}".format(int(x.nobs))}))

#### By looking at DADS

In [ ]:
(df.ZVALM_2011<=200).value_counts(normalize = True)

### Retirement

In [ ]:
df["Father_get_retired"] = ((df["YRSTI_2011_father"]<2000) & (df["YRSTI_2014_father"]>6000)).astype("int")
df["Mother_get_retired"] = ((df["YRSTI_2011_mother"]<2000) & (df["YRSTI_2014_mother"]>6000)).astype("int")

In [ ]:
###Basic
max_child = 4
regression_fit_list = list()

for parent in ["Father", "Mother"]:
    endogenous_var = "{}_get_retired".format(  parent)
    m = sm.Probit.from_formula("({})~   {} + Baby_in_household+Age_father+Age_mother".format(endogenous_var,
            ' + '.join(children_number[:max_child]+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                       flatten_thresholds_list+hardcoded_between[:max_child]+hardcoded_over[:max_child]))
                        , df.query("((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))"+
                                  "&(NBPAC_2011 == NBPAC_2014)&(NBPAC_2011 <= @max_child)&(@both_wage)")) #SAME NBPAC
    fit = m.fit(maxiter=100)
    regression_fit_list.append(fit)

print(summary_col(regression_fit_list, stars = True, info_dict={'adjusted-R2':lambda x:"{:4.2f}%".format(x.rsquared_adj*100), "N":lambda x:"{:d}".format(int(x.nobs))}))

In [ ]:
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"+
                              "&(NBPAC_2011 == NBPAC_2014)")) #SAME NBPAC
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("(200000>Rbg_2011>50000)"+
                              "&(NBPAC_2011 == NBPAC_2014)&(ZPERM_2011<50)&(ZPERM_2014<50)")) #SAME NBPAC
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))"))
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("((200000>Rbg_2011>50000)&(200000>Rbg_2014>50000))&(ZPERM_2011<50)&(ZPERM_2014<50)"))
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("((200000>Rbg_2011>50000)|(200000>Rbg_2014>50000))&(ZRSTM_2014<50)"))
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
m = sm.OLS.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df.query("(200000>Rbg_2011>50000)|(200000>Rbg_2014>50000)&(ZPERM_2011<50)"))
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
## Controls
# m = sm.OLS.from_formula("(Delta_rbg)~   {} + child_1:Over_threshold_3_child + child_2:Over_threshold_3_child+ child_3:Over_threshold_4_child".format(
#         ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
#                   between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
#                   interaction_child_between_threshold+interaction_child_over_threshold))
#                     , df)
# fit = m.fit()
# summary = fit.summary(); print(summary)

In [ ]:
## Controls
# m = sm.OLS.from_formula("(Delta_rbg)~   {}  ".format(
#         ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
#                   between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
#                   interaction_child_between_threshold+interaction_child_over_threshold))
#                     , df.query("NBPAC_2011>0"))
# fit = m.fit()
# summary = fit.summary(); print(summary)

In [ ]:
m = sm.RLM.from_formula("(Delta_rbg)~   {}  ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                    , df)
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
#Logarithm
# df_temp = df[df.Rbg_2011>0]
# m = sm.OLS.from_formula("np.log(Rbg_2014/Rbg_2011)~   {}  ".format(
#         ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
#                   between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
#                   interaction_child_between_threshold+interaction_child_over_threshold))
#                     , df_temp)
# fit = m.fit()
# summary = fit.summary(); print(summary)

In [ ]:
## RLM
# m = sm.RLM.from_formula("(Delta_rbg)~   {}  ".format(
#         ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
#                   between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
#                   interaction_child_between_threshold+interaction_child_over_threshold))
#                     , df)
# fit = m.fit()
# summary = fit.summary(); print(summary)

## Young vs. Old

In [ ]:
df.Age_father.mean()

In [ ]:
m = sm.RLM.from_formula("(Delta_rbg)~   {}  ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                                                                                        , df.query("(Age_father<=46)"))
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
m = sm.RLM.from_formula("(Delta_rbg)~   {}  ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                                                                                        , df.query("(Age_father>46)"))
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
m = sm.RLM.from_formula("(Delta_rbg)~   {}  ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                                                                                        , df.query("NBPAC_2011!=0"))
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
between_threshold_for_child_rank[0]

In [ ]:
### Stays in same treatment
# m = sm.OLS.from_formula("(Delta_rbg)~   {}  ".format(
#         ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
#                   between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
#                   interaction_child_between_threshold+interaction_child_over_threshold))
#                                                                                         , df.query("Stays_in_same_treatment==True"))
# fit = m.fit()
# summary = fit.summary(); print(summary)

In [ ]:
m = sm.RLM.from_formula("(Delta_rbg)~   {} +Rbg_splines ".format(
        ' + '.join(children_number #below_threshold_for_child_rank+interaction_child_below_thresholds
                  +between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                                                                                        , df.sample(frac = 1))
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
sm.QuantReg.from_formula("(Delta_rbg)~   {} ".format(
        ' + '.join(children_number+interaction_child_over_threshold[:-3]
                   
                 ))
            , df.sample(frac = 1)).fit(q =0.95).summary()

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }<\style>"))

In [ ]:
df.loc[((df.Age_father>58)&(df.Age_mother<62)), 'Likely_to_get_retired'] = True

In [ ]:
df["Delta_ZIMPOM"] = df.ZIMPOM_2014 - df.ZIMPOM_2011

In [ ]:
m = sm.RLM.from_formula("(Delta_ZIMPOM)~   {} - 1 + Not_touched_by_reform + Delta_Rbg_times_not_touched_reform".format(
        ' + '.join( interaction_child_over_threshold+interactions_rbg_over+interactions_rbg_between))
            , df.sample(frac = 1))
fit = m.fit()
fit.conf_int()
summary = fit.summary(alpha = 0.01); print(summary)

In [ ]:
interaction

In [ ]:
crmi=child_rank_min = 0
crma = child_rank_max = 7
m = sm.RLM.from_formula("(Delta_ZIMPOM)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number[crmi:crma]+ over_threshold_for_child_rank+
                  interaction_child_between_threshold[crmi:crma]+interaction_child_over_threshold[crmi:crma]))
                    , df
                       )
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
df.Age_father.isnull().value_counts()

In [ ]:
crmi=child_rank_min = 0
crma = child_rank_max = 7
m = sm.RLM.from_formula("(Delta_ZIMPOM)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number[crmi:crma]+ over_threshold_for_child_rank+
                  interaction_child_between_threshold[crmi:crma]+interaction_child_over_threshold[crmi:crma]))
                    , df.query("(Age_father<45)&(Age_mother<45)  \
                                      & (ZIMPOM_2011>0) &(ZIMPOM_2014>0)&(NBPAC_2011!={} ) &(NBPAC_2011<={} ) ".
                                        format(crmi, crma,))
                       )
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:

m = sm.RLM.from_formula("(Delta_ZIMPOM)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number[:]+ over_threshold_for_child_rank+
                  interaction_child_between_threshold[:]+interaction_child_over_threshold[:]))
                    , df.query("(Age_father<45)&(Age_mother<45)  \
                                      & (ZIMPOM_2011>0) &(ZIMPOM_2014>0) ")
                       )
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
# m = sm.RLM.from_formula("(Delta_ZIMPOM)~   {} - 1 + Not_touched_by_reform + Delta_Rbg_times_not_touched_reform".format(
#         ' + '.join( interactions_child_over+interactions_rbg_over+interactions_rbg_between))
#             , df.query("(Age_me<45) \
#                                       & (ZIMPOM_2011>0) &(ZIMPOM_2014>0) ").sample(frac = 1))
# fit = m.fit()
# fit.conf_int()
# summary = fit.summary(alpha = 0.01); print(summary)

In [ ]:
# m = sm.RLM.from_formula("(Delta_ZIMPOM)~   {} - 1 + Not_touched_by_reform + Delta_Rbg_times_not_touched_reform".format(
#         ' + '.join( interactions_child_over+interactions_rbg_over+interactions_rbg_between))
#             , df.query("(Age_me<45) \
#                                       & (ZIMPOM_2011>0) &(ZIMPOM_2014>0) ").sample(frac = 1))
# fit = m.fit()
# fit.conf_int()
# summary = fit.summary(alpha = 0.01); print(summary)

In [ ]:
# m = sm.RLM.from_formula("(Delta_ZIMPOM)~   {} - 1 + Not_touched_by_reform + Delta_Rbg_times_not_touched_reform".format(
#         ' + '.join( interactions_child_over+interactions_rbg_over+interactions_rbg_between))
#             , df.query("(Age_me<45) \
#                                       & (ZIMPOM_2011>0) &(ZIMPOM_2014>0) ").sample(frac = 1))
# fit = m.fit()
# fit.conf_int()
# summary = fit.summary(alpha = 0.01); print(summary)

In [ ]:
### Do not modify put in article !
m = sm.OLS.from_formula("(Delta_ZIMPOM)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number[:]+ over_threshold_for_child_rank+
                  interaction_child_between_threshold[:]+interaction_child_over_threshold[:]))
                                                                                        , df)
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
m = sm.RLM.from_formula("(Delta_ZIMPOM)~   {}+  "
    " (Delta_rbg:child_1):Between_threshold_1_child - Delta_rbg:child_1:Between_threshold_1_child[False] ".format(
        ' + '.join(children_number[:]+ over_threshold_for_child_rank+
                  interaction_child_over_threshold[:]))
                                                                                        , df)
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
from patsy import ModelDesc

In [ ]:
ModelDesc.from_formula("y~a.b")

In [ ]:
# m = sm.RLM.from_formula("(Delta_ZIMPOM)~   {}+  "
#     " (child_1):Between_threshold_1_child+"
#    " Delta_rbg:child_2/Between_threshold_2_child+Delta_rbg/child_3/Between_threshold_3_child +Delta_rbg/child_4/Between_threshold_4_child ".format(
#         ' + '.join(between_threshold_for_child_rank+children_number[:]+ over_threshold_for_child_rank+
#                   interaction_child_between_threshold[:]+interaction_child_over_threshold[:]))
#                                                                                         , df.sample(frac = 0.5))
# fit = m.fit()
# summary = fit.summary(); print(summary)

In [ ]:
df.Over_threshold_1_child = df.Over_threshold_1_child.astype("bool")

In [ ]:
df["Interactions_rbg_child1_between"] = df.eval("Delta_rbg*child_1*Between_threshold_1_child")
df["Interactions_rbg_child2_between"] = df.eval("Delta_rbg*child_2*Between_threshold_2_child")
df["Interactions_rbg_child3_between"] = df.eval("Delta_rbg*child_3*Between_threshold_3_child")
df["Interactions_rbg_child4_between"] = df.eval("Delta_rbg*child_4*Between_threshold_4_child")
df["Interactions_rbg_child5_between"] = df.eval("Delta_rbg*child_5*Between_threshold_5_child")
df["Interactions_rbg_child6_between"] = df.eval("Delta_rbg*child_6*Between_threshold_6_child")



df["Interactions_rbg_child1_over"] = df.eval("Delta_rbg*child_1*Over_threshold_1_child")
df["Interactions_rbg_child2_over"] = df.eval("Delta_rbg*child_2*Over_threshold_2_child")
df["Interactions_rbg_child3_over"] = df.eval("Delta_rbg*child_3*Over_threshold_3_child")
df["Interactions_rbg_child4_over"] = df.eval("Delta_rbg*child_4*Over_threshold_4_child")
df["Interactions_rbg_child5_over"] = df.eval("Delta_rbg*child_5*Over_threshold_5_child")
df["Interactions_rbg_child6_over"] = df.eval("Delta_rbg*child_6*Over_threshold_6_child")


df["Interactions_child1_over"] = (df["child_1"]* df["Over_threshold_1_child"]).astype("int")
df["Interactions_child2_over"] = (df["child_2"]* df["Over_threshold_2_child"]).astype("int")
df["Interactions_child3_over"] = (df["child_3"]* df["Over_threshold_3_child"]).astype("int")
df["Interactions_child4_over"] = (df["child_4"]* df["Over_threshold_4_child"]).astype("int")
df["Interactions_child5_over"] = (df["child_5"]* df["Over_threshold_5_child"]).astype("int")
df["Interactions_child6_over"] = (df["child_6"]* df["Over_threshold_6_child"]).astype("int")


In [ ]:
interactions_rbg_between = ["Interactions_rbg_child{}_between".format(child) for child in range(1,7)]
interactions_rbg_over = ["Interactions_rbg_child{}_over".format(child) for child in range(1,7)]
interactions_child_over = ["Interactions_child{}_over".format(child) for child in range(1,7)]

In [ ]:
m = sm.OLS.from_formula("(Delta_ZIMPOM)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number[:]+ over_threshold_for_child_rank+
                  interaction_child_between_threshold[:]+interaction_child_over_threshold[:]))
                                                                                        , df)
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
df["Delta_Rbg_times_not_touched_reform"] = df.Not_touched_by_reform * df.Delta_rbg

In [ ]:
df["Delta_ZIMPOM"] = df.ZIMPOM_DECL_2014 - df.ZIMPOM_2011

In [ ]:
### To_keep article !!
m = sm.RLM.from_formula("(Delta_ZIMPOM)~   {} - 1 + Not_touched_by_reform + Delta_Rbg_times_not_touched_reform".format(
        ' + '.join( interactions_child_over+interactions_rbg_over+interactions_rbg_between))
                                                                                        , df.query("(200000>Rbg_2011>50000)|(200000>Rbg_2014>50000)").sample(frac = 1))
fit = m.fit()
fit.conf_int()
summary = fit.summary(alpha = 0.01); print(summary)

In [ ]:
### To_keep article !!
m = sm.RLM.from_formula("(Delta_ZIMPOM)~   {} - 1 + Not_touched_by_reform + Delta_Rbg_times_not_touched_reform".format(
        ' + '.join( interactions_child_over+interactions_rbg_over+interactions_rbg_between))
                                                                                        , df.query("(200000>Rbg_2011>50000)&(200000>Rbg_2014>50000)").sample(frac = 1))
fit = m.fit()
fit.conf_int()
summary = fit.summary(alpha = 0.01); print(summary)

In [ ]:
print((df.ZIMPOM_2014>=0).value_counts() )
print((df.ZIMPOM_2011>=0).value_counts()   )

In [ ]:
df["Delta_Rbg_times_not_touched_reform"] = df.Not_touched_by_reform * df.Delta_rbg

In [ ]:
### To_keep article !!
m = sm.OLS.from_formula("(Delta_ZIMPOM)~   {} - 1 + Not_touched_by_reform + Delta_Rbg_times_not_touched_reform".format(
        ' + '.join( interactions_child_over+interactions_rbg_over+interactions_rbg_between))
                                                                                        , df.query("Stays_in_same_treatment").sample(frac = 1))
fit = m.fit()
fit.conf_int()
summary = fit.summary(alpha = 0.01); print(summary)

In [ ]:
### To_keep article !!
m = sm.OLS.from_formula("(Delta_ZIMPOM)~   {} - 1 + Not_touched_by_reform + Delta_Rbg_times_not_touched_reform".format(
        ' + '.join( interactions_child_over+interactions_rbg_over+interactions_rbg_between))
                                                                                        , df.sample(frac = 1))
fit = m.fit()
fit.conf_int()
summary = fit.summary(alpha = 0.01); print(summary)

In [ ]:
### To_keep article !!
m = sm.RLM.from_formula("(Delta_ZIMPOM)~   {} - 1 + Not_touched_by_reform + Delta_Rbg_times_not_touched_reform".format(
        ' + '.join(interactions_child_over[:-1]
                   +interactions_rbg_over[:-1]
                   +interactions_rbg_between[:-1]))
                                                                                        , df.sample(frac = 1))
fit = m.fit()
fit.conf_int()
summary = fit.summary(alpha = 0.01); print(summary)

In [ ]:
### only those hom pays tax
m = sm.OLS.from_formula("(Delta_ZIMPOM)~   {} - 1 + Not_touched_by_reform + Delta_Rbg_times_not_touched_reform".format(
        ' + '.join( interactions_child_over+interactions_rbg_over+interactions_rbg_between))
                                                                                        , df.query("(ZIMPOM_2014>0) & (ZIMPOM_2011>0)").sample(frac = 1))
fit = m.fit()
fit.conf_int()
summary = fit.summary(alpha = 0.01); print(summary)

In [ ]:
### only those hom pays tax
m = sm.OLS.from_formula("(Delta_ZIMPOM)~   {} - 1 + Not_touched_by_reform + Delta_Rbg_times_not_touched_reform".format(
        ' + '.join( interactions_child_over+interactions_rbg_over+interactions_rbg_between))
                                                                                        , df.query("(ZIMPOM_2014>0) & (ZIMPOM_2011>0)").sample(frac = 1))
fit = m.fit()
fit.conf_int()
summary = fit.summary(alpha = 0.01); print(summary)

In [ ]:

m = sm.OLS.from_formula("(Delta_ZIMPOM)~   {} - 1".format(
        ' + '.join( interactions_child_over+interactions_rbg_over+interactions_rbg_between))
                                                                                        , df.sample(frac = 1))
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:

m = sm.OLS.from_formula("(Delta_ZIMPOM)~   {} - 1".format(
        ' + '.join( interactions_child_over+interactions_rbg_over+interactions_rbg_between))
                                                                                        , df.sample(frac = 1))
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
import numpy as np

In [ ]:
np.sqrt(145)

In [ ]:
30*0.8+14*0.2

In [ ]:
0.37*0.14+(1-0.38)*0.3

In [ ]:
17*30/14

In [ ]:
m = sm.RLM.from_formula("(Delta_ZIMPOM)~   {}+  "
    " (child_1):Between_threshold_1_child+Delta_rbg/child_2/Between_threshold_2_child+Delta_rbg/child_3/Between_threshold_3_child +Delta_rbg/child_4/Between_threshold_4_child ".format(
        ' + '.join(between_threshold_for_child_rank+children_number[:]+ over_threshold_for_child_rank+
                  interaction_child_between_threshold[:]+interaction_child_over_threshold[:]))
                                                                                        , df.sample(frac = 0.5))
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
any_rank_interation_between = list()
for threshold in between_threshold_for_child_rank:
    any_rank_interation_between +=  ["{}:{}".format(child,threshold) for child in children_number]
any_rank_interation_over = list()
for threshold in over_threshold_for_child_rank:
    any_rank_interation_over +=  ["{}:{}".format(child,threshold) for child in children_number]

###### pOTENTIAL ROBUSTNESS CHECK

In [ ]:
m = sm.RLM.from_formula("(Delta_rbg)~   {} + Between_threshold_3_child:child_4".format(
        ' + '.join(between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold+ any_rank_interation_between+any_rank_interation_over ))
                                                                                        , df)
fit = m.fit()
#summary = fit.summary(); print(summary)

In [ ]:
df["Birth"] = (df.NBPAC_2011 < df.NBPAC_2014).astype('int')
(df.NBPAC_2011 < df.NBPAC_2014).value_counts(normalize = False)

### Add control variables to triple diff

In [ ]:
# Age pere, Age mere, localité, Taille appartement (CSDEP_2011), 
# date d'entrée dans le local (fisc_l Datepers), nombre de pièces (fisc_l, NBPIEC), OCC code occupation du local
#Education

In [ ]:
fisc_logement = load_fisc_l_by_year(2014)

In [ ]:
fisc_logement.OCC.value_counts(dropna=False)

In [ ]:
fisc_logement["Proprietaire"] = fisc_logement.OCC == "P"

In [ ]:
df["ID_FISC_LOG_DIFF"] = df.ID_FISC_LOG_DIFF_2014

In [ ]:
df_log = pd.merge(df, fisc_logement, on = "ID_FISC_LOG_DIFF")

In [ ]:
df_log["Anciennete_dans_logement"] = 2015 - df_log.DATEPERS.astype("int64")
df_log["Rbg_2011_squared"] = df_log.Rbg_2011 **2

In [ ]:
#df_log.CSDEP_2014.value_counts(dropna = False)

In [ ]:
df_log["CSDEP"] = df_log.CSDEP_2014.replace({"2A": "40", "2B":"40"}).astype("int")

In [ ]:
df_log.shape

In [ ]:
df_log["Age_wo_squared"] = df_log.Age_wo **2
df_log["Age_me_squared"] = df_log.Age_me ** 2 

In [ ]:
(( merge.ZTSAM_2011*0.9) + merge.ZRAGM_2011 + merge.ZRACM_2011 + merge.ZRNCM_2011 + merge.ZFONM_2011 + merge.ZPERM_2011*0.9
                    + merge.ZRICM_2011)

In [ ]:
only_wage_income = df_log[["ZRAGM_2011", "ZRACM_2011","ZRNCM_2011", "ZFONM_2011", "ZPERM_2011", "ZRICM_2011",
                                             "ZRAGM_2014", "ZRACM_2014","ZRNCM_2014", "ZFONM_2014", "ZPERM_2014", "ZRICM_2014"]].sum(axis = 1) == 0

In [ ]:
only_wage_or_retirement_income = df_log[["ZSALM","ZRAGM_2011", "ZRACM_2011","ZRNCM_2011", "ZFONM_2011", "ZRICM_2011",
                                             "ZRAGM_2014", "ZRACM_2014","ZRNCM_2014", "ZFONM_2014", "ZRICM_2014"]].sum(axis = 1) == 0

In [ ]:
only_liberals_income = df_log[["ZRAGM_2011", "ZRACM_2011","ZSALM_2011", "ZFONM_2011", "ZPERM_2011", "ZRICM_2011",
                                             "ZRAGM_2014", "ZRACM_2014","ZSALM_2014", "ZFONM_2014", "ZPERM_2014", "ZRICM_2014"]].sum(axis = 1) == 0

In [ ]:
only_wage_income.shape

In [ ]:
df_log["Delta_ZRNCM"] =  df_log.ZRNCM_2011- df.ZRNCM_2011

In [ ]:

m = sm.RLM .from_formula("(Delta_Rbg)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number[:-2]+ over_threshold_for_child_rank+
                  interaction_child_between_threshold[:-2]+interaction_child_over_threshold[:-2]))
        + "+ Proprietaire +CSDEP +Age_Elder_child+Age_Youngest_child"
                                            , df_log.query("(NBPAC_2011<5)&(ZRNCM_2011>0)").sample(frac = 1))
fit = m.fit()
summary = fit.summary(); print(summary)                                                                                                                                                                                                                                                                                                 

In [ ]:
m = sm.RLM.from_formula("(Delta_ZIMPOM)~   {} - 1 + Not_touched_by_reform + Delta_Rbg_times_not_touched_reform".format(
        ' + '.join( interactions_child_over+interactions_rbg_over+interactions_rbg_between))
            , df.query("(Age_me<45) \
                                      (NBPAC_2011<5)&(ZRNCM_2011>0)").sample(frac = 1))
fit = m.fit()
fit.conf_int()
summary = fit.summary(alpha = 0.01); print(summary)

In [ ]:

m = sm.RLM .from_formula("(Delta_ZIMPOM)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number[:-2]+ over_threshold_for_child_rank+
                  interaction_child_between_threshold[:-2]+interaction_child_over_threshold[:-2]))
                                            , df_log[only_wage_or_retirement_income].query("NBPAC_2011<5").sample(frac = 1))
fit = m.fit()
summary = fit.summary(); print(summary)                                                                                                                                                                                                                                                                                                 

In [ ]:
m = sm.RLM .from_formula("(Delta_rbg)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number[:]+ over_threshold_for_child_rank+
                  interaction_child_between_threshold[:]+interaction_child_over_threshold[:]))
        + "+ Proprietaire +CSDEP + Age_wo + Age_me+ Age_wo_squared + Age_me_squared"
                , df_log.query("(Age_me<55)& ((NBPAC_2011!=0 ))").sample(frac = 1))
fit = m.fit()
summary = fit.summary(); print(summary)                                                                                                                                                                                                                                                                                                 

In [ ]:
df_log["Mean_age_y_e_children"] = (df_log.Age_Elder_child+df.Age_Youngest_child)/2

In [ ]:
below_threshold_for_child_rank = ["Below_thresholds_{}_child_rbg_2014".format(nb_child) for nb_child in range(1,nb_enf_max_for_annalysis+1)]
interaction_child_below_thresholds  = ["{}:{}".format(child,interaction) for child, interaction in zip(children_number, below_threshold_for_child_rank)]

In [ ]:

for nb_enf in range(1,7):
    df_log["Below_thresholds_{}_child_rbg_2014".format(nb_enf)] = np.zeros(df_log.shape[0])
for nb_enf in range(1,7):
    df_log["Below_thresholds_{}_child_rbg_2014".format(nb_enf)
          ] +=((df_log.Rbg_2011<  threshold_byr_rbg["2014"].iloc[nb_enf-1])).astype("float")

    


In [ ]:
df_log["Has_child"] = (df_log.NBPAC_2011>0).astype("int")
df_log.Has_child.value_counts(dropna=False)

In [ ]:
df_log["Touched_by_reform_bool"] = ~ df_log.Not_touched_by_reform_bool.astype("bool")

In [ ]:
df_log["Not_touched_by_reform_bool"] = df_log["Not_touched_by_reform"].astype("bool")

In [ ]:
(df_log.Below_thresholds_1_child_rbg_2014 == df_log.child_1).value_counts()

In [ ]:
df_not_touched = df_log.query("Not_touched_by_reform_bool")

In [ ]:
df_not_touched.query("Over_threshold_5_child").NBPAC_2011.value_counts()

In [ ]:
interaction_child_below_thresholds  = ["child_2:Below_thresholds_5_child_rbg_2014"]

m = sm.OLS.from_formula("(Delta_rbg)~   {}  ".format(
        ' + '.join(children_number+below_threshold_for_child_rank))
        + "+ Proprietaire +CSDEP + Age_wo + Age_me+ Age_wo_squared + Age_me_squared+YSALI_wo_2011"
        + "+Age_Elder_child:Has_child + Age_Youngest_child:Has_child+Mean_age_y_e_children:Has_child"
        #+"+Age_Elder_child:child_1+Age_Elder_child:child_2+Age_Elder_child:child_3+Age_Elder_child:child_4+Age_Elder_child:child_5"
        #+"+Age_Youngest_child:child_1+Age_Youngest_child:child_2+Age_Youngest_child:child_3+Age_Youngest_child:child_4+Age_Youngest_child:child_5"
                                                                                        , df_log.query("Not_touched_by_reform_bool"))
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
m = sm.OLS.from_formula("(Delta_rbg)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold)) , df)
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
df_log["CNE"] = df_log.CNE.astype("str")

In [ ]:
df_log.shape

In [ ]:
m = sm.OLS.from_formula("(Delta_rbg)~   {} + ".format(
        ' + '.join(between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
        + "AGERF_2011 +Anciennete_dans_logement + Rbg_2011_squared   "
        + "+YSALI_wo_2011+YSALI_me_2011 " + "+Age_Youngest_child+Rbg_splines"
#        + "+ Age_Youngest_child:child_1+ Age_Youngest_child:child_2 + Age_Youngest_child:child_3++ Age_Youngest_child:child_4+ Age_Youngest_child:child_5"# CSDEP_2011+CHAUFF + CNE
#        +  "+ Age_Elder_child:child_1+ Age_Elder_child:child_2 + Age_Elder_child:child_3++ Age_Elder_child:child_4+ Age_Elder_child:child_5"                                                                        
                        , df_log)
fit = m.fit()
summary = fit.summary(); print(summary)

### Tres bon robustness check ! 

Mais on ne devrait rien avoir de significatif...

In [ ]:
m = sm.OLS.from_formula("(counterfactual_delta_rbg)~   {} + ".format(
        ' + '.join(between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
        + "AGERF_2011 +Anciennete_dans_logement + Rbg_2011_squared   "
        + "+YSALI_wo_2011+YSALI_me_2011 " + "+Age_Youngest_child"
#        + "+ Age_Youngest_child:child_1+ Age_Youngest_child:child_2 + Age_Youngest_child:child_3++ Age_Youngest_child:child_4+ Age_Youngest_child:child_5"# CSDEP_2011+CHAUFF + CNE
#        +  "+ Age_Elder_child:child_1+ Age_Elder_child:child_2 + Age_Elder_child:child_3++ Age_Elder_child:child_4+ Age_Elder_child:child_5"                                                                        
                        , df_log.sample(frac = 0.5))
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
df_log_not_dropped_by_statsmodels = df_log.drop(m.data.missing_row_idx)

In [ ]:
df_log_not_dropped_by_statsmodels.shape

In [ ]:
fit.fittedvalues.shape

In [ ]:
fit.resid.shape

In [ ]:
print np.mean(fit.resid * df_log_not_dropped_by_statsmodels.child_1*df_log_not_dropped_by_statsmodels.Over_threshold_1_child  )
print np.mean(fit.resid * df_log_not_dropped_by_statsmodels.child_2*df_log_not_dropped_by_statsmodels.Over_threshold_2_child  )
print np.mean(fit.resid * df_log_not_dropped_by_statsmodels.child_3*df_log_not_dropped_by_statsmodels.Over_threshold_3_child  )
print np.mean(fit.resid * df_log_not_dropped_by_statsmodels.child_4*df_log_not_dropped_by_statsmodels.Over_threshold_4_child  )
print np.mean(fit.resid * df_log_not_dropped_by_statsmodels.child_5*df_log_not_dropped_by_statsmodels.Over_threshold_5_child  )
print np.mean(fit.resid * df_log_not_dropped_by_statsmodels.child_6*df_log_not_dropped_by_statsmodels.Over_threshold_6_child  )
print np.mean(fit.resid * df_log_not_dropped_by_statsmodels.child_6*df_log_not_dropped_by_statsmodels.Over_threshold_6_child  )
print np.mean(fit.resid * df_log_not_dropped_by_statsmodels.child_1*df_log_not_dropped_by_statsmodels.Between_threshold_1_child  )
print np.mean(fit.resid * df_log_not_dropped_by_statsmodels.child_2*df_log_not_dropped_by_statsmodels.Between_threshold_2_child  )
print np.mean(fit.resid * df_log_not_dropped_by_statsmodels.child_3*df_log_not_dropped_by_statsmodels.Between_threshold_3_child  )
print np.mean(fit.resid * df_log_not_dropped_by_statsmodels.child_4*df_log_not_dropped_by_statsmodels.Between_threshold_4_child  )
print np.mean(fit.resid * df_log_not_dropped_by_statsmodels.child_5*df_log_not_dropped_by_statsmodels.Between_threshold_5_child  )
print np.mean(fit.resid * df_log_not_dropped_by_statsmodels.child_6*df_log_not_dropped_by_statsmodels.Between_threshold_6_child  )


In [ ]:
df_log.child_1.shape

In [ ]:
fit.resid.shape

In [ ]:
from scipy import stats

In [ ]:
print stats.ttest_1samp((fit.resid * df_log_not_dropped_by_statsmodels.child_1*df_log_not_dropped_by_statsmodels.Over_threshold_1_child) , 0)
print stats.ttest_1samp((fit.resid * df_log_not_dropped_by_statsmodels.child_2*df_log_not_dropped_by_statsmodels.Over_threshold_2_child) , 0)
print stats.ttest_1samp((fit.resid * df_log_not_dropped_by_statsmodels.child_3*df_log_not_dropped_by_statsmodels.Over_threshold_3_child) , 0)
print stats.ttest_1samp((fit.resid * df_log_not_dropped_by_statsmodels.child_4*df_log_not_dropped_by_statsmodels.Over_threshold_4_child) , 0)
print stats.ttest_1samp((fit.resid * df_log_not_dropped_by_statsmodels.child_5*df_log_not_dropped_by_statsmodels.Over_threshold_5_child) , 0)
print stats.ttest_1samp((fit.resid * df_log_not_dropped_by_statsmodels.child_6*df_log_not_dropped_by_statsmodels.Over_threshold_6_child) , 0)
print "\n"
print stats.ttest_1samp((fit.resid * df_log_not_dropped_by_statsmodels.child_1*df_log_not_dropped_by_statsmodels.Between_threshold_1_child) , 0)
print stats.ttest_1samp((fit.resid * df_log_not_dropped_by_statsmodels.child_2*df_log_not_dropped_by_statsmodels.Between_threshold_2_child) , 0)
print stats.ttest_1samp((fit.resid * df_log_not_dropped_by_statsmodels.child_3*df_log_not_dropped_by_statsmodels.Between_threshold_3_child) , 0)
print stats.ttest_1samp((fit.resid * df_log_not_dropped_by_statsmodels.child_4*df_log_not_dropped_by_statsmodels.Between_threshold_4_child) , 0)
print stats.ttest_1samp((fit.resid * df_log_not_dropped_by_statsmodels.child_5*df_log_not_dropped_by_statsmodels.Between_threshold_5_child) , 0)
print stats.ttest_1samp((fit.resid * df_log_not_dropped_by_statsmodels.child_6*df_log_not_dropped_by_statsmodels.Between_threshold_6_child) , 0)


In [ ]:
print df_log.Over_threshold_1_child.isnull().value_counts()
print df_log.child_1.isnull().value_counts()
print fit.resid.isnull().value_counts()

In [ ]:
print fit.resid[(fit.resid * df_log.child_1).isnull()]

In [ ]:
fit.resid.shape

In [ ]:
df_log.child_1.shape

In [ ]:
print (df_log.child_1*df_log.Over_threshold_1_child).isnull().value_counts()

In [ ]:
(fit.resid * df_log.child_1*df_log.Over_threshold_1_child)[
    (fit.resid * df_log.child_1*df_log.Over_threshold_1_child).isnull()]

In [ ]:
stats.ttest_1samp([1,2,3] , 0)

In [ ]:
(fit.resid * df_log.child_1*df_log.Over_threshold_1_child).describe()

In [ ]:
(fit.resid[df_log.Over_threshold_1_child ] 
 * (df_log.child_1*df_log.Over_threshold_1_child)[df_log.Over_threshold_1_child ] 
).mean()

In [ ]:
sm.stats.ttest_ind((fit.resid * df_log.child_1*df_log.Over_threshold_1_child)[:900],
                   np.zeros(fit.resid.shape[0]))

### Elasticity a la Gruber Saez (2002)

##### Change in MTR

$log(z_1$

In [ ]:
df_log["log_ratio_income"] = np.log(df_log.Rbg_2014/df_log.Rbg_2011)

In [ ]:
import importlib
year = 2014

In [ ]:
ir = ir_2014

In [ ]:
parts_fiscales_enfants = ir.parts_fiscales_enfants(df.NBPAC_2011)

In [ ]:
df["ZTSAM_10_percent_2014"] =  df["ZTSAM_2014"]*0.9


df["Irpp_2014"] = ir.ir_avec_plafond_qf_enfant(df.Rbg_2014, parts_fiscales_enfants)
df["Irpp_2014_plus_10_euros"] = ir.ir_avec_plafond_qf_enfant(df.Rbg_2014+10, parts_fiscales_enfants)

df["Counterfactual_Irpp_2014"] = ir_2011.ir_avec_plafond_qf_enfant(df.Rbg_2014, parts_fiscales_enfants)
df["Counterfactual_Irpp_2014_plus_10_euros"] = ir_2011.ir_avec_plafond_qf_enfant(df.Rbg_2014+10, parts_fiscales_enfants)

df["Irpp_2011"] = ir_2011.ir_avec_plafond_qf_enfant(df.Rbg_2011, parts_fiscales_enfants)
df["Irpp_2011_plus_10_euros"] = ir_2011.ir_avec_plafond_qf_enfant(df.Rbg_2011+10, parts_fiscales_enfants)



df["Counterfactual_Irpp_2011"] = ir.ir_avec_plafond_qf_enfant(df.Rbg_2011, parts_fiscales_enfants)
df["Counterfactual_Irpp_2011_plus_10_euros"] = ir.ir_avec_plafond_qf_enfant(df.Rbg_2011+10, parts_fiscales_enfants)

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.scatter(df.ZIMPOM_2011,df.Irpp_2011, s=0.02 )
plt.xlim(0,15000)
plt.ylim(0,15000)

In [ ]:
fig = plt.figure(figsize=(20,10))
df_temp = df#.query("NBPAC_2011 == 2")
plt.scatter(df_temp.ZIMPOM_DECL_2014,df_temp.Irpp_2014, s=0.02 )
plt.xlim(0,15000)
plt.ylim(0,15000)

In [ ]:
erreur_impot = df.ZIMPOM_DECL_2014 - df.Irpp_2014
erreur_impot = erreur_impot[erreur_impot<10000]
sns.distplot(erreur_impot, bins = 200, )
plt.xlim(-10000,10000)

erreur_impot = df.ZIMPOM_2011 - df.Irpp_2011
erreur_impot = erreur_impot[erreur_impot<10000]
sns.distplot(erreur_impot, bins = 200, )
plt.xlim(-10000,10000)

In [ ]:
erreur_impot = df.query('NBPAC_2011==4').ZIMPOM_DECL_2014 - df.query('NBPAC_2011==4').Irpp_2014
erreur_impot = erreur_impot[erreur_impot<10000]
sns.distplot(erreur_impot, bins = 200, )
plt.xlim(-10000,10000)

In [ ]:
df.query('NBPAC_2011==4').ZIMPOM_DECL_2014.hist(bins = 500)
plt.xlim(-5000,10000)

In [ ]:
erreur_impot = df.query('NBPAC_2011==6').ZIMPOM_2011 - df.query('NBPAC_2011==6').Irpp_2011
erreur_impot = erreur_impot[erreur_impot<10000]
sns.distplot(erreur_impot, bins = 200, )
plt.xlim(-10000,10000)

In [ ]:
df.ZIMPOM14_2014.hist(bins = 1000)
plt.xlim(-5000,30000)

### Taux marginal imposition

In [ ]:
df["MTR_2014"] = np.round((df.Irpp_2014_plus_10_euros - df.Irpp_2014)/10,2)
df["MTR_2011"] = np.round((df.Irpp_2011_plus_10_euros - df.Irpp_2011)/10,2)

df["Counterfactual_MTR_2014"
                     ] = np.round((df.Counterfactual_Irpp_2014_plus_10_euros - df.Counterfactual_Irpp_2014)/10,2)

df["Counterfactual_MTR_2011"
                     ] = np.round((df.Counterfactual_Irpp_2011_plus_10_euros - df.Counterfactual_Irpp_2011)/10,2)

In [ ]:
df.Counterfactual_MTR_2014.value_counts(normalize=True).head(5)

In [ ]:
df.Counterfactual_MTR_2011.value_counts(normalize=True).head(5)

In [ ]:
print(df.MTR_2011.value_counts(normalize=True).head(5))
df.MTR_2014.value_counts(normalize=True).head(5)

In [ ]:
df["MTR_2011"].hist(bins = 100)

In [ ]:
df["MTR_2014"].hist(bins = 100)

In [ ]:
(df.Counterfactual_MTR_2014>df.MTR_2014).value_counts()

In [ ]:
((df_log.log_ratio_MTR == df_log.log_ratio_counterfactual_MTR)).value_counts()

In [ ]:
np.log((1-0.41)/(1-0.41))

In [ ]:
df_log.MTR_2014.value_counts().head(4)

In [ ]:
#df_log[(df_log.Counterfactual_MTR_2014>df_log.MTR_2014)][["NBPAC_2011","Rbg_2014", "Irpp_2014", "Counterfactual_Irpp_2014","MTR_2011", "Counterfactual_MTR_2014", "MTR_2014", "log_ratio_counterfactual_MTR", "log_ratio_MTR"]]

In [ ]:
#df_log[["Counterfactual_MTR_2014", "MTR_2014"]]

In [ ]:
((df_log.Irpp_2011_plus_10_euros - df_log.Irpp_2011)/10).hist(bins = 100)

In [ ]:
((df_log.Irpp_2014_plus_10_euros - df_log.Irpp_2014)/10).hist(bins = 100)

In [ ]:
plt.figure(figsize=(15,10))
df_temp = df_log.query("NBPAC_2011>=0")
ax = sns.countplot(x="value", hue = "variable", data = pd.melt(df_temp[["MTR_2011", "MTR_2014"]]),
              order = [0,0.05,0.14,0.3,0.41]) 
for p in ax.patches:
    height = p.get_height()
    height = np.nan_to_num(height)
    ax.text(p.get_x()+p.get_width()/2, height*(1+0.04), '{:1.1f}%'.format(
            (height/(len(df_temp)*2))*100), ha = "center", fontsize= 15)
plt.xlim(0,4.6)
plt.xlabel("Marginal Tax Rate", fontsize = 25)
plt.ylabel(u"Nombre de ménages", fontsize = 25)

In [ ]:
plt.figure(figsize=(15,10))
df_temp = df_log.query("NBPAC_2011>=0")
ax = sns.countplot(x="value", hue = "variable", data = pd.melt(df_temp[["MTR_2011", "MTR_2014"]]),
              order = [0,0.05,0.14,0.3,0.41]) 
for p in ax.patches:
    height = p.get_height()
    height = np.nan_to_num(height)
    ax.text(p.get_x()+p.get_width()/2, height*(1+0.04), '{:1.1f}%'.format(
            (height/(len(df_temp)*2))*100), ha = "center", fontsize= 15)
plt.xlim(0,4.6)
plt.xlabel("Marginal Tax Rate", fontsize = 25)
plt.ylabel(u"Nombre de ménages", fontsize = 25)

In [ ]:

df_temp = df_log.query("NBPAC_2011>=0")
ax = sns.countplot(x="value", hue = "variable", data = pd.melt(df_temp[["MTR_2011", "MTR_2014"]]),
              order = [0,0.05,0.14,0.3,0.41]) 
for p in ax.patches:
    height = p.get_height()
    height = np.nan_to_num(height)
    ax.text(p.get_x()+p.get_width()/2, height+2004, '{:1.1f}%'.format(
            (height/(len(df_temp)*2))*100), ha = "center")
plt.xlim(0,4.6)
plt.xlabel("Marginal Tax Rate")
plt.ylabel(u"Nombre de ménages")

In [ ]:
marginal_rates_2014 = ((df_log.Irpp_2014_plus_10_euros - df_log.Irpp_2014)/10)
marginal_rates_2011 = ((df_log.Irpp_2011_plus_10_euros - df_log.Irpp_2011)/10)

In [ ]:
df_mtr_2011 = pd.DataFrame({"MTR":marginal_rates_2011}); df_mtr_2011["Year"] = 2011
df_mtr_2014 = pd.DataFrame({"MTR":marginal_rates_2014}); df_mtr_2014["Year"] = 2014

In [ ]:
df_mtr = pd.concat((df_mtr_2011, df_mtr_2014))

In [ ]:
pd.round(df_mtr.MTR,2).value_counts()

Change marginal tax rate: 

In [ ]:
np.round(df_log.MTR_2011, 2).value_counts()

In [ ]:
(np.abs(df_log.MTR_2011 - df_log.MTR_2014)<0.01).value_counts(normalize = True)

In [ ]:
((df_log.MTR_2011 - df_log.Counterfactual_MTR_2014)==0).value_counts(normalize = True)

In [ ]:
df_log[["MTR_2011", "MTR_2014", "Counterfactual_MTR_2014", "Counterfactual_MTR_2011", "Rbg_2014", "Rbg_2011", "NBPAC_2011", "NBPAC_2014" ]].head()

Grueber saez : log(z_2014/z_2011) = beta_1 log[(1-MTR_2014)/(1-MTR_2011)] + beta_2 log[(z_2014 - T_2(z_2014))/(z_2011-T_2011(z_2011))]

In [ ]:
df_log["log_ratio_income"] = np.log(df_log.Rbg_2014/df_log.Rbg_2011)
df_log["log_ratio_MTR"] = np.log((1 - df_log.MTR_2014)/(1-(df_log.MTR_2011)))
df_log["log_ratio_disp_income"] = np.log((df_log.Rbg_2014 - df_log.Irpp_2014)/
                                                        (df_log.Rbg_2011 - df_log.Irpp_2011))


df_log["log_ratio_counterfactual_MTR"] = np.log((1 - df_log.Counterfactual_MTR_2014)/(1-(df_log.MTR_2011)))
df_log["log_ratio_counterfactual_disp_income"] = np.log((df_log.Rbg_2014 - df_log.Counterfactual_Irpp_2014)/
                                                        (df_log.Rbg_2011 - df_log.Irpp_2011))                                            

In [ ]:
m = sm.OLS.from_formula("(log_ratio_income)~  log_ratio_MTR +log_ratio_disp_income + "                                            
                        +"+Age_me+Age_wo+Age_wo_squared + child_1*log_ratio_MTR + child_2*log_ratio_MTR + child_3*log_ratio_MTR + child_4*log_ratio_MTR+ child_5*log_ratio_MTR"
                        +"+ child_1*log_ratio_disp_income + child_2*log_ratio_disp_income + child_3*log_ratio_disp_income + child_4*log_ratio_disp_income+ child_5*log_ratio_disp_income", df_log)
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
import statsmodels

In [ ]:
df_log.to_csv("C:/Users/IMPTEMP_A_PACIFIC/Desktop/df_log.csv")

In [ ]:
m = sm.RLM.from_formula("(log_ratio_income)~  log_ratio_MTR + log_ratio_disp_income "                                            
                        , df_log)
fit = m.fit()
summary = fit.summary(); print(summary)

### Counterfactual instrument 

In [ ]:
#df_log.Age_me.value_counts()

In [ ]:
m = sm.OLS.from_formula("(log_ratio_income)~  log_ratio_MTR +log_ratio_income + "                                            
                        +"+np.log(Age_me)+np.log(Age_wo) ", df_log)
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
m = sm.OLS.from_formula("(log_ratio_income)~  log_ratio_MTR + log_ratio_disp_income + \
                        log_ratio_MTR:child_1 +log_ratio_MTR:child_2 + log_ratio_MTR:child_3 + log_ratio_MTR:child_4 + log_ratio_MTR:child_5+Rbg_2011\
                        +log_ratio_income:child_1 +log_ratio_income:child_2 + log_ratio_income:child_3 + log_ratio_income:child_4 + log_ratio_income:child_5+Rbg_2011"                                            
                        +"+Age_me+Age_wo+Age_wo_squared ", df_log)
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
np.corrcoef(df_log.log_ratio_MTR,df_log.log_ratio_disp_income )

### Estimate the income growth (counterfactual income in the absence of the reform)

Deduce the income one would have in the absence of the reform. Strategy : run an estimation of those not touched by the reform then compute the counterfactual income of every one. Then do that multiple times such that the rise in income would not imply to be touched by the reform for any household.

In [ ]:
df_log_save = df_log.copy()

In [ ]:
df_log["Proprietaire"] = df_log.Proprietaire.astype("int")

In [ ]:
# #%%time
# df_temp =  df_log.query("Not_touched_by_reform")
# df_log["Not_touched_by_reform_counterfactual"] = df_log.Not_touched_by_reform
# for counter in range(0,1):
# #df_temp = df_temp.query("Not_touched_by_reform")                                    
#     m = sm.OLS.from_formula("(Rbg_2014)~  "                                           
#                             +"Rbg_2011+Age_me+Age_wo+Age_me_squared+Age_wo_squared + child_1 + child_2+ child_3+ child_4+ child_5 \
#                             +Proprietaire  \
#                             +YSALI_wo_2011+YCHOI_wo_2011+YALRI_wo_2011+YBICI_wo_2011+YBNCI_wo_2011+YRAGI_wo_2011+ YRSTI_wo_2011",
#                             df_log.query("Not_touched_by_reform_counterfactual"))
#     fit = m.fit()
#     df_log["counterfactual_2014"] = np.multiply(df_temp[fit.params.index[1:]].values, fit.params.values[1:]).sum(axis = 1)+ fit.params.values[0]
#     df_log["Not_touched_by_reform"] = (df_log.NBPAC_2011 == 0).astype("int")
#     df_log["Not_touched_by_reform_counterfactual"] = False
#     for nb_enf in range(1,7): ### in modification
#         df_log["Not_touched_by_reform_counterfactual"
#                              ] +=((df_log.counterfactual_2014< threshold_byr_rbg["2014"].iloc[nb_enf-1])
#                     *( df_log.NBPAC_2011 == nb_enf)).astype("bool")
# print 

In [ ]:
for nb_enf in range(1,7):
    df_log["Between_threshold_{}_child_counterfactual".format(nb_enf)] = np.zeros(df_log.shape[0])
for nb_enf in range(1,7): ### in modification
    df_log["Between_threshold_{}_child_counterfactual".format(nb_enf)
          ] +=(((df_log.Counterfactual_rbg_2014>  threshold_byr_rbg["2014"].iloc[nb_enf-1])
               & (df_log.Counterfactual_rbg_2014< threshold_byr_rbg["2011"].iloc[nb_enf-1])) *( df_log.NBPAC_2011 == nb_enf)).astype("float")

for nb_enf in range(1,7):
    df_log["Over_threshold_{}_child_counterfactual".format(nb_enf)] = np.zeros(df_log.shape[0])
for nb_enf in range(1,7): ### in modification
    df_log["Over_threshold_{}_child_counterfactual".format(nb_enf)
          ] +=(((df_log.Counterfactual_rbg_2014> threshold_byr_rbg["2011"].iloc[nb_enf-1]))
               *( df_log.NBPAC_2011 == nb_enf)).astype("float")


# df["Over_threshold_child_rbg_2014"] = np.zeros(df.shape[0])
# df["Over_threshold_child_rbg_2011"] = np.zeros(df.shape[0])
# for nb_enf in range(1,7): ### in modification
#     df["Over_threshold_child_rbg_2014".format(nb_enf)
#           ] +=(((df.Rbg_2014>  threshold_byr_rbg["2014"].iloc[nb_enf-1])
#                & (df.Rbg_2014< threshold_byr_rbg["2011"].iloc[nb_enf-1])) *( df.NBPAC_2011 == nb_enf)).astype("float")
#     df["Over_threshold_child_rbg_2011".format(nb_enf)
#           ] +=(((df.Rbg_2011>  threshold_byr_rbg["2014"].iloc[nb_enf-1])
#                & (df.Rbg_2011< threshold_byr_rbg["2011"].iloc[nb_enf-1])) *( df.NBPAC_2011 == nb_enf)).astype("float")

In [ ]:
df_log.Between_threshold_3_child_counterfactual.value_counts()

In [ ]:
df_log.Between_threshold_3_child.value_counts()

In [ ]:
sns.distplot(df_log.Rbg_2014)
sns.distplot(df_log.Counterfactual_rbg_2014)

In [ ]:
df_log.Between_threshold_5_child.value_counts()

In [ ]:
df_log.Between_threshold_5_child_counterfactual.value_counts()

In [ ]:
number_touched_list = list()
mean_income_list = list()
df_log["Not_touched_by_reform_counterfactual"] = df_log.Not_touched_by_reform.astype("bool")
for counter in range(1,10):

    df_temp =  df_log.query("Not_touched_by_reform_counterfactual") #On enlève les touchés



    #On evalue l'évolution du salaire sur les non touchés.
    m = sm.OLS.from_formula("(Rbg_2014)~  "                                           
                                +"Rbg_2011+Age_me+Age_wo+Age_me_squared+Age_wo_squared + child_1 + child_2+ child_3+ child_4+ child_5 \
                                +Proprietaire  \
                                +YSALI_wo_2011+YCHOI_wo_2011+YALRI_wo_2011+YBICI_wo_2011+YBNCI_wo_2011+YRAGI_wo_2011+ YRSTI_wo_2011",
                                df_temp)
    fit = m.fit()

    # ## On calcule le salaire contrefactuel

    df_log["Counterfactual_rbg_2014"] = np.multiply(
                                        df_log[fit.params.index[1:]].values, fit.params.values[1:]
                                                ).sum(axis = 1)+ fit.params.values[0]
    df_log["Not_touched_by_reform_counterfactual"] = False
    for nb_enf in range(1,7): ### in modification
        df_log["Not_touched_by_reform_counterfactual"
                             ] +=((df_log.Counterfactual_rbg_2014< threshold_byr_rbg["2014"].iloc[nb_enf-1])
                    *( df_log.NBPAC_2011 == nb_enf)).astype("bool")
    #print df_log.Not_touched_by_reform_counterfactual.value_counts()
    number_touched_list.append(df_log.Not_touched_by_reform_counterfactual.value_counts()[True])
    mean_income_list.append(df_log.Counterfactual_rbg_2014.mean())

In [ ]:
print fit.summary()

In [ ]:
plt.plot(mean_income_list)

In [ ]:
plt.plot(number_touched_list)

In [ ]:
df_log["counterfactual_delta_rbg"] = df_log.counterfactual_2014 - df_log.Rbg_2011

In [ ]:
df_temp["Proprietaire"] = df_temp.Proprietaire.astype("int")

In [ ]:
#df_temp[["Touched_by_reform", "Not_touched_by_reform","NBPAC_2011"]]

In [ ]:

#summary = fit.summary(); print(summary)

In [ ]:
fit.params.index[1:]

In [ ]:
 fit.params.values[0]

In [ ]:
df_temp["counterfactual_2014"] = np.multiply(df_temp[fit.params.index[1:]].values, fit.params.values[1:]).sum(axis = 1)+ fit.params.values[0]

In [ ]:
(df_temp.counterfactual_2014 - df_temp.Rbg_2014).hist(bins = 100)

In [ ]:
df_temp[["Rbg_2011","Rbg_2014","counterfactual_2014" ]].head(10)

In [ ]:
df_temp[fit.params.index[1:]].values

In [ ]:
df_log["Revenu_2014"] = df_log[["ZTSAM_10_percent_2014","ZPERM_2014", "ZRTOM_2014",
                                     "ZRNCM_2014","ZRAGM_2014", "ZRICM_2014" ]].sum(axis = 1)

In [ ]:
#df[["Between_thresholds", "child_1", "Between_threshold_1_child"]]

### Change in log disposable income

In [ ]:
import importlib

In [ ]:
year = 2014

In [ ]:
if year == 2014:
    ir = importlib.import_module("IRPP_from_scratch_civil_union")
else :
    ir = importlib.import_module("IRPP_from_scratch_{}".format(year))

In [ ]:
df_log["log_ratio_income"] = np.log(
    df_log.Rbg_2014 - IRPP)

In [ ]:
ir.ir_avec_plafond_qf_enfant()

In [ ]:
m = sm.OLS.from_formula("(Delta_rbg)~   {} + ".format(
        ' + '.join(between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
        + "AGERF_2011 +Anciennete_dans_logement + Rbg_2011_squared + CSDEP_2011  "
                                                                                        , df_log.sample(n = 10000))
fit = m.fit(cov_type = "HC3")
summary = fit.summary(); print(summary)

In [ ]:
df.query("Not_touched_by_reform==1")

In [ ]:
m = sm.OLS.from_formula("(Delta_rbg)~   {} + ".format(
        ' + '.join(between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
        + "AGERF_2011 +Anciennete_dans_logement + Rbg_2011_squared + CSDEP_2011 +Not_touched_by_reform "
                                                                                        , df_log)
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
df_log.Age_Elder_child.hist(bins = 74)
plt.xlim(0,20)

In [ ]:
m = sm.RLM.from_formula("(Delta_rbg)~   {} + ".format(
        ' + '.join(between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
        + "AGERF_2011 +Anciennete_dans_logement + Rbg_2011_squared +Age_Elder_child:Has_child + Age_Youngest_child:Has_child+Has_child+ CSDEP_2011"
                                                                                        , df_log)
fit = m.fit()
summary = fit.summary(); print(summary)

#Reg reaction extensive Margins

##Stat descriptives transitions

### Start to work

In [ ]:
df_log["Wo_start_working"] = (
    ((df_log["YSALI_wo_2011"]+df_log["YBICI_wo_2011"]+df_log["YBNCI_wo_2011"])<2000)
    &((df_log["YSALI_wo_2014"]+df_log["YBICI_wo_2014"]+df_log["YBNCI_wo_2014"])>10000)
    ).astype("int")

df_log["Me_start_working"] = (
    ((df_log["YSALI_me_2011"]+df_log["YBICI_me_2011"]+df_log["YBNCI_me_2011"])<2000)
    &((df_log["YSALI_me_2014"]+df_log["YBICI_me_2014"]+df_log["YBNCI_me_2014"])>10000)
    ).astype("int")

In [ ]:
print df_log["Wo_start_working"].value_counts(normalize = True)
print df_log["Me_start_working"].value_counts(normalize = True)

###Stop to work 

In [ ]:
df_log["Wo_stop_working"] = (
    ((df_log["YSALI_wo_2011"]+df_log["YBICI_wo_2011"]+df_log["YBNCI_wo_2011"])>10000)
    &((df_log["YSALI_wo_2014"]+df_log["YBICI_wo_2014"]+df_log["YBNCI_wo_2014"])<2000)
    ).astype("int")

df_log["Me_stop_working"] = (
    ((df_log["YSALI_me_2011"]+df_log["YBICI_me_2011"]+df_log["YBNCI_me_2011"])>10000)
    &((df_log["YSALI_me_2014"]+df_log["YBICI_me_2014"]+df_log["YBNCI_me_2014"])>2000)
    ).astype("int")

In [ ]:
print df_log["Wo_stop_working"].value_counts(normalize = True)
print df_log["Me_stop_working"].value_counts(normalize = True)

In [ ]:
df_log["Wo_stop_working_for_retraite"] = ((
    ((df_log["YSALI_wo_2011"]+df_log["YBICI_wo_2011"]+df_log["YBNCI_wo_2011"])>10000)
    &((df_log["YSALI_wo_2014"]+df_log["YBICI_wo_2014"]+df_log["YBNCI_wo_2014"])<2000)
    )
      &    ((df_log["YRSTI_wo_2011"]<2000) & (df_log["YRSTI_wo_2014"]>6000))                               
                                           ).astype("int")

df_log["Me_stop_working_for_retraite"] = ((
    ((df_log["YSALI_me_2011"]+df_log["YBICI_me_2011"]+df_log["YBNCI_me_2011"])>10000)
    &((df_log["YSALI_me_2014"]+df_log["YBICI_me_2014"]+df_log["YBNCI_me_2014"])<2000)
    )
      &    ((df_log["YRSTI_me_2011"]<2000) & (df_log["YRSTI_me_2014"]>6000))                               
                                           ).astype("int")



In [ ]:
df["Wo_get_retired"] = ((df["YRSTI_2011_mother"]<2000) & (df["YRSTI_2014_mother"]>6000)).astype("int")





In [ ]:
df["Me_get_retired"] = ((df["YRSTI_2011_father"]<2000) & (df["YRSTI_2014_father"]>6000)).astype("int")





In [ ]:
print(df["Wo_get_retired"].value_counts(normalize = True, dropna = False))
print(df["Me_get_retired"].value_counts(normalize = True, dropna = False))

In [ ]:
print df_log["Wo_stop_working_for_retraite"].value_counts(normalize = True, dropna = False)
print df_log["Me_stop_working_for_retraite"].value_counts(normalize = False, dropna = False)

In [ ]:
interaction_child_over_threshold_extensive_start_work_wo  = ["{}/{}".format(interaction, "Wo_stop_working") for  interaction in interaction_child_over_threshold]

In [ ]:
child_1Over_threshold_1_child[False]:Wo_stop_working

In [ ]:

#Over men women
##start
for child in range(1,7):
    df_log["Itrctin_over_ext_work_wo_start_{}_child".format(child)
                         ] = (df_log["child_{}".format(child)] 
                            * df_log["Over_threshold_{}_child".format(child)]
                            * df_log["Wo_start_working".format(child)])
for child in range(1,7):
    df_log["Itrctin_over_ext_work_me_start_{}_child".format(child)
                         ] = (df_log["child_{}".format(child)] 
                            * df_log["Over_threshold_{}_child".format(child)]
                            * df_log["Me_start_working".format(child)])
## stop
for child in range(1,7):
    df_log["Itrctin_over_ext_work_wo_stop_{}_child".format(child)
                         ] = (df_log["child_{}".format(child)] 
                            * df_log["Over_threshold_{}_child".format(child)]
                            * df_log["Wo_stop_working".format(child)])
for child in range(1,7):
    df_log["Itrctin_over_ext_work_me_stop_{}_child".format(child)
                         ] = (df_log["child_{}".format(child)] 
                            * df_log["Over_threshold_{}_child".format(child)]
                            * df_log["Me_stop_working".format(child)])
#Between men women    
##start
for child in range(1,7):
    df_log["Itrctin_between_ext_work_wo_start_{}_child".format(child)
                         ] = (df_log["child_{}".format(child)] 
                            * df_log["Between_threshold_{}_child".format(child)]
                            * df_log["Wo_start_working".format(child)])
for child in range(1,7):
    df_log["Itrctin_between_ext_work_me_start_{}_child".format(child)
                         ] = (df_log["child_{}".format(child)] 
                            * df_log["Between_threshold_{}_child".format(child)]
                            * df_log["Me_start_working".format(child)])
## stop
for child in range(1,7):
    df_log["Itrctin_between_ext_work_wo_stop_{}_child".format(child)
                         ] = (df_log["child_{}".format(child)] 
                            * df_log["Between_threshold_{}_child".format(child)]
                            * df_log["Wo_stop_working".format(child)])
for child in range(1,7):
    df_log["Itrctin_between_ext_work_me_stop_{}_child".format(child)
                         ] = (df_log["child_{}".format(child)] 
                            * df_log["Between_threshold_{}_child".format(child)]
                            * df_log["Me_stop_working".format(child)])   

In [ ]:
### Start x Over
df_log["Start_x_over".format(child)
                         ] =  df_log["Over_threshold"]* df_log["Wo_start_working"]
### Start x Between
df_log["Start_x_between".format(child)
                         ] =  df_log["Between_thresholds"]* df_log["Wo_start_working"]

### Start x Below
df_log["Start_x_below".format(child)
                         ] =  (np.abs((df_log["Between_thresholds"]+df_log["Over_threshold"])-1)* 
                                df_log["Wo_start_working"])


### Stop x Over
df_log["Stop_x_over".format(child)
                         ] =  df_log["Over_threshold"]* df_log["Wo_stop_working"]
### Stop x Between
df_log["Stop_x_between".format(child)
                         ] =  df_log["Between_thresholds"]* df_log["Wo_stop_working"]

### Stop x Below
df_log["Stop_x_below".format(child)
                         ] =  (np.abs((df_log["Between_thresholds"]+df_log["Over_threshold"])-1)* 
                                df_log["Wo_stop_working"])

In [ ]:
print df_log[["Start_x_over", "Start_x_between", "Start_x_below"]].sum()
print df_log[["Stop_x_over", "Stop_x_between", "Stop_x_below"]].sum()

In [ ]:
interaction_child_over_threshold_extensive_start_work_wo = ["Itrctin_over_ext_work_wo_start_{}_child".format(child) for child in range(1,7)]
interaction_child_over_threshold_extensive_start_work_me = ["Itrctin_over_ext_work_me_start_{}_child".format(child) for child in range(1,7)]
interaction_child_between_threshold_extensive_start_work_wo = ["Itrctin_between_ext_work_wo_start_{}_child".format(child) for child in range(1,7)]
interaction_child_between_threshold_extensive_start_work_me = ["Itrctin_between_ext_work_me_start_{}_child".format(child) for child in range(1,7)]

In [ ]:
interaction_child_over_threshold_extensive_stop_work_wo = ["Itrctin_over_ext_work_wo_stop_{}_child".format(child) for child in range(1,7)]
interaction_child_over_threshold_extensive_stop_work_me = ["Itrctin_over_ext_work_me_stop_{}_child".format(child) for child in range(1,7)]
interaction_child_between_threshold_extensive_stop_work_wo = ["Itrctin_between_ext_work_wo_stop_{}_child".format(child) for child in range(1,7)]
interaction_child_between_threshold_extensive_stop_work_me = ["Itrctin_between_ext_work_me_stop_{}_child".format(child) for child in range(1,7)]

In [ ]:
threshold_byr_rbg

In [ ]:
threshold_byr_rbg.loc[1,"2014"]

In [ ]:
for cbild in range(1,7):
    escape_increase_in_MTR =   df_log.Rbg_2014<threshold_byr_rbg.loc[1,"2014"]
    escape_increase_in_MTR_over = (df_log.Rbg_2014<threshold_byr_rbg.loc[1,"2014"]) * df_log["Over_threshold_{}_child".format(child)]
    escape_increase_in_MTR_below = (df_log.Rbg_2014<threshold_byr_rbg.loc[1,"2014"]) * df_log["Between_threshold_{}_child".format(child)]

In [ ]:

m = sm.RLM.from_formula("(Delta_rbg)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number[:]+ over_threshold_for_child_rank+
                  interaction_child_between_threshold[:]+interaction_child_over_threshold[:]+
                  interaction_child_over_threshold_extensive_start_work_wo))
                                                                                        , df_log)
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:

m = sm.OLS.from_formula("(Delta_rbg)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number[:]+ over_threshold_for_child_rank+
                  interaction_child_between_threshold[:]+interaction_child_over_threshold[:]+
                  interaction_child_over_threshold_extensive_start_work_wo+ interaction_child_over_threshold_extensive_start_work_me+
                  interaction_child_between_threshold_extensive_start_work_wo+interaction_child_between_threshold_extensive_start_work_me+
                  interaction_child_over_threshold_extensive_stop_work_wo+interaction_child_over_threshold_extensive_stop_work_me+
                  interaction_child_between_threshold_extensive_stop_work_wo+interaction_child_between_threshold_extensive_stop_work_me))
                 + "+ Proprietaire +CSDEP + Age_wo + Age_me+ Age_wo_squared + Age_me_squared+Age_Elder_child"
                 + "+escape_increase_in_MTR_over+escape_increase_in_MTR_below" 
                    +"+Start_x_over+ Start_x_between + Start_x_below"
                    +"+Stop_x_over+ Stop_x_between + Stop_x_below"
                                                                                        , df_log)
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
interaction_child_over_threshold

In [ ]:
m = sm.Probit.from_formula("(Wo_stop_working)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number[:]+ over_threshold_for_child_rank+
                  interaction_child_between_threshold[:]+interaction_child_over_threshold[:]))
        + "+ Proprietaire +CSDEP + Age_wo + Age_me+ Age_wo_squared + Age_me_squared+Age_Elder_child"
                                                                                        , df_log.sample(frac = 1))
fit = m.fit(maxiter = 500, method = "basinhopping")
summary = fit.summary(); print(summary)

In [ ]:
m.fit()

In [ ]:
df.YSALI_2011_father

1. Women not working anymore
2. Men not working anymore
3. Men working more when women not working anymore

In [ ]:
for var in ["YSALI", "YCHOI", "YBNCI", "YBICI", "YCHOI", "YRSTI"]:
    df["{}_2011".format(var)] = df[["{}_2011_mother".format(var), "{}_2011_father".format(var)]].sum(axis = 1)
    df["{}_2014".format(var)] = df[["{}_2014_mother".format(var), "{}_2014_father".format(var)]].sum(axis = 1)
    df["Delta_{}".format(var)] = df["{}_2014".format(var)] - df["{}_2011".format(var)]
    df["Delta_{}_mother".format(var)] = df["{}_2014_mother".format(var)]-df["{}_2011_mother".format(var)]
    df["Delta_{}_father".format(var)] = df["{}_2014_father".format(var)]-df["{}_2011_father".format(var)]

In [ ]:
df_log["YSALI_2011"] = df_log[["YSALI_wo_2011", "YSALI_me_2011"]].sum(axis = 1)
df_log["YSALI_2014"] = df_log[["YSALI_wo_2014", "YSALI_me_2014"]].sum(axis = 1)
df_log["Delta_YSALI"] = df_log.YSALI_2014 - df_log.YSALI_2011
df_log["Delta_YSALI_wo"] = df_log.YSALI_wo_2014-df_log.YSALI_wo_2011
df_log["Delta_YSALI_me"] = df_log.YSALI_me_2014-df_log.YSALI_me_2011

In [ ]:
df_log["YBNCI_2011"] = df_log[["YBNCI_wo_2011", "YBNCI_me_2011"]].sum(axis = 1)
df_log["YBNCI_2014"] = df_log[["YBNCI_wo_2014", "YBNCI_me_2014"]].sum(axis = 1)
df_log["Delta_YBNCI"] = df_log.YBNCI_2014 - df_log.YBNCI_2011
df_log["Delta_YBNCI_wo"] = df_log.YBNCI_wo_2014-df_log.YBNCI_wo_2011
df_log["Delta_YBNCI_me"] = df_log.YBNCI_me_2014-df_log.YBNCI_me_2011

In [ ]:
df_log["YBICI_2011"] = df_log[["YBICI_wo_2011", "YBICI_me_2011"]].sum(axis = 1)
df_log["YBICI_2014"] = df_log[["YBICI_wo_2014", "YBICI_me_2014"]].sum(axis = 1)
df_log["Delta_YBICI"] = df_log.YBICI_2014 - df_log.YBICI_2011
df_log["Delta_YBICI_wo"] = df_log.YBICI_wo_2014-df_log.YBICI_wo_2011
df_log["Delta_YBICI_me"] = df_log.YBICI_me_2014-df_log.YBICI_me_2011

In [ ]:
df_log["YRSTI_2011"] = df_log[["YRSTI_wo_2011", "YRSTI_me_2011"]].sum(axis = 1)
df_log["YRSTI_2014"] = df_log[["YRSTI_wo_2014", "YRSTI_me_2014"]].sum(axis = 1)
df_log["Delta_YRSTI"] = df_log.YRSTI_2014 - df_log.YRSTI_2011
df_log["Delta_YRSTI_wo"] = df_log.YRSTI_wo_2014-df_log.YRSTI_wo_2011
df_log["Delta_YRSTI_me"] = df_log.YRSTI_me_2014-df_log.YRSTI_me_2011

In [ ]:
df_log["Delta_YSALI_wo"] = df_log.YSALI_wo_2014-df_log.YSALI_wo_2011
df_log["Delta_YSALI_me"] = df_log.YSALI_me_2014-df_log.YSALI_me_2011

In [ ]:
df.Delta_YSALI_father

In [ ]:
type_of_income = "YRSTI" ; sexe = "father"
m = sm.RLM.from_formula("(Delta_{}_{})~   {}  ".format( type_of_income,sexe,
        ' + '.join(between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
        
                                    ,df[~df["YSALI_2011_father"].isnull()].query(
                                                    "({}_2014_{}>3000)|({}_2011_{}>3000)".format( type_of_income,sexe,
                                                                                                  type_of_income,sexe,
                                                                                          ))
                       )
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
type_of_income = "YRSTI" ; sexe = "mother"
m = sm.RLM.from_formula("(Delta_{}_{})~   {}  ".format( type_of_income,sexe,
        ' + '.join(between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
        
                                    , df.query(
                                                    "({}_2014_{}>3000)|({}_2011_{}>3000)".format( type_of_income,sexe,
                                                                                                  type_of_income,sexe,
                                                                                          ))
                       )
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
desc_stat_taux_emploi_wo_me = pd.DataFrame()
for var in ["YSALI", "YCHOI", "YBNCI", "YBICI", "YCHOI", "YRSTI"]:
    desc_stat_taux_emploi_wo_me.loc[var+"_Mean",'Women']= df_log[var+"_wo_2014"].mean()
    desc_stat_taux_emploi_wo_me.loc[var+"_Share",'Women']=(df_log[var+"_wo_2014"]>1000).value_counts(normalize = True)[True]
    desc_stat_taux_emploi_wo_me.loc[var+"_Mean",'Men']= df_log[var+"_me_2014"].mean()
    desc_stat_taux_emploi_wo_me.loc[var+"_Share",'Men']=(df_log[var+"_me_2014"]>1000).value_counts(normalize = True)[True] 

In [ ]:
desc_stat_taux_emploi_wo_me

In [ ]:
fiscr.TYPMEN9.value_counts()

In [ ]:
m = sm.OLS.from_formula("(Delta_YRSTI_wo)~   {}   ".format(
        ' + '.join(between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
        
                                                                                        , df_log)
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
# To keep article ! retraite
m = sm.OLS.from_formula("(Delta_YRSTI_me)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
        
                                                                                        , df_log)
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
1.41e-19

In [ ]:
df_log["Delta_YSALI_prct"] = (df_log.YSALI_2014 - df_log.YSALI_2011)/df_log.YSALI_2011
df_log["Delta_YSALI_wo_prct"] = (df_log.YSALI_wo_2014 - df_log.YSALI_wo_2011)/df_log.YSALI_wo_2011
df_log["Delta_YSALI_me_prct"] = (df_log.YSALI_me_2014 - df_log.YSALI_me_2011)/df_log.YSALI_me_2011

In [ ]:
df_log.De

In [ ]:
m = sm.RLM.from_formula("(Delta_YSALI_me)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
        
        , df_log.query("(NBPAC_2011<6)& (YSALI_me_2011>1000)& (YSALI_me_2014>1000)&((NBPAC_2011==4)|(NBPAC_2011==0))").sample(frac = 1)
                       )
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
m = sm.OLS.from_formula("(Delta_YSALI_me)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
        
                                                                                        , df_log.query("YSALI_2011>0"))
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
df_log[[]].head()

In [ ]:
sns.distplot(df_log.YSALI_me_2011)
sns.distplot(df_log.YSALI_wo_2011)

In [ ]:
sns.distplot(df_log.YSALI_me_2014.fillna(0))
sns.distplot(df_log.YSALI_wo_2014.fillna(0))

In [ ]:
plt.hist(df_log.YSALI_wo_2011/(df_log.YSALI_2011.fillna(0)))

In [ ]:
sns.distplot((df_log.YSALI_wo_2011/df_log.YSALI_2011).fillna(0))
plt.vlines(0.5,0,7, color = 'r', alpha = 0.2)

In [ ]:
m = sm.OLS.from_formula("(Delta_YBNCI_me)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
        
                                                                                        , df_log.query("YBNCI_me_2011>0"))
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
m = sm.OLS.from_formula("(Delta_YSALI_me)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
        
                                                                                        , df_log)
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
df_log.YSALI_me_2011.describe()

In [ ]:
df_log.YSALI_wo_2011.describe()

In [ ]:
df_log.Rb

In [ ]:
df_log["Wo_start_working"] = (
    ((df_log["YSALI_wo_2011"]+df_log["YBICI_wo_2011"]+df_log["YBNCI_wo_2011"])<5000)
    &((df_log["YSALI_wo_2014"]+df_log["YBICI_wo_2014"]+df_log["YBNCI_wo_2014"])>15000)
    ).astype("int")

df_log["Me_start_merking"] = (
    ((df_log["YSALI_me_2011"]+df_log["YBICI_me_2011"]+df_log["YBNCI_me_2011"])<5000)
    &((df_log["YSALI_me_2014"]+df_log["YBICI_me_2014"]+df_log["YBNCI_me_2014"])>15000)
    ).astype("int")

In [ ]:
df_log["Wo_increase_work_by_30"] = (
    (((df_log["YSALI_wo_2014"])-(df_log["YSALI_wo_2011"]))/(df_log["YSALI_wo_2011"]))>0.3
    ).astype("int")


df_log["Me_increase_work_by_30"] = (
    (((df_log["YSALI_me_2014"])-(df_log["YSALI_me_2011"]))/(df_log["YSALI_me_2011"]))>0.3
    ).astype("int")

In [ ]:
#df_log[["Wo_increase_work_by_30","YSALI_wo_2014", "YSALI_wo_2011" ]]

In [ ]:
df_log["Wo_start_working"].value_counts(dropna = False)

In [ ]:
m = sm.Probit.from_formula("(Me_increase_work_by_30)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number[:]+ over_threshold_for_child_rank+
                  interaction_child_between_threshold[:]+interaction_child_over_threshold[:]))
        + "+ Proprietaire +CSDEP + Age_wo + Age_me+ Age_wo_squared + Age_me_squared+Age_Elder_child"
                                                                                        , df_log)
fit = m.fit(maxiter = 500, method = "newton")
summary = fit.summary(); print(summary)

In [ ]:
m = sm.Probit.from_formula("(Me_get_retired)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number[:]+ over_threshold_for_child_rank+
                  interaction_child_between_threshold[:]+interaction_child_over_threshold[:]))
                                                                                        , df)
fit = m.fit(maxiter = 500, method = "newton")
summary = fit.summary(); print(summary)

In [ ]:
m = sm.Probit.from_formula("(Me_get_retired)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number[:]+ over_threshold_for_child_rank+
                  interaction_child_between_threshold[:]+interaction_child_over_threshold[:]))
        + "+ Proprietaire +CSDEP + Age_wo + Age_me+ Age_wo_squared + Age_me_squared+Age_Elder_child"
                                                                                        , df_log)
fit = m.fit(maxiter = 500, method = "newton")
summary = fit.summary(); print(summary)

In [ ]:
m = sm.Probit.from_formula("(Wo_get_retired)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number[:]+ over_threshold_for_child_rank+
                  interaction_child_between_threshold[:]+interaction_child_over_threshold[:]))
        + "+ Proprietaire +CSDEP + Age_wo + Age_me+ Age_wo_squared + Age_me_squared+Age_Elder_child"
                                                                                        , df_log)
fit = m.fit(maxiter = 500, method = "newton")
summary = fit.summary(); print(summary)

In [ ]:
df_log.YSALI_me_2011.hist(bins = 100)
df_log.YSALI_wo_2011.hist(bins = 100)

In [ ]:
df_log.query("YBICI_me_2011>0").YBICI_me_2011.hist(bins = 100)
df_log.query("YBICI_wo_2011>0").YBICI_wo_2011.hist(bins = 100)

In [ ]:
df_log.query("YBNCI_me_2011>1000").YBNCI_me_2011.hist(bins = 100, alpha = .5, color = 'g')
df_log.query("YBNCI_wo_2011>1000").YBNCI_wo_2011.hist(bins = 100, alpha = .5, color = 'b')

In [ ]:
df_log.query("YBNCI_me_2011>40000").shape

In [ ]:
df_log.query("YBNCI_wo_2011>40000").shape

In [ ]:
"Delta_bnci_me"

### Merge with panact

In [ ]:
df_id_diff_wo = df_log[df_log.Id_diff_wo>1]; df_id_diff_wo.shape

In [ ]:
df_id_diff_me = df_log[df_log.Id_diff_me>1]; df_id_diff_me.shape

In [ ]:
m = sm.OLS.from_formula("(Delta_rbg)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number[:]+ over_threshold_for_child_rank+
                  interaction_child_between_threshold[:]+interaction_child_over_threshold[:]))
        + "+ Proprietaire +CSDEP + Age_wo + Age_me+ Age_wo_squared + Age_me_squared"
                                                                                        , df_id_diff_wo)
fit = m.fit()
summary = fit.summary(); print(summary)

On va chercher Panact dans les DADS de l'EDP 2016 '

In [ ]:
try : 
    df_panact = pd.read_hdf('C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2015/Data/hdf/dads_edp_2016.h5','dads_start_2009')
except : 
    df_panact = pd.read_csv("C:\Users\IMPTEMP_A_PACIFIC\Desktop\EDP_2015\Data\csv\PANACT_2016.csv")
    df_panact_start_2009 = df_panact[df_panact.AN>=2009]
    df_panact_start_2009.to_hdf('C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2015/Data/hdf/dads_edp_2016.h5','dads_start_2009')

In [ ]:
df_panact_2011 = df_panact[df_panact.AN == 2011]

In [ ]:
df_panact_2014 = df_panact[df_panact.AN == 2014]

In [ ]:
panac_2014_2011 = pd.merge(df_panact_2011, df_panact_2014, on = 'ID_DIFF', suffixes=("_2011","_2014" ), how = 'outer')

In [ ]:
panac_2014_2011.NBHEUR_2011.fillna(0,inplace=True)
panac_2014_2011.NBHEUR_2014.fillna(0,inplace=True)

In [ ]:
id_diff_control = merge_panac_reg_wo.ID_DIFF






In [ ]:
TODO : doute sur le fait que je choppe le bon sexe, trop peu de difference entre homme et femme

In [ ]:
fisci = load_fisc_i_by_year(2011)

In [ ]:
#fisci = fisci.ID_DIFF.isin(id_diff_control)

In [ ]:
#fisci.SEXE.value_counts() #TODO : tout doit être égal à sexe femme

In [ ]:
merge_panac_reg_wo= pd.merge(panac_2014_2011, df_id_diff_wo, left_on= "ID_DIFF", right_on= "Id_diff_wo", how = 'inner')
merge_panac_reg_wo['Delta_heur'] = merge_panac_reg_wo.NBHEUR_2014 - merge_panac_reg_wo.NBHEUR_2011
merge_panac_reg_wo['Work_more_in_hours'] = (merge_panac_reg_wo.Delta_heur>0).astype('int')
merge_panac_reg_wo['Work_less_in_hours'] = (merge_panac_reg_wo.Delta_heur<0).astype('int')
merge_panac_reg_wo['Start_to_work_hours'] = ((merge_panac_reg_wo['NBHEUR_2014']>100)&(merge_panac_reg_wo['NBHEUR_2011']<100)).astype('int')
merge_panac_reg_wo['Switch_to_full_time'] = ((merge_panac_reg_wo['NBHEUR_2014']>=1700)&(merge_panac_reg_wo['NBHEUR_2011']<1700)).astype('int')
merge_panac_reg_wo['Switch_to_part_time'] = ((merge_panac_reg_wo['NBHEUR_2014']<=1700)&(merge_panac_reg_wo['NBHEUR_2011']>1700)).astype('int')

In [ ]:
merge_panac_reg_me= pd.merge(panac_2014_2011, df_id_diff_me, left_on= "ID_DIFF", right_on= "Id_diff_me", how = 'inner')
merge_panac_reg_me['Delta_heur'] = merge_panac_reg_me.NBHEUR_2014 - merge_panac_reg_me.NBHEUR_2011
merge_panac_reg_me['Work_more_in_hours'] = (merge_panac_reg_me['Delta_heur']>0).astype('int')
merge_panac_reg_me['Work_less_in_hours'] = (merge_panac_reg_me['Delta_heur']<0).astype('int')
merge_panac_reg_me['Start_to_work_hours'] = ((merge_panac_reg_me['NBHEUR_2014']>100)&(merge_panac_reg_me['NBHEUR_2011']<100)).astype('int')
merge_panac_reg_me['Switch_to_full_time'] = ((merge_panac_reg_me['NBHEUR_2014']>=1700)&(merge_panac_reg_me['NBHEUR_2011']<1700)).astype('int')
merge_panac_reg_me['Switch_to_part_time'] = ((merge_panac_reg_me['NBHEUR_2014']<=1700)&(merge_panac_reg_me['NBHEUR_2011']>1700)).astype('int')

In [ ]:
#Pas les memes ID_DIFF entre EDP 2015 et EDP2016...
#merge_panac_reg_me[["ID_DIFF", "Id_diff_me", "YSALI_me_2011", "NETNET_2011"]]

In [ ]:
merge_panac_reg_me.NETNET_2011.describe()

In [ ]:
filter_net_net
(merge_panac_reg_wo.NETNET_2011 - merge_panac_reg_wo.YSALI_wo_2011).hist(bins = 5000)
plt.xlim(-4000,4000)

In [ ]:
merge_panac_reg_wo["Correct_sali_between_dads_impo"] = np.abs(
    (merge_panac_reg_wo.YSALI_2011 - merge_panac_reg_wo.NETNETR_2011)/merge_panac_reg_wo.YSALI_2011)*(merge_panac_reg_wo.YSALI_2011>10000)

In [ ]:
merge_panac_reg_wo.query("Correct_sali_between_dads_impo<1")["Correct_sali_between_dads_impo"].hist(bins= 100)

In [ ]:
merge_panac_reg_wo.query("(Correct_sali_between_dads_impo<0.01)&(YSALI_2011>10000)"
                        )[["Correct_sali_between_dads_impo", "NETNETR_2011", "YSALI_2011"]].head()

In [ ]:
#merge_panac_reg_wo[["NETNET_2011", "NETNETR_2011", "YSALI_wo_2011","YSALI_me_2011", "Age_wo", "Age_me"]]

In [ ]:
merge_panac_reg_wo.NBHEUR_2011.hist(bins = 100)
merge_panac_reg_me.NBHEUR_2011.hist(bins = 100)

In [ ]:
merge_panac_reg_me.NBHEUR_2011.describe()

In [ ]:
print merge_panac_reg_me['Work_less_in_hours'].value_counts()
print merge_panac_reg_wo['Work_less_in_hours'].value_counts()
print merge_panac_reg_me['Work_more_in_hours'].value_counts()
print merge_panac_reg_wo['Work_more_in_hours'].value_counts()
print merge_panac_reg_me['Start_to_work_hours'].value_counts()
print merge_panac_reg_wo['Start_to_work_hours'].value_counts()

In [ ]:
merge_panac_reg_wo.NBHEUR_2011.mean()

In [ ]:
merge_panac_reg_wo.NBHEUR_2011.hist(bins = 100)

In [ ]:
(merge_panac_reg_wo.NBHEUR_2011>1400).value_counts(normalize = True)

In [ ]:
(merge_panac_reg_me.NBHEUR_2011>1400).value_counts(normalize = True)

In [ ]:
merge_panac_reg_wo.Delta_heur.hist(bins = 100)

In [ ]:
merge_panac_reg_me.Delta_heur.hist(bins = 100)

In [ ]:
m = sm.Probit.from_formula("(Work_less_in_hours)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number[:]+ over_threshold_for_child_rank+
                  interaction_child_between_threshold[:]+interaction_child_over_threshold[:]))
        + "+ Proprietaire +CSDEP + Age_wo + Age_me+ Age_wo_squared + Age_me_squared+Age_Elder_child"
                                                                                        , merge_panac_reg_wo)
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
m = sm.Probit.from_formula("(Switch_to_part_time)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number[:]+ over_threshold_for_child_rank+
                  interaction_child_between_threshold[:]+interaction_child_over_threshold[:]))
        + "+ Proprietaire +CSDEP + Age_wo + Age_me+ Age_wo_squared + Age_me_squared+Age_Elder_child"
                                                                                        , merge_panac_reg_me)
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
merge_panac_reg_me.Start_to_work_hours.value_counts(dropna = False)

In [ ]:
m = sm.Probit.from_formula("(Start_to_work_hours)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number[:]+ over_threshold_for_child_rank+
                  interaction_child_between_threshold[:]+interaction_child_over_threshold[:]))
        + "+ Proprietaire +CSDEP + Age_wo + Age_me+ Age_wo_squared + Age_me_squared"
                                                                                        , merge_panac_reg_wo)
fit = m.fit(maxiter = 500, method = "bfgs")
summary = fit.summary(); print(summary)

In [ ]:
merge_panac_reg_me.

In [ ]:
m = sm.Probit.from_formula("(Stop_to_work_hours)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number[:]+ over_threshold_for_child_rank+
                  interaction_child_between_threshold[:]+interaction_child_over_threshold[:]))
        + "+ Proprietaire +CSDEP + Age_wo + Age_me+ Age_wo_squared + Age_me_squared"
                                                                                        , merge_panac_reg_me)
fit = m.fit(maxiter = 500,  method = "bfgs")
summary = fit.summary(); print(summary)

In [ ]:

m = sm.RLM.from_formula("(Delta_heur)~   {}+AGERF_2011  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number[:]+ over_threshold_for_child_rank+
                  interaction_child_between_threshold[:]+interaction_child_over_threshold[:]))
        + "+ Proprietaire +CSDEP + Age_wo + Age_me+ Age_wo_squared + Age_me_squared"
                                                                                        , merge_panac_reg_me)
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
merge_panac_reg.NBHEUR

In [ ]:
df_log.ZIMPOM14_2014.describe(percentiles=[0.2,0.95,.99,.999])

In [ ]:
df_log["Rbg_wo"]=(
    ( df_log.ZTSAM_wo*0.9) + df_log.ZRAGM_wo 
    + df_log.ZRACM_wo + df_log.ZRNCM_wo 
    + df_log.ZFONM_wo + df_log.ZPERM_wo*0.9
    + df_log.ZRICM_wo)

In [ ]:
income_vars = ["YSALI", "YALRI", "YCHOI", "YBICI", "YBNCI", "YRAGI", "YRSTI"]
income_vars_me = [string + "_me_2011" for string in income_vars]
income_vars_wo = [string + "_wo_2011" for string in income_vars]

In [ ]:
df_log["Income_me"] = df_log[income_vars_me].sum(axis = 1)
df_log["Income_wo"] = df_log[income_vars_wo].sum(axis = 1)

In [ ]:
df_log[income_vars_me].div(df_log["Income_me"], axis = 0).YSALI_me_2011.hist(bins = 10000)
plt.xlim(0,1)

In [ ]:
df_log[income_vars_me].div(df_log["Income_me"], axis = 0).fillna(0).mean()

In [ ]:
df_log[income_vars_wo].div(df_log["Income_wo"], axis = 0).fillna(0).mean()

In [ ]:
df.YSALI_wo_2011.describe()

In [ ]:
labor_income_vars = ["YSALI", "YBICI", "YBNCI", "YRAGI"]
labor_income_vars_me_2011 = [string + "_me_2011" for string in income_vars]
labor_income_vars_wo_2011 = [string + "_wo_2011" for string in income_vars]
labor_income_vars_me_2014 = [string + "_me_2014" for string in income_vars]
labor_income_vars_wo_2014 = [string + "_wo_2014" for string in income_vars]

In [ ]:
df_log["Labor_income_me_2011"] = df_log[labor_income_vars_me_2011].sum(axis= 1)
df_log["Labor_income_me_2014"] = df_log[labor_income_vars_me_2014].sum(axis= 1)
df_log["Labor_income_wo_2011"] = df_log[labor_income_vars_wo_2011].sum(axis= 1)
df_log["Labor_income_wo_2014"] = df_log[labor_income_vars_wo_2014].sum(axis= 1)


In [ ]:
df_log["Delta_labor_income_me"] = df_log.Labor_income_me_2014 - df_log.Labor_income_me_2011
df_log["Delta_labor_income_wo"] = df_log.Labor_income_wo_2014 - df_log.Labor_income_wo_2011

In [ ]:
df_log[["Age_wo", "Age_me", "YSALI_me_2014", "YCHOI_me_2014", "YSALI_wo_2014", "YCHOI_wo_2014", "ID_FISC_LOG_DIFF"]].head()

In [ ]:
df_log.NBPERSM_2011 -df_log.NBPAC_2011.value_counts()

In [ ]:
fisci = load_fisc_i_by_year(2014)

In [ ]:
fisci.query("ID_FISC_LOG_DIFF ==121")

In [ ]:
fiscr = load_fisc_r_by_year(2014)

In [ ]:
fiscr.query("ID_FISC_LOG_DIFF ==121")

In [ ]:
m = sm.OLS.from_formula("(Delta_labor_income_me)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number[:]+ over_threshold_for_child_rank+
                  interaction_child_between_threshold[:]+interaction_child_over_threshold[:]))
        + "+ Proprietaire +CSDEP + Age_wo + Age_me+ Age_wo_squared + Age_me_squared+ Age_me:Between_threshold_5_child:child_5"
                                                                                        , df_log)
fit_me = m.fit(maxiter =50)
summary = fit_me.summary(); print(summary)
m = sm.OLS.from_formula("(Delta_labor_income_wo)~   {}  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number[:]+ over_threshold_for_child_rank+
                  interaction_child_between_threshold[:]+interaction_child_over_threshold[:]))
        + "+ Proprietaire +CSDEP + Age_wo + Age_me+ Age_wo_squared + Age_me_squared"
                                                                                        , df_log)
fit_wo = m.fit(maxiter =50)
summary = fit_wo.summary(); print(summary)

In [ ]:
fit_me.params - fit_wo.params

In [ ]:
df_log.working_me_2014.value_counts(normalize=True)

In [ ]:
df_log.working_wo_2011.value_counts(normalize=True)

In [ ]:
df_log.working_wo_2011.value_counts(normalize=True)

In [ ]:
print df_2011_keep[(df_2011_keep.SEXE==2)&(df_2011_keep.YSALI == 0)&(df_2011_keep.YRSTI==0)].shape ;print "femmes"

In [ ]:
df.working_me_2011.value_counts() #Probleme double comptage...

In [ ]:
print "men started working: ", df.Men_started_working.value_counts()[1]
print "men stopped working: ", df.Men_stopped_working.value_counts()[1]
print "Women started working: ", df.Wo_started_working.value_counts()[1]
print "Women stopped working: ", df.Wo_stopped_working.value_counts()[1]

In [ ]:
print "men started working: ", df.Men_started_working.value_counts()[1]
print "men stopped working: ", df.Men_stopped_working.value_counts()[1]
print "Women started working: ", df.Wo_started_working.value_counts()[1]
print "Women stopped working: ", df.Wo_stopped_working.value_counts()[1]

In [ ]:
# m = sm.Logit.from_formula("(Wo_started_working)~   {} ".format(
#         ' + '.join(children_number+between_threshold_for_child_rank+ over_threshold_for_child_rank+interaction_child_over_threshold
#                  + interaction_child_between_threshold))
#                                         , df)
# fit = m.fit(maxiter = 500)
# summary = fit.summary(); print(summary)

In [ ]:
m = sm.Probit.from_formula("(Men_stopped_working)~   {} + AGERF_2011 + Over_threshold + Between_thresholds".format(
        ' + '.join(children_number+between_threshold_for_child_rank
                  ))
                                        , df)
fit = m.fit(maxiter=500)
summary = fit.summary(); print(summary)

In [ ]:
m = sm.Probit.from_formula("(Wo_started_working)~   {} + AGERF_2011 + Over_threshold + Between_thresholds".format(
        ' + '.join(children_number+between_threshold_for_child_rank
                  ))
                                        , df)
fit = m.fit(maxiter=500)
summary = fit.summary(); print(summary)

In [ ]:
m = sm.Probit.from_formula("(Wo_stopped_working)~   {} + AGERF_2011  ".format(
        ' + '.join(children_number+ over_threshold_for_child_rank+between_threshold_for_child_rank+
                  interaction_child_between_threshold))
                                        , working_by_sex_reg)
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
df[df.Wo_started_working==1][["Wo_started_working", "YSALI_wo_2011", "YSALI_wo_2014"]]

In [ ]:
m = sm.Probit.from_formula("(Wo_started_working)~   {} + AGERF_2011  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                                        , df_log)
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
m = sm.Probit.from_formula("(Wo_started_working)~   {} + AGERF_2011  ".format(
        ' + '.join(between_threshold_for_child_rank+children_number_temp+ over_threshold_for_child_rank_temp+
                  interaction_child_between_threshold_temp+interaction_child_over_threshold_temp))
                                        , working_by_sex_reg)
fit = m.fit()
summary = fit.summary(); print(summary)

In [ ]:
df.head()

#fIN mais ce qui avait été fait à l'époque peut être potentiellement intéressant...

# Reg sur tout le monde et pas sur ceux gardant uniquement le même nombre d'enfants.

In [ ]:
suffixes = fisc_r_2012.columns

In [ ]:
len(fisc_r_2014.columns)

In [ ]:
fisc_r_2012.columns = [fisc_r_2012.columns[0]]+[label+"_2012" for label in fisc_r_2012.columns[1:] ]
fisc_r_2013.columns = [fisc_r_2012.columns[0]]+[label+"_2013" for label in fisc_r_2012.columns[1:] ]
fisc_r_2010.columns = [fisc_r_2010.columns[0]]+[label+"_2010" for label in fisc_r_2010.columns[1:] ]

In [ ]:
%%time
df_fiscr_2010_to_2014 = pd.merge(df_fiscr_2011_2014,fisc_r_2010, on='ID_FISC_LOG_DIFF', suffixes=('ERROR','ERROR') )

In [ ]:
%%time
df_fiscr_2010_to_2014 = pd.merge(df_fiscr_2010_to_2014,fisc_r_2012, on='ID_FISC_LOG_DIFF', suffixes=('ERROR','ERROR') )

In [ ]:
%%time
df_fiscr_2010_to_2014 = pd.merge(df_fiscr_2010_to_2014,fisc_r_2013, on='ID_FISC_LOG_DIFF', suffixes=('ERROR','ERROR') )

In [ ]:
(df_fiscr_2010_to_2014.ZIMPOM_2014 -(df_fiscr_2010_to_2014.ZIMPOM_2011)).describe()

In [ ]:
(df_fiscr_2010_to_2014.ZIMPOM_2014 /(df_fiscr_2010_to_2014.ZIMPOM_2011)).describe()

## Nombre d'individus potentiellement concernés.

## Avec 1 enfant

In [ ]:
((df_fiscr_2010_to_2014.ZSALM_2011>36000) & (df_fiscr_2010_to_2014.ZSALM_2011<41000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([41]))).value_counts()

In [ ]:
((df_fiscr_2010_to_2014.ZSALM_2011>41000)
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([41]))).value_counts()

## Avec 2 enfants

In [ ]:
((df_fiscr_2010_to_2014.ZSALM_2011>41000) & (df_fiscr_2010_to_2014.ZSALM_2011<52000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([42]))).value_counts()

In [ ]:
((df_fiscr_2010_to_2014.ZSALM_2011>52000)
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([42]))).value_counts()

## Avec 3 enfants

In [ ]:
((df_fiscr_2010_to_2014.ZSALM_2011>53000) & (df_fiscr_2010_to_2014.ZSALM_2011<76000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([43]))).value_counts()

In [ ]:
((df_fiscr_2010_to_2014.ZSALM_2011>76000)
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([43]))).value_counts()


### Créer rapidement des variables binaires pour chaque groupe pour voir si ça a un impact sur zsalm d'apartenir au groupe...

In [ ]:
df_fiscr_2010_to_2014["Groupe_1"]=((df_fiscr_2010_to_2014.ZSALM_2011>36000) & (df_fiscr_2010_to_2014.ZSALM_2011<41000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([41])))

In [ ]:
df_fiscr_2010_to_2014["Groupe_1"] += ((df_fiscr_2010_to_2014.ZSALM_2011>41000) & (df_fiscr_2010_to_2014.ZSALM_2011<52000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([42])))

In [ ]:
df_fiscr_2010_to_2014["Groupe_1"] +=((df_fiscr_2010_to_2014.ZSALM_2011>53000) & (df_fiscr_2010_to_2014.ZSALM_2011<76000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([43])))

In [ ]:
df_fiscr_2010_to_2014["Groupe_1"] = df_fiscr_2010_to_2014.Groupe_1.astype('int', inplace = True)

In [ ]:
df_fiscr_2010_to_2014["Groupe_2"]=((df_fiscr_2010_to_2014.ZSALM_2011>41000)  &
                                   (df_fiscr_2010_to_2014.TYPMEN9_2011.isin([41])))

In [ ]:
df_fiscr_2010_to_2014["Groupe_2"] += ((df_fiscr_2010_to_2014.ZSALM_2011>52000) 
                                &(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([42])))

In [ ]:
df_fiscr_2010_to_2014["Groupe_2"] +=((df_fiscr_2010_to_2014.ZSALM_2011>76000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([43])))

In [ ]:
df_fiscr_2010_to_2014["Groupe_2"] = df_fiscr_2010_to_2014.Groupe_2.astype('int', inplace = True)

In [ ]:
df_fiscr_2010_to_2014["Groupe_1_enfant"]=((df_fiscr_2010_to_2014.ZSALM_2011>36000) & (df_fiscr_2010_to_2014.ZSALM_2011<41000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([41]))).astype('int', inplace = True)
df_fiscr_2010_to_2014["Groupe_1_2_enfants"] = ((df_fiscr_2010_to_2014.ZSALM_2011>41000) & (df_fiscr_2010_to_2014.ZSALM_2011<52000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([42]))).astype('int', inplace = True)
df_fiscr_2010_to_2014["Groupe_1_3enfants"] =((df_fiscr_2010_to_2014.ZSALM_2011>53000) & (df_fiscr_2010_to_2014.ZSALM_2011<76000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([43]))).astype('int', inplace = True)

In [ ]:
df_fiscr_2010_to_2014["Groupe_2_1_enfant"]=((df_fiscr_2010_to_2014.ZSALM_2011>41000)  &
                                   (df_fiscr_2010_to_2014.TYPMEN9_2011.isin([41]))).astype('int', inplace = True)
df_fiscr_2010_to_2014["Groupe_2_2enfants"] =((df_fiscr_2010_to_2014.ZSALM_2011>52000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([42]))).astype('int', inplace = True)
df_fiscr_2010_to_2014["Groupe_2_3enfants"] =((df_fiscr_2010_to_2014.ZSALM_2011>76000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([43]))).astype('int', inplace = True)


In [ ]:
diff_income  =df_fiscr_2010_to_2014.ZSALM_2014- df_fiscr_2010_to_2014.ZSALM_2010

### ON controle très rapidement pour les mêmes income brackets mais sans prendre en compte la structure du ménage

In [ ]:
df_fiscr_2010_to_2014["Control_group_1_child_bracket"]=((df_fiscr_2010_to_2014.ZSALM_2011>36000) & (df_fiscr_2010_to_2014.ZSALM_2011<41000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([41,42,43])))

In [ ]:
((df_fiscr_2010_to_2014.ZSALM_2011>36000) & (df_fiscr_2010_to_2014.ZSALM_2011<41000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([41]))).value_counts()

In [ ]:
df_fiscr_2010_to_2014["Control_group_2_child_bracket"]= ((df_fiscr_2010_to_2014.ZSALM_2011>41000) & (df_fiscr_2010_to_2014.ZSALM_2011<52000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([41,42,43])))

In [ ]:
df_fiscr_2010_to_2014["Control_group_3_child_bracket"]=((df_fiscr_2010_to_2014.ZSALM_2011>53000) & (df_fiscr_2010_to_2014.ZSALM_2011<76000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([43])))

In [ ]:
df_fiscr_2010_to_2014["Control_group_over_1_child_bracket"]=((df_fiscr_2010_to_2014.ZSALM_2011>41000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([41,42,43])))
df_fiscr_2010_to_2014["Control_group_over_2_child_bracket"]=((df_fiscr_2010_to_2014.ZSALM_2011>52000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([41,42,43])))
df_fiscr_2010_to_2014["Control_group_over_3_child_bracket"]=((df_fiscr_2010_to_2014.ZSALM_2011>76000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([41,42,43])))

In [ ]:
df_fiscr_2010_to_2014["Net_Control_group_over_1_child_bracket"]=((df_fiscr_2010_to_2014.ZSALM_2011>41000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([42,43])))
df_fiscr_2010_to_2014["Net_Control_group_over_2_child_bracket"]=((df_fiscr_2010_to_2014.ZSALM_2011>52000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([41,43])))
df_fiscr_2010_to_2014["Net_Control_group_over_3_child_bracket"]=((df_fiscr_2010_to_2014.ZSALM_2011>76000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([41,42])))

In [ ]:
df_fiscr_2010_to_2014["Ctrl_grp_1"] = df_fiscr_2010_to_2014.Control_group_1_child_bracket.astype('float', inplace = True)
df_fiscr_2010_to_2014["Ctrl_grp_2"] = df_fiscr_2010_to_2014.Control_group_2_child_bracket.astype('float', inplace = True)
df_fiscr_2010_to_2014["Ctrl_grp_3"] = df_fiscr_2010_to_2014.Control_group_3_child_bracket.astype('float', inplace = True)

In [ ]:
df_fiscr_2010_to_2014["Ctrl_over_grp_1"] = df_fiscr_2010_to_2014.Control_group_over_1_child_bracket.astype('float', inplace = True)
df_fiscr_2010_to_2014["Ctrl_over_grp_2"] = df_fiscr_2010_to_2014.Control_group_over_2_child_bracket.astype('float', inplace = True)
df_fiscr_2010_to_2014["Ctrl_over_grp_3"] = df_fiscr_2010_to_2014.Control_group_over_3_child_bracket.astype('float', inplace = True)

In [ ]:
df_fiscr_2010_to_2014["Net_Ctrl_over_grp_1"] = df_fiscr_2010_to_2014.Net_Control_group_over_1_child_bracket.astype('float', inplace = True)
df_fiscr_2010_to_2014["Net_Ctrl_over_grp_2"] = df_fiscr_2010_to_2014.Net_Control_group_over_2_child_bracket.astype('float', inplace = True)
df_fiscr_2010_to_2014["Net_Ctrl_over_grp_3"] = df_fiscr_2010_to_2014.Net_Control_group_over_3_child_bracket.astype('float', inplace = True)

In [ ]:
df_fiscr_2010_to_2014['Diff_income'] = df_fiscr_2010_to_2014.ZSALM_2014- df_fiscr_2010_to_2014.ZSALM_2010

In [ ]:
df_fiscr_2010_to_2014['Comp_fam_1enf'] = df_fiscr_2010_to_2014.TYPMEN9_2011.isin([41]).astype('float', inplace = True)
df_fiscr_2010_to_2014['Comp_fam_2enf'] = df_fiscr_2010_to_2014.TYPMEN9_2011.isin([42]).astype('float', inplace = True)
df_fiscr_2010_to_2014['Comp_fam_3enf'] = df_fiscr_2010_to_2014.TYPMEN9_2011.isin([43]).astype('float', inplace = True)

In [ ]:
df_fiscr_2010_to_2014['Comp_fam_1enf'].value_counts(dropna = False)

In [ ]:
df_fiscr_2010_to_2014['Comp_fam_1enf'].value_counts(dropna = False)

Variables d'interation

In [ ]:
df_fiscr_2010_to_2014["Ctrl_grp_1_times_grp1"] = df_fiscr_2010_to_2014["Ctrl_grp_1"]* df_fiscr_2010_to_2014["Groupe_1"]
df_fiscr_2010_to_2014["Ctrl_grp_2_times_grp1"] = df_fiscr_2010_to_2014["Ctrl_grp_2"] * df_fiscr_2010_to_2014["Groupe_1"]
df_fiscr_2010_to_2014["Ctrl_grp_3_times_grp1"] = df_fiscr_2010_to_2014["Ctrl_grp_3"] *  df_fiscr_2010_to_2014["Groupe_1"]


### de la merde
df_fiscr_2010_to_2014["Ctrl_grp_1_times_grp1_child1"] = (df_fiscr_2010_to_2014["Ctrl_grp_1"]* 
                                                         df_fiscr_2010_to_2014["Groupe_1"] * df_fiscr_2010_to_2014['Comp_fam_1enf'])
df_fiscr_2010_to_2014["Ctrl_grp_2_times_grp1_child2"] = (df_fiscr_2010_to_2014["Ctrl_grp_2"]
                                                  * df_fiscr_2010_to_2014["Groupe_2"] * df_fiscr_2010_to_2014['Comp_fam_2enf'])
df_fiscr_2010_to_2014["Ctrl_grp_2_times_grp1_child3"] = (df_fiscr_2010_to_2014["Ctrl_grp_3"] *  df_fiscr_2010_to_2014["Groupe_2"]
                                                * df_fiscr_2010_to_2014['Comp_fam_3enf'])

#########

df_fiscr_2010_to_2014["Ctrl_grp_1_not_treated"] = (df_fiscr_2010_to_2014["Ctrl_grp_1"]) *  (~df_fiscr_2010_to_2014["Groupe_1"])
df_fiscr_2010_to_2014["Ctrl_grp_2_not_treated"] = (df_fiscr_2010_to_2014["Ctrl_grp_2"]) *  (~df_fiscr_2010_to_2014["Groupe_1"])
df_fiscr_2010_to_2014["Ctrl_grp_3_not_treated"] = (df_fiscr_2010_to_2014["Ctrl_grp_3"])  * (~df_fiscr_2010_to_2014["Groupe_1"])



df_fiscr_2010_to_2014["Ctrl_grp_1_not_treated"] = (df_fiscr_2010_to_2014["Ctrl_grp_1"]) *  (~df_fiscr_2010_to_2014["Groupe_1"])
df_fiscr_2010_to_2014["Ctrl_grp_2_not_treated"] = (df_fiscr_2010_to_2014["Ctrl_grp_2"]) *  (~df_fiscr_2010_to_2014["Groupe_1"])
df_fiscr_2010_to_2014["Ctrl_grp_3_not_treated"] = (df_fiscr_2010_to_2014["Ctrl_grp_3"])  * (~df_fiscr_2010_to_2014["Groupe_1"])



df_fiscr_2010_to_2014["Ctrl_2child_not_treated2"] = (df_fiscr_2010_to_2014["Ctrl_grp_1"]) *  (~df_fiscr_2010_to_2014["Groupe_2"])
df_fiscr_2010_to_2014["Ctrl_3_not_treated2"] = (df_fiscr_2010_to_2014["Ctrl_grp_2"]) *  (~df_fiscr_2010_to_2014["Groupe_2"])
df_fiscr_2010_to_2014["Ctrl_grp_3_not_treated2"] = (df_fiscr_2010_to_2014["Ctrl_grp_3"])  * (~df_fiscr_2010_to_2014["Groupe_2"])


In [ ]:
df_fiscr_2010_to_2014["Ctrl_grp_1_times_grp1"].value_counts()

In [ ]:
df_fiscr_2010_to_2014["Ctrl_grp_2_times_grp1_child2"].value_counts()

In [ ]:
#Fake groups for robustness check
df_fiscr_2010_to_2014["Control_group_1_child_bracket"]=((df_fiscr_2010_to_2014.ZSALM_2011>36000) & (df_fiscr_2010_to_2014.ZSALM_2011<41000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([41,42,43])))


df_fiscr_2010_to_2014["Control_group_2_child_bracket"]= ((df_fiscr_2010_to_2014.ZSALM_2011>41000) & (df_fiscr_2010_to_2014.ZSALM_2011<52000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([41,42,43])))

df_fiscr_2010_to_2014["Control_group_3_child_bracket"]=((df_fiscr_2010_to_2014.ZSALM_2011>53000) & (df_fiscr_2010_to_2014.ZSALM_2011<76000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([43])))

df_fiscr_2010_to_2014["Control_group_over_1_child_bracket"]=((df_fiscr_2010_to_2014.ZSALM_2011>41000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([41,42,43])))
df_fiscr_2010_to_2014["Control_group_over_2_child_bracket"]=((df_fiscr_2010_to_2014.ZSALM_2011>52000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([41,42,43])))
df_fiscr_2010_to_2014["Control_group_over_3_child_bracket"]=((df_fiscr_2010_to_2014.ZSALM_2011>76000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([41,42,43])))


df_fiscr_2010_to_2014["Net_Control_group_over_1_child_bracket"]=((df_fiscr_2010_to_2014.ZSALM_2011>41000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([42,43])))
df_fiscr_2010_to_2014["Net_Control_group_over_2_child_bracket"]=((df_fiscr_2010_to_2014.ZSALM_2011>52000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([41,43])))
df_fiscr_2010_to_2014["Net_Control_group_over_3_child_bracket"]=((df_fiscr_2010_to_2014.ZSALM_2011>76000) 
&(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([41,42])))

df_fiscr_2010_to_2014["Ctrl_grp_1"] = df_fiscr_2010_to_2014.Control_group_1_child_bracket.astype('float', inplace = True)
df_fiscr_2010_to_2014["Ctrl_grp_2"] = df_fiscr_2010_to_2014.Control_group_2_child_bracket.astype('float', inplace = True)
df_fiscr_2010_to_2014["Ctrl_grp_3"] = df_fiscr_2010_to_2014.Control_group_3_child_bracket.astype('float', inplace = True)

df_fiscr_2010_to_2014["Ctrl_over_grp_1"] = df_fiscr_2010_to_2014.Control_group_over_1_child_bracket.astype('float', inplace = True)
df_fiscr_2010_to_2014["Ctrl_over_grp_2"] = df_fiscr_2010_to_2014.Control_group_over_2_child_bracket.astype('float', inplace = True)
df_fiscr_2010_to_2014["Ctrl_over_grp_3"] = df_fiscr_2010_to_2014.Control_group_over_3_child_bracket.astype('float', inplace = True)

df_fiscr_2010_to_2014["Net_Ctrl_over_grp_1"] = df_fiscr_2010_to_2014.Net_Control_group_over_1_child_bracket.astype('float', inplace = True)
df_fiscr_2010_to_2014["Net_Ctrl_over_grp_2"] = df_fiscr_2010_to_2014.Net_Control_group_over_2_child_bracket.astype('float', inplace = True)
df_fiscr_2010_to_2014["Net_Ctrl_over_grp_3"] = df_fiscr_2010_to_2014.Net_Control_group_over_3_child_bracket.astype('float', inplace = True)


df_fiscr_2010_to_2014["Ctrl_grp_1_times_grp1"] = df_fiscr_2010_to_2014["Ctrl_grp_1"]* df_fiscr_2010_to_2014["Groupe_1"]
df_fiscr_2010_to_2014["Ctrl_grp_2_times_grp1"] = df_fiscr_2010_to_2014["Ctrl_grp_2"] * df_fiscr_2010_to_2014["Groupe_1"]
df_fiscr_2010_to_2014["Ctrl_grp_3_times_grp1"] = df_fiscr_2010_to_2014["Ctrl_grp_3"] *  df_fiscr_2010_to_2014["Groupe_1"]


### de la merde
df_fiscr_2010_to_2014["Ctrl_grp_1_times_grp1_child1"] = (df_fiscr_2010_to_2014["Ctrl_grp_1"]* 
                                                         df_fiscr_2010_to_2014["Groupe_1"] * df_fiscr_2010_to_2014['Comp_fam_1enf'])
df_fiscr_2010_to_2014["Ctrl_grp_2_times_grp1_child2"] = (df_fiscr_2010_to_2014["Ctrl_grp_2"]
                                                  * df_fiscr_2010_to_2014["Groupe_2"] * df_fiscr_2010_to_2014['Comp_fam_2enf'])
df_fiscr_2010_to_2014["Ctrl_grp_2_times_grp1_child3"] = (df_fiscr_2010_to_2014["Ctrl_grp_3"] *  df_fiscr_2010_to_2014["Groupe_2"]
                                                * df_fiscr_2010_to_2014['Comp_fam_3enf'])

#########

df_fiscr_2010_to_2014["Ctrl_grp_1_not_treated"] = (df_fiscr_2010_to_2014["Ctrl_grp_1"]) *  (~df_fiscr_2010_to_2014["Groupe_1"])
df_fiscr_2010_to_2014["Ctrl_grp_2_not_treated"] = (df_fiscr_2010_to_2014["Ctrl_grp_2"]) *  (~df_fiscr_2010_to_2014["Groupe_1"])
df_fiscr_2010_to_2014["Ctrl_grp_3_not_treated"] = (df_fiscr_2010_to_2014["Ctrl_grp_3"])  * (~df_fiscr_2010_to_2014["Groupe_1"])



df_fiscr_2010_to_2014["Ctrl_grp_1_not_treated"] = (df_fiscr_2010_to_2014["Ctrl_grp_1"]) *  (~df_fiscr_2010_to_2014["Groupe_1"])
df_fiscr_2010_to_2014["Ctrl_grp_2_not_treated"] = (df_fiscr_2010_to_2014["Ctrl_grp_2"]) *  (~df_fiscr_2010_to_2014["Groupe_1"])
df_fiscr_2010_to_2014["Ctrl_grp_3_not_treated"] = (df_fiscr_2010_to_2014["Ctrl_grp_3"])  * (~df_fiscr_2010_to_2014["Groupe_1"])



df_fiscr_2010_to_2014["Ctrl_2child_not_treated2"] = (df_fiscr_2010_to_2014["Ctrl_grp_1"]) *  (~df_fiscr_2010_to_2014["Groupe_2"])
df_fiscr_2010_to_2014["Ctrl_3_not_treated2"] = (df_fiscr_2010_to_2014["Ctrl_grp_2"]) *  (~df_fiscr_2010_to_2014["Groupe_2"])
df_fiscr_2010_to_2014["Ctrl_grp_3_not_treated2"] = (df_fiscr_2010_to_2014["Ctrl_grp_3"])  * (~df_fiscr_2010_to_2014["Groupe_2"])


In [ ]:
reaction_store_path = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2015/Data/hdf/reaction.h5'

In [ ]:
%%time
df_fiscr_2010_to_2014.to_hdf(reaction_store_path, 'reaction_fiscr')

In [ ]:
df_fiscr_2010_to_2014 = pd.read_hdf(reaction_store_path,  'reaction_fiscr' )

On selectionne que les menages traites

In [ ]:
%%time
df_selct = df_fiscr_2010_to_2014.iloc[0:100000][(df_fiscr_2010_to_2014.iloc[0:100000].TYPMEN9_2011.isin([41,42,43]))]

In [ ]:
%%time
df_selct = df_fiscr_2010_to_2014.iloc[0:100000][(df_fiscr_2010_to_2014.iloc[0:100000].TYPMEN9_2011.isin([41,42,43]))]

In [ ]:
%%time
df_selct = df_fiscr_2010_to_2014.iloc[0:100000].loc[(df_fiscr_2010_to_2014.iloc[0:100000].TYPMEN9_2011.isin([41,42,43]))]

In [ ]:
%%time
df_selct = df_fiscr_2010_to_2014.loc[(df_fiscr_2010_to_2014.TYPMEN9_2011.isin([41,42,43]))]
df_selct = df_selct.loc[(df_selct.ZSALM_2011>0) & (df_selct.ZSALM_2014>0)]

In [ ]:
%%time
df_selct.to_hdf(reaction_store_path, 'fisci_fiscr_2010_2014')

In [ ]:
df_selct.shape

In [ ]:
df_fiscr_2010_to_2014 = pd.read_hdf(reaction_store_path,  'fisci_fiscr_2010_2014' )

In [ ]:
%%time
pd.stats.ols.OLS(df_selct['Diff_income'], df_selct[["Groupe_2", "Groupe_1", ]
                                                   + ["Ctrl_grp_1","Ctrl_grp_2","Ctrl_grp_3"]
                                                    + ["ZIMPOM_2014"]
                                                   +['Comp_fam_1enf', 'Comp_fam_2enf', 'Comp_fam_3enf',]
                                                   ] )

In [ ]:
df_selct_2 = df_selct.loc[df_selct.ZSALM_2011>2000*12]

In [ ]:
import statsmodels.api as sm

In [ ]:
import statsmodels.formula as statformula

In [ ]:
X =  df_selct_2[["Groupe_2", "Groupe_1",]
                                                   + ["Ctrl_grp_1","Ctrl_grp_2","Ctrl_grp_3"]
                                                    + ["ZIMPOM_2014"]
                                                   +['Comp_fam_1enf', 'Comp_fam_2enf', 'Comp_fam_3enf']
                                                   ]
X = sm.add_constant(X)
hello = statformula.api.OLS(df_selct_2['Diff_income'], X)
hello = hello.fit()
hello.summary()

In [ ]:
summary =  hello.summary()

In [ ]:
table = summary.tables[1]

In [ ]:
table.data[3] = ['Groupe_18LOL',
 '  691.1210',
 '  149.856',
 '    4.612',
 ' 0.000',
 '  397.407   984.835']

In [ ]:
statformula.api.OLS()

In [ ]:
X =  df_selct_2[["Groupe_2", "Groupe_1",]
                                                  # + ["Ctrl_grp_1","Ctrl_grp_2","Ctrl_grp_3"]
                                                    +  ["Ctrl_grp_1_not_treated","Ctrl_grp_2_not_treated","Ctrl_grp_3_not_treated"]
                                                    + ["ZIMPOM_2014"]
                                                   +['Comp_fam_1enf', 'Comp_fam_2enf', 'Comp_fam_3enf']
                                                   ]
X = sm.add_constant(X)
hello = statformula.api.OLS(df_selct_2['Diff_income'], X)
hello = hello.fit()
hello.summary()

In [ ]:
df_selct_2['not_"Ctrl_grp_1_times_grp1_child1'] = np.abs(df_selct_2['"Ctrl_grp_1_times_grp1_child1"']-1)

In [ ]:
X =  df_selct_2[["Groupe_2", "Groupe_1",]
                                                   + ["Ctrl_grp_1","Ctrl_grp_2","Ctrl_grp_3"]
                                                    + [ "Ctrl_grp_2_times_grp1_child2",
                                                       'Ctrl_grp_2_times_grp1_child3']
                                                    + ["ZIMPOM_2014"]
                                                   +['Comp_fam_1enf', 'Comp_fam_2enf', 'Comp_fam_3enf']
                                                   ]
X = sm.add_constant(X)
hello = statformula.api.OLS(df_selct_2['Diff_income'], X)
hello = hello.fit()
hello.summary()

In [ ]:
X =  df_selct_2[["Groupe_2", "Groupe_1",]
                                                   + ["Ctrl_grp_1","Ctrl_grp_2","Ctrl_grp_3"]
                                                    + [ "Ctrl_grp_1_not_treated", "Ctrl_grp_2_not_treated"
                                                       'Ctrl_grp_3_not_treated']
                                                    + ["ZIMPOM_2014"]
                                                   +['Comp_fam_1enf', 'Comp_fam_2enf', 'Comp_fam_3enf']
                                                   ]
X = sm.add_constant(X)
hello = statformula.api.OLS(df_selct_2['Diff_income'], X)
hello = hello.fit()
hello.summary()

In [ ]:
X =  df_selct_2[["Groupe_2", "Groupe_1",]
                                                   #+ ["Ctrl_grp_1","Ctrl_grp_2","Ctrl_grp_3"]
                                                    + [ "Ctrl_grp_1_not_treated", "Ctrl_grp_2_not_treated"
                                                       'Ctrl_grp_3_not_treated']
                                                    + ["ZIMPOM_2014"]
                                                   +['Comp_fam_1enf', 'Comp_fam_2enf', 'Comp_fam_3enf']
                                                   ]
X = sm.add_constant(X)
hello = statformula.api.OLS(df_selct_2['Diff_income'], X)
hello = hello.fit()
hello.summary()

In [ ]:
df_selct_2.Groupe_1_3enfants

In [ ]:
X =  df_selct_2[["Groupe_2_1_enfant", "Groupe_2_2enfants","Groupe_2_3enfants"]
                +['Groupe_1_enfant','Groupe_1_2_enfants','Groupe_1_3enfants']
                                                   + ["Ctrl_grp_1","Ctrl_grp_2","Ctrl_grp_3"]
                                                    + ["ZIMPOM_2014"]
                                                   +['Comp_fam_1enf', 'Comp_fam_2enf', 'Comp_fam_3enf']
                                                   ]
X = sm.add_constant(X)
hello = statformula.api.OLS(df_selct_2['Diff_income'], X)
hello = hello.fit()
hello.summary()

In [ ]:
%%time
pd.stats.ols.OLS(df_selct_2['Diff_income'], df_selct_2[["Groupe_2", "Groupe_1", "ZSALM_2011", "ZSALM_2014"]
                                                   + ["Ctrl_grp_1","Ctrl_grp_2","Ctrl_grp_3"]
                                                    + ["ZIMPOM_2014"]
                                                   +['Comp_fam_1enf', 'Comp_fam_2enf', 'Comp_fam_3enf']
                                                   ] )

In [ ]:
%%time
pd.stats.ols.OLS(df_selct_2['ZSALM_2014'], df_selct_2[["Groupe_2", "Groupe_1", "ZSALM_2011"]
                                                   + ["Ctrl_grp_1","Ctrl_grp_2","Ctrl_grp_3"]
                                                    + ["ZIMPOM_2014"]
                                                   +['Comp_fam_1enf', 'Comp_fam_2enf', 'Comp_fam_3enf']
                                                   ] )

In [ ]:
df_selct.Comp_fam_1enf.value_counts()

In [ ]:
pd.stats.ols.OLS(df_selct['ZSALM_2014']/df_selct['ZSALM_2011'],
                df_selct[["Groupe_2", "Groupe_1", "ZSALM_2011", "ZSALM_2014"]
                                                   + ["Ctrl_grp_1","Ctrl_grp_2","Ctrl_grp_3"]
                                                    + ["ZIMPOM_2014"]
                                                   +['Comp_fam_1enf', 'Comp_fam_2enf', 'Comp_fam_3enf']
                                                   ] )

In [ ]:
pd.stats.ols.OLS(diff_income, df_fiscr_2010_to_2014[["Groupe_2", "Groupe_1", "ZSALM_2011", "ZSALM_2014"]
                                                   + ["Ctrl_grp_1","Ctrl_grp_2","Ctrl_grp_3"]
                                                    + ["ZIMPOM_2014"]
                                                   ] )

In [ ]:
pd.stats.ols.OLS(diff_income, df_fiscr_2010_to_2014[["Groupe_2", "Groupe_1", "ZSALM_2011", "ZSALM_2014"]] )

In [ ]:
pd.stats.ols.OLS(diff_income, df_fiscr_2010_to_2014[["Groupe_2", "Groupe_1", "ZSALM_2014"]] )

In [ ]:
pd.stats.ols.OLS(pd.Series(diff_income.iloc[:50]), df_fiscr_2010_to_2014[["Groupe_2", "Groupe_1"]].iloc[0:50] )

In [ ]:
(diff_income.iloc[:50])

In [ ]:
%%time
df_panact = pd.read_hdf('C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2015/Data/hdf/dads','start_2005')

In [ ]:
df_panact.head()

In [ ]:
%%time
df_panact[:500000].NBHEUR.plot(kind='hist', bins = 200)

In [ ]:
df_panact.NETNETR.mean()/12

In [ ]:
df_panact.head()

In [ ]:
df_fisci = load_fisc_i_by_year(2013);

In [ ]:
print df_fisci.AN_FISC.value_counts()

In [ ]:
id_diff_fisci =  df_fisci.ID_DIFF.unique()

In [ ]:
id_diff_panact = df_panact.ID_DIFF.unique()

In [ ]:
df_panact_2013 = df_panact[df_panact.AN == 2013]

In [ ]:
id_diff_panact_2013 = df_panact_2013.ID_DIFF.unique()

In [ ]:
#del df_panact

In [ ]:
%time
id_diff_panact = set(id_diff_panact)

In [ ]:
%time
id_diff_panact_2013 = set(id_diff_panact_2013)

In [ ]:
%time
id_diff_fisci = set(id_diff_fisci)

In [ ]:
id_diff_intersect = id_diff_fisci.intersection(id_diff_panact)

In [ ]:
len(id_diff_intersect)/10**6

In [ ]:
id_diff_intersect_2013 = id_diff_fisci.intersection(id_diff_panact_2013)

In [ ]:
len(id_diff_intersect_2013)/10**6

In [ ]:
df_panact_2013.ID_DIFF.nunique()

In [ ]:
df_panact_2013

Voir si on retrouve les revenus individuels entre panact et revdet

In [ ]:
df_fiscrevdet = load_df_fiscrevdet_by_year(2013)

In [ ]:
df_fiscrevdet.ID_FISC_FOY_DIFF.head(2)

In [ ]:
df_fiscrevdet.TYPE_FISC.value_counts()

In [ ]:
df_fisci = df_fisci[df_fisci.TYPE_FISC.isin(['1','2','3','4','5','6'])]

In [ ]:
df_fisci['TYPE_FISC'] = df_fisci.TYPE_FISC.astype('float')

In [ ]:
df_fisci_revdet = df_fisci.merge(df_fiscrevdet, on =['ID_FISC_FOY_DIFF','ORDREFIP', 'TYPE_FISC'])

In [ ]:
df_fiscrevdet.columns

In [ ]:
assert np.all(df_fisci_revdet.AN_FISC_x == df_fisci_revdet.AN_FISC_y)

In [ ]:
df_fisci_revdet_to_merge = df_fisci_revdet[["ID_DIFF","YSALI"]]

In [ ]:
del df_fisci_revdet

In [ ]:
del df_fisci

In [ ]:
#panact_2014 = df_panact[df_panact.AN == 2013] ; 

In [ ]:
del df_panact

In [ ]:
df_fisci_revdet_to_merge[df_fisci_revdet_to_merge.ID_DIFF == 4855]

In [ ]:
df_fisci_revdet_to_merge.shape

In [ ]:
fsic_panact_merge = df_panact_2013.merge(df_fisci_revdet_to_merge, on = 'ID_DIFF')

In [ ]:
fsic_panact_merge[['YSALI','NETNET']]

In [ ]:
(fsic_panact_merge['YSALI']/fsic_panact_merge['NETNET']).describe()


Voir allègement fillon :

In [ ]:
((fsic_panact_merge[fsic_panact_merge.YSALI >30000]['YSALI']/ fsic_panact_merge[fsic_panact_merge.YSALI >30000]['NETNET'])>1.1
).value_counts()

In [ ]:
(fsic_panact_merge['YSALI']/fsic_panact_merge['NETNET']).describe()

In [ ]:
fsic_panact_merge.NBHEUR.value_counts()


Voir la différence du nombre d'heure d'une année sur l'autre du dernier décile.

In [ ]:
fsic_panact_merge[(((fsic_panact_merge['YSALI']/fsic_panact_merge['NETNET']))>0.9)&
(((fsic_panact_merge['YSALI']/fsic_panact_merge['NETNET']))<1.1)]